__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x38_d03_reorganisieren.ipynb)__

# Reorganisieren und Reinigen
* speichert in data_03
* lex_test level 3
* 41 min

## ToDo
* Nom-Merkmale == lemma löschen
* TODO_verwaiste_lemmas
* TODO_TAGS_IN_MERKMALE


* löschliste am Ende nochmal anwenden
* Prüfen: ändert sich am score in translate_lex_full noch was, wenn echte Statistikzahlen da mit einfließen? Es geht um die Zuordnung lex --> lemma bei gegebenem tagZZ, also z.B. bei 'Alben' (--> Alb, Album, etc.).
* Symmetrie: Es gibt Lexeme für das Lemma "Ackermann/NN". Die gleichen Lexeme sollte es auch für das Lemma "Ackermann/NE" geben.
* Der gleiche Fall triff auch beim Lexem "ertragene" auf. Das sollte nicht nur ein Lexem des Verbs sein, sondern auch ein Lexem des Adjektivs.
* Noch besser: Lemmas zusammenfassen und beide Tag-Felder nutzen

Steigerbare Adverbien prüfen (TODO_Steigerbare Adverbien):
* 'sehr', 'mehr','am meisten'
* 'gern', 'lieber','am liebsten'
* 'bald', 'eher','am ehesten'
* 'oft',  'öfter','am öftesten'  # drin
* 'wohl', 'besser','am besten'   # drin

## Spacy Nachschlagen 
* Kann das Lemma stimmen? >> Nachschlagen in wiktionary_merkmal (hier gibt es alle lex-lemma-Kombinationen, z.B. bei "Alben")
* Wenn nicht fündig geworden: Nachschlagen in translate_lex (hier gibt es nur 1 Lemma pro tagZZ)
* Resultat: Lemma

## Import

In [417]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 07:37:28


In [418]:
# pandas
if environment['in_colab']:
    !pip install --upgrade pandas
import pandas as pd

In [419]:
# alle anderen
try:
    import pandasklar as pak, bpyth as bpy, swifter

except ImportError as e:
    !pip install pandasklar swifter
    import pandasklar as pak, bpyth as bpy, swifter
    
grid       = pak.grid
from functools import partial
gridt = partial(grid, backend='dtale')    # dTale-Nutzung erzwungen
check_mask = pak.check_mask     


## Einstellungen

In [420]:
# Einstellungen

# Was debuggen?
suche_debug = ['besonnte','überlegte',]   
suche_debug = ['besonne',]  # auch gut: besinnen
suche_debug = ['inner-','innere','inner']  
suche_debug = ['Citrone',]  
suche_debug = ['Dezember','December','Citrone',]  
suche_debug = ['Anna',]  
# 'einen','beigen','Montage',
# '24/7','Schriftwerk'
suche_debug = ['gefriergetrocknet','gefriertrocknen','missraten']  
suche_debug = ['calvinistisch','kalvinistisch',]  


suche_debug = ['Euro','€','tugendhaftestes','zu viele','zuviele', 'zu viel']  


gründlich = True # einige Arbeitsschritte können weggelassen werden, wenn gründlich = True

# verbose
pak.Config.set('VERBOSE', True)

# Darstellung der Dataframes
pak.Config.set('GRID_BACKEND', '')   # standardmäßig kein dTale
pd.set_option('max_colwidth', 2000)  # aber dafür breiter

# Load
verzeichnis_load                       = 'data_02'
wiktionary_lemma_filename              = verzeichnis_load + '/wiktionary_lemma.pickle'  
translate_lex_filename                 = verzeichnis_load + '/wiktionary_nolemma.pickle'    # hieß vorher wiktionary_nolemma
wiktionary_merkmal_filename            = verzeichnis_load + '/wiktionary_merkmal.pickle'  
wiktionary_merkmal_text_filename       = verzeichnis_load + '/wiktionary_merkmal_text.pickle'  

# Steuertabellen
lex_test_filename                      = 'Steuertabellen/lex_test.xlsx'  
löschliste_filename                    = 'Steuertabellen/löschliste.xlsx'   
lexeme_manuell_filename                = 'Steuertabellen/lexeme_manuell.xlsx'                              # manuell eingepflegte Lexeme 
     
# Save
verzeichnis_save                       = 'data_03'
wiktionary_lemma_filename_save         = verzeichnis_save + '/wiktionary_lemma.pickle'  
wiktionary_lemma_fix_filename_save     = verzeichnis_save + '/wiktionary_lemma_fix.pickle'  
translate_lex_filename_save            = verzeichnis_save + '/translate_lex.pickle'  
translate_lex_trash_filename_save      = verzeichnis_save + '/translate_lex_trash.pickle'  
translate_lemma_filename_save          = verzeichnis_save + '/translate_lemma.pickle'  
wiktionary_merkmal_filename_save       = verzeichnis_save + '/wiktionary_merkmal.pickle'  
wiktionary_merkmal_text_filename_save  = verzeichnis_save + '/wiktionary_merkmal_text.pickle'  

VERBOSE = True
--> setting verbose=True as default for all pandasklar functions

GRID_BACKEND = 
--> setting backend= as default for all pandasklar functions



## Vorbereiten

In [421]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

In [422]:
# Steuertabellen und Python-Files von Github herunterladen
import os
if not os.path.exists('Steuertabellen'):
   !git clone https://github.com/djekra/wiktionary_wortschatz.git
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/Steuertabellen Steuertabellen
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/*.py .
#   !rm -r wiktionary_wortschatz

In [423]:
# Lade Ergebnisse der vorherigen Verarbeitungsstufe von Google Drive, falls nicht lokal vorhanden
# Dieses Verzeichnis wurde in der vorherigen Verarbeitungsstufe angelegt 
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av /content/drive/MyDrive/{verzeichnis_load} .

In [424]:
# Weitere Imports
from Steuertabellen.s01_Steuertabellen import *
from x32_Funktionen                    import *

## Daten laden
* Steuertabellen
* wiktionary_lemma
* translate_lex
* wiktionary_merkmal     
* wiktionary_merkmal_text 
* jeweils einen Trash generieren

In [425]:
#blab.help(plan_merkmal_erstellen)

In [426]:
# plan_merkmal Steuertabelle
plan_merkmal = plan_merkmal_erstellen()
grid(plan_merkmal, rows=9, column_definitions={ 'collect': {'maxWidth': 50,}, ' ': {'maxWidth': 50,}                   },     )

103 rows


,merkmal,template,collect,plan,sort,is_lex
0,noSteig,kSt.,E,1,,False
1,noSing,kSg.,E,1,,False
2,nurWortart,Wortart fehlt,P,1,,False
3,istName,Navigationsleiste Anthroponyme,E,1,,False
4,Wikispecies,Wikispecies,E,1,,False
...,...,...,...,...,...,...
98,Superlativ,,,3,x3 Superlativ,True
99,Stamm,,,3,z1 Stamm,False
100,ipa,,,3,z1 ipa,False
101,noFlex,,E,3,z1 noFlex,True


In [427]:
a= pak.analyse_freqs(plan_merkmal,['plan','merkmal'])
grid(a, width=200)

3 rows


,plan,plan_count,plan_percent,merkmal,merkmal_count
0,3,59,57.3,"[Befehl du, Befehl ihr, Gerundivum, PTKVZ, VVIZU, lemma, lex, lexAlt, rhymes, Genus, Nom Sg, Nom Sg f, Nom Sg m, Nom Sg n, Gen Sg, Gen Sg f, Gen Sg m, Gen Sg n, Dat Sg, Dat Sg f]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,1,38,36.9,"[übersicht, noSteig, noSing, nurWortart, istName, Wikispecies, Wikivoyage, Taxonomie, Präfix, Suffix, VMaßeinheiten, def, herkunft, syn, lateral, gegen, klein, up, up_part, down]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,2,6,5.8,"[def, herkunft, bsp, bsp_re, bsp_gw, bsp_sp]","[1, 1, 1, 1, 1, 1]"


In [428]:
# lex_test, lemma_test
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=3)
grid(lex_test)

180 rows loaded
Delete 3 rows from 180
177 rows


,level,lex,lemma,lex_tag_soll,notiz,check_tag
0,1,…,…,$(,,
1,3,°,°,$(,,
2,3,€,€,NN,,
3,1,Ackermann,Ackermann,NE,,
5,1,Alben,Album,NN,,
...,...,...,...,...,...,...
175,3,zugrunde liegen,zugrunde liegen,VVFIN,,
176,1,zurecht,zurecht,PTKVZ,,
177,3,zuviele,zuviele,ADV,,
178,3,zu viel,zu viel,ADV,,


In [429]:
# lexeme_manuell laden
lexeme_manuell = pak.load_excel( lexeme_manuell_filename, tabcol='tab' )
lexeme_manuell = pak.rename_col( lexeme_manuell, 'tag',    'lemma_tag' )
lexeme_manuell = pak.rename_col( lexeme_manuell, 'tag_lex','lex_tag' )
lexeme_manuell = pak.move_cols(  lexeme_manuell, 'lex_tag','lemma_tag')
pak.sample(lexeme_manuell)

1794 rows loaded


,lex,lemma,lemma_tag,lex_tag,member,genus,tab
0,abberufen,abberufen,VVFIN,Ptz2,<NA>,<NA>,Partizipien
164,gewußt,wissen,VVFIN,Ptz2,<NA>,<NA>,Partizipien
304,überzeugend,überzeugen,VVFIN,Ptz1,<NA>,<NA>,Partizipien
487,Kids,Kid,NN,<NA>,<NA>,n,Diverses
489,Kontraindikationen,Kontraindikation,NN,<NA>,<NA>,f,Diverses
530,vergleiche,vergleichen,VVFIN,VVIMP,<NA>,<NA>,Diverses
561,zu viele,zu viel,ADV,ADV,<NA>,<NA>,Diverses
565,<NA>,24/7,ADJA,<NA>,<NA>,<NA>,Diverses
585,<NA>,bpm,NN,<NA>,Einheit,<NA>,Diverses
721,❀,_,$(,<NA>,<NA>,<NA>,Symbole


In [430]:
# Laden     
wiktionary_lemma         = pak.load_pickle(wiktionary_lemma_filename)
translate_lex            = pak.load_pickle(translate_lex_filename)
wiktionary_merkmal       = pak.load_pickle(wiktionary_merkmal_filename)
wiktionary_merkmal_text  = pak.load_pickle(wiktionary_merkmal_text_filename)

171220 rows loaded
667548 rows loaded
4838893 rows loaded
657508 rows loaded


In [431]:
# rename_col
wiktionary_merkmal_text = pak.rename_col(wiktionary_merkmal_text,'section_id2','lemma_id')
translate_lex           = pak.rename_col(translate_lex,'lex','data')
#translate_lex           = pak.rename_col(translate_lex,'score','data_score')

In [432]:
# trash erzeugen     
wiktionary_lemma_trash        = wiktionary_lemma.head(0)
wiktionary_lemma_trash        = pak.write_empty_col(wiktionary_lemma_trash, 'msg', 'string')

translate_lex_trash           = translate_lex.head(0)
translate_lex_trash           = pak.write_empty_col(translate_lex_trash, 'msg', 'string')

wiktionary_merkmal_trash      = wiktionary_merkmal.head(0)
wiktionary_merkmal_trash      = pak.write_empty_col(wiktionary_merkmal_trash, 'msg', 'string')

wiktionary_merkmal_text_trash = wiktionary_merkmal_text.head(0)
wiktionary_merkmal_text_trash = pak.write_empty_col(wiktionary_merkmal_text_trash, 'msg', 'string')

## suche_debug

In [433]:
# wiktionary_lemma suche_debug
grid(search_str(wiktionary_lemma, suche_debug))

3 rows


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
453,Euro,11127,0,Euro,euro,True,1,NN,NN,sCiXEFydRB,NOM,N,Einheit manuell,,1403.598999
2607,zu_viel,89575,0,zu viel,zu viel,False,2,ADV,ADV,xNV7xeMT9x,ADV,A,Wortverbindung manuell,,51.254711
29704,Euro_1,11127,1,Euro,euro,True,1,NN,NN,BYgDx5pPHJ,NOM,N,,,36.994484


In [434]:
# wiktionary_lemma_trash
mask = wiktionary_lemma_trash.lemma.isin(suche_debug) #== 'gefriergetrocknet'
wiktionary_lemma_trash[mask]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg


In [435]:
pak.analyse_freqs(wiktionary_lemma_trash,['msg','lemma_id'])

'No rows'

In [436]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

3 rows


,nolemma_id,data,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score,lemma_id,lemma,lemma_lower,lemma_tagZZ
13852,Euros,Euros,LEX_A,LEX_A,4Lg01EzDZP,,,Deklinierte_Form,0.908025,Euro,Euro,euro,N
60439,tugendhaftestes,tugendhaftestes,LEX_A,LEX_A,cJwt4iRVUK,,,Deklinierte_Form,1.194590,tugendhaft,tugendhaft,tugendhaft,A
667529,zu_viele,zu viele,TODO,TODO,fvpCzBBWAJ,X_todo,X,Wortverbindung,1.020573,zu_viel,zu viel,zu viel,A


In [437]:
# wiktionary_merkmal
mask = wiktionary_merkmal.lemma_id == suche_debug[0]
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

35 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
3036866,Euro,Akk Pl,Euro,Euro,,NN,,,153.715988,True
3033702,Euro,Akk Pl,Euros,Euros,,NN,,,153.715988,True
2584202,Euro,Akk Sg,Euro,Euro,,NN,,,153.715988,True
2951785,Euro,Dat Pl,Euro,Euro,,NN,,,153.715988,True
2951094,Euro,Dat Pl,Euros,Euros,,NN,,,153.715988,True
2506856,Euro,Dat Sg,Euro,Euro,,NN,,,153.715988,True
2865325,Euro,Gen Pl,Euro,Euro,,NN,,,153.715988,True
2863829,Euro,Gen Pl,Euros,Euros,,NN,,,153.715988,True
2384024,Euro,Gen Sg,Euro,Euro,,NN,,,153.715988,True
2446725,Euro,Gen Sg,Euros,Euros,,NN,,,153.715988,True


In [438]:
# wiktionary_merkmal
mask = wiktionary_merkmal.data.isin(suche_debug) #== 'gefriergetrocknet'
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

67 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
1830808,zu_viel,ADV,zu_viele,zu viele,,ADV,,manuell,14.845442,True
3036866,Euro,Akk Pl,Euro,Euro,,NN,,,153.715988,True
2584202,Euro,Akk Sg,Euro,Euro,,NN,,,153.715988,True
2576854,Euro_1,Akk Sg,Euro,Euro,,NN,,,4.110498,True
2951785,Euro,Dat Pl,Euro,Euro,,NN,,,153.715988,True
...,...,...,...,...,...,...,...,...,...,...
1668969,zurücktauschen,kombi,Euro,Euro,,VVFIN,1,,53.270210,False
1353679,zustecken,kombi,Euro,Euro,,VVFIN,1,,68.361298,False
1320515,zuverdienen,kombi,Euro,Euro,,VVFIN,1,,46.501858,False
525704,zweihundert,kombi,Euro,Euro,,CARD,1,zweihundert,66.969849,False


In [439]:
# wiktionary_merkmal
# data_id ist bei allen gm-Merkmalen potentiell falsch!
mask1 = wiktionary_merkmal.data == 'besonnen'
mask2 = wiktionary_merkmal.merkmal == 'gm'
mask = mask1  &  mask2
a = wiktionary_merkmal[mask].sort_values(['merkmal','lemma_id','data'])
grid(a)

29 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
1526920,besonn,gm,besonnen,besonnen,,,,,1.333814,False
1526922,besonne,gm,besonnen,besonnen,,,,,1.644078,False
1526928,besonnend,gm,besonnen,besonnen,,ADJA,,,0.465876,False
716611,besonnene,gm,besonnen,besonnen,,,,,1.790744,False
716606,besonnenem,gm,besonnen,besonnen,,,,,0.786124,False
716607,besonnenen,gm,besonnen,besonnen,,,,,2.242026,False
716608,besonnener,gm,besonnen,besonnen,,,,,1.233051,False
716609,besonnener_1,gm,besonnen,besonnen,,,,,0.068565,False
716616,besonnenere,gm,besonnen,besonnen,,,,,1.790744,False
716612,besonnenerem,gm,besonnen,besonnen,,,,,0.740669,False


In [440]:
# Genügend Datensätze?
print(wiktionary_merkmal.shape[0])
assert wiktionary_merkmal.shape[0] > 4800 * 1000

4838893


In [441]:
# Prüfen: is_unique
assert wiktionary_lemma.lemma_id.is_unique
assert translate_lex.nolemma_id.is_unique

## wiktionary_merkmal
* is_lex

In [442]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [443]:
# korrigieren: Punkt hinten ergänzen
startet_nicht_mit = ( '.', 'Industrie', 'St.')
endet_nicht_mit   = ( '.', 'Tag', 'Ort','Länge','Sieg')
mask1 =  wiktionary_merkmal.data.str.contains('.', regex=False)  
mask2 = ~wiktionary_merkmal.data.str.contains('[0123456789]', regex=True)  
mask3 = ~wiktionary_merkmal.data.str.startswith(startet_nicht_mit)
mask4 = ~wiktionary_merkmal.data.str.endswith(  endet_nicht_mit)
mask5 = (wiktionary_merkmal.data.str.split('.').str[-1].str.len() <= 5) # letztes Wort nicht zu lang
mask6 =  wiktionary_merkmal.data.str.len() > 2
mask7 = ~wiktionary_merkmal.merkmal.isin(['ipa'])
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7
check_mask( wiktionary_merkmal, mask, 233 )

wiktionary_merkmal.loc[mask,'data'] += '.'
pak.sample(wiktionary_merkmal[mask])

check_mask:  241 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
5374,Straßenbahn,abk,Straßenbahn~Str.-B,Str.-B.,,NN,1,,263.622375,True
64801,a._D.,alt_2,a._D.~a. d. D,a. d. D.,,,3,,11.459572,True
78720,p._A.,alt_2,p._A.~p. Adr,p. Adr.,,TODO,1,,6.032382,True
101860,Ingenieur,abk,Ingenieur~Dipl.-Ing,Dipl.-Ing.,,NN,1,,467.962463,True
102378,Geburtstag,abk,Geburtstag~Geb.-T,Geb.-T.,,NN,"1,2",,1581.322998,True
141173,BfA,up,,K. d. ö. R.,,NN,1,,18.144148,False
200438,North_Carolina_1,abk,North_Carolina_1~N.C,N.C.,,NE,1,,1.624077,True
381098,östliche_Länge,abk,östliche_Länge~östl. L,östl. L.,,NN,,,33.685715,True
435149,Reserve,abk,Reserve~d. R,d. R.,,NN,4,,584.237366,True
586750,Mag.,weib,Mag.a,Mag.a.,,TODO,1,,56.269928,True


In [444]:
#wiktionary_merkmal.head(1)

In [445]:
# Merkmale, die eigentlich tags sind
mask = wiktionary_merkmal.merkmal.isin(alle_tags)
check_mask( wiktionary_merkmal, mask, 10 )

wiktionary_merkmal.loc[mask,'lemma_tag'] = ''
wiktionary_merkmal.loc[mask,'data_tag']  = wiktionary_merkmal[mask].merkmal
wiktionary_merkmal.loc[mask,'merkmal']   = 'lex'
wiktionary_merkmal.loc[mask,'is_lex']    = True
grid(wiktionary_merkmal,mask)

check_mask:  10 rows
10 rows out of 4838893


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
1830808,zu_viel,lex,zu_viele,zu viele,ADV,,,manuell,14.845442,True
1830811,vergleichen,lex,vergleiche,vergleiche,VVIMP,,,manuell,96.039993,True
1830812,einreden,lex,einzureden,einzureden,VVIZU,,,manuell,77.859055,True
1830813,loswerden,lex,loszuwerden,loszuwerden,VVIZU,,,manuell,520.682129,True
1830814,herumpimpeln_M,lex,herumzupimpeln,herumzupimpeln,VVIZU,,,manuell,2.239461,True
1830815,hereinlassen_M,lex,hereinlassen_M~hereinzulassen,hereinzulassen,VVIZU,,,manuell,2.239461,True
1830816,herauslassen_M,lex,herauslassen_M~herauszulassen,herauszulassen,VVIZU,,,manuell,2.239461,True
1830817,aufrütteln_M,lex,aufzurütteln,aufzurütteln,VVIZU,,,manuell,2.239461,True
1830818,anstaunen,lex,anzustaunen,anzustaunen,VVIZU,,,manuell,26.530531,True
1830819,angucken_M,lex,anzugucken,anzugucken,VVIZU,,,manuell,2.239461,True


In [446]:
# Mehrfache Spaces >> Einfacher Space (eigentlich hätten die alle schon von Preprocess erwischt werden müssen)
wiktionary_merkmal['data'] = pak.replace_str( wiktionary_merkmal.data, {'  ':' '})

In [447]:
# Mehrfachbindestrich
mask =  wiktionary_merkmal.data == '--'
wiktionary_merkmal.loc[mask,'data'] = '-'

In [448]:
# Hochkommas entfernen
mask1 = wiktionary_merkmal.data.str.count("\'") > 1
mask2 = wiktionary_merkmal.merkmal.isin(['def','abk']) 
mask  =  mask1  &  mask2 
check_mask(wiktionary_merkmal,mask, 0, 20)
wiktionary_merkmal.loc[mask,'data'] = wiktionary_merkmal[mask].data.str.replace("'", '')

check_mask:  7 rows


In [449]:
# Leerstring data Müll löschen
mask = wiktionary_merkmal['data'] == ''
check_mask(wiktionary_merkmal,mask,0,20)

msg = 'Leerstring data Müll'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)
grid(wiktionary_merkmal_trash, msg)

check_mask:  6 rows
Move 6 rows from 4838893
6 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg
2151033,Außerm_Pfarrgarten,Genus,,,,NE,,,26.970110,False,Leerstring data Müll
2177557,Heisei,Genus,,,,NN,,,44.313446,False,Leerstring data Müll
2177564,Reiwa,Genus,,,,NN,,,45.144207,False,Leerstring data Müll
2183522,Covid,Genus,,,,NE,,,44.336594,False,Leerstring data Müll
2201278,Am_Bahnhof,Genus,,,,NE,,,26.080570,False,Leerstring data Müll
2201279,Am_Hauptbahnhof,Genus,,,,NE,,,50.251770,False,Leerstring data Müll


In [450]:
# fillna
wiktionary_merkmal['meta']       = wiktionary_merkmal.meta.fillna('')
wiktionary_merkmal['data_tag']   = wiktionary_merkmal.data_tag.fillna('')
wiktionary_merkmal['lemma_tag']  = wiktionary_merkmal.lemma_tag.fillna('')
wiktionary_merkmal['num']        = wiktionary_merkmal.num.fillna('')

## translate_lex ==========
* hieß bislang wiktionary_nolemma
* Gerundivum als Merkmal speichern
* verwaiste löschen
* Spalten umbenennen
* pflege_translate_tabelle

In [451]:
assert not pak.any_nan(translate_lex)

In [452]:
# lemma_id vorhanden?
mask = translate_lex.lemma_id == ''
check_mask(translate_lex,mask,0)

check_mask:  0 rows


In [453]:
# nolemma_id vorhanden und nicht widersprüchlich?
mask = translate_lex.nolemma_id == ''
check_mask(translate_lex,mask,0)

a = pak.same_but_different(translate_lex,same='nolemma_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


### Gerundivum als Merkmal speichern

In [454]:
# Gerundivum finden
mask1 =  translate_lex.data.str.contains('zu')
mask2 = ~translate_lex.data.str.endswith('zu')
mask3 =  translate_lex.tag.str.contains('LEX_A') 
mask4 =  translate_lex.member.str.contains('Dekliniertes_Gerundivum') 
mask = mask1  & mask2  &  mask3  &  mask4
check_mask(translate_lex,mask,16480)

df = translate_lex[mask].copy()
pak.sample(df)

check_mask:  16480 rows


,nolemma_id,data,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score,lemma_id,lemma,lemma_lower,lemma_tagZZ
359940,aufzuwärmende,aufzuwärmende,LEX_A,LEX_A,Hon0KQW59P,,,Dekliniertes_Gerundivum Adjektiv,1.813820,aufwärmen,aufwärmen,aufwärmen,V
364155,abzustrafendem,abzustrafendem,LEX_A,LEX_A,XsYGpeVN4T,,,Dekliniertes_Gerundivum Adjektiv,0.823314,abstrafen,abstrafen,abstrafen,V
369464,anzudenkendem,anzudenkendem,LEX_A,LEX_A,00ZZsHPXSI,,,Dekliniertes_Gerundivum Adjektiv,0.806785,andenken,andenken,andenken,V
409567,abzuarbeitende,abzuarbeitende,LEX_A,LEX_A,kfFIdZAMi1,,,Dekliniertes_Gerundivum Adjektiv,1.799718,abarbeiten,abarbeiten,abarbeiten,V
409571,abzuarbeitendem,abzuarbeitendem,LEX_A,LEX_A,c5ZWTaXEod,,,Dekliniertes_Gerundivum Adjektiv,0.841909,abarbeiten,abarbeiten,abarbeiten,V
409587,abzubackende,abzubackende,LEX_A,LEX_A,azcHg7Mfys,,,Dekliniertes_Gerundivum Adjektiv,1.842026,abbacken,abbacken,abbacken,V
410758,abzubalgende,abzubalgende,LEX_A,LEX_A,R4gDrEFHPD,,,Dekliniertes_Gerundivum Adjektiv,1.759974,abbalgen,abbalgen,abbalgen,V
411125,abzubüßende,abzubüßende,LEX_A,LEX_A,QCJttyEtUS,,,Dekliniertes_Gerundivum Adjektiv,2.572382,abbüßen,abbüßen,abbüßen,V
429605,aufzubettender,aufzubettender,LEX_A,LEX_A,zzrzQxczjp,,,Dekliniertes_Gerundivum Adjektiv,1.265103,aufbetten,aufbetten,aufbetten,V
486788,einzubezahlenden,einzubezahlenden,LEX_A,LEX_A,001IwUm5tP,,,Dekliniertes_Gerundivum Adjektiv,2.202282,einbezahlen,einbezahlen,einbezahlen,V


In [455]:
# anfügen vorbereiten
df = pak.rename_col( df, 'lex',        'data' )
df = pak.rename_col( df, 'nolemma_id', 'data_id' )
#df = pak.rename_col( df, 'data_score', 'score' )
df['merkmal']  = 'Gerundivum'
df['meta']     = ''
df['num']      = ''
df['is_lex']   = True
df['data_tag'] = 'ADJA'
df = pak.drop_cols( df, set(df.columns) - set(wiktionary_merkmal.columns) )
df = pak.update_col(df, wiktionary_lemma, left_on='lemma_id', right_on='lemma_id', col='tag', col_rename='lemma_tag')

update_col: 16480 cells written into new column


In [456]:
# Prüfen: Alle Gerundiva verweisen auf ein Verb
mask_L = wiktionary_lemma.tagZZ == 'V'
verben = wiktionary_lemma[mask_L].lemma_id

mask = ~df.lemma_id.isin(verben)
check_mask(df,mask,0)

check_mask:  0 rows


In [457]:
# anfügen
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, df)

16480 rows added, now a total of 4855367


In [458]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [459]:
mask = wiktionary_merkmal.merkmal == 'Gerundivum'
pak.sample(wiktionary_merkmal[mask])

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
4838887,aufwärmen,Gerundivum,aufzuwärmende,aufzuwärmende,ADJA,VVFIN,,,1.813820,True
4839021,weiterlesen,Gerundivum,weiterzulesendem,weiterzulesendem,ADJA,VVFIN,,,0.835711,True
4840738,abarbeiten,Gerundivum,abzuarbeitende,abzuarbeitende,ADJA,VVFIN,,,1.799718,True
4840742,abarbeiten,Gerundivum,abzuarbeitendem,abzuarbeitendem,ADJA,VVFIN,,,0.841909,True
4841987,abbannen,Gerundivum,abzubannende,abzubannende,ADJA,VVFIN,,,1.758692,True
4849295,widereinanderstoßen,Gerundivum,widereinanderzustoßenden,widereinanderzustoßenden,ADJA,VVFIN,,,3.497060,True
4850033,hinziehen,Gerundivum,hinzuziehendem_1,hinzuziehendem,ADJA,VVFIN,,,0.102914,True
4853536,übriglassen,Gerundivum,übrigzulassendes,übrigzulassendes,ADJA,VVFIN,,,1.262538,True
4855366,entgegenwerfen,Gerundivum,entgegenzuwerfendes,entgegenzuwerfendes,ADJA,VVFIN,,,1.171513,True


### translate_lex pflegen

In [460]:
#grid(translate_lex,pak.sample)

In [461]:
# Verwaiste löschen
mask = ~translate_lex.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lex,mask,0,30)

msg = 'verwaist_lemma_id'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

check_mask:  9 rows
Move 9 rows from 667548
9 rows


,nolemma_id,data,tag,tag_0,tag_1,tagZ,tagZZ,member,data_score,lemma_id,lemma,lemma_lower,lemma_tagZZ,msg
884,zagen_1,zagen,LEX_A,LEX_A,zAtWHHXdwp,,,Deklinierte_Form,0.328490,zage,zag,zag,A,verwaist_lemma_id
28326,Kremes,Kremes,LEX_A,LEX_A,HcpfytfyAZ,,,Deklinierte_Form,0.753066,Cremes,Kreme,kreme,N,verwaist_lemma_id
177595,Meßgewande,Meßgewande,LEX_A,LEX_A,YB9uak5o7V,,,Deklinierte_Form,1.053846,Messgewande,Meßgewand,meßgewand,N,verwaist_lemma_id
204892,Zierates,Zierates,LEX_A,LEX_A,tnfVv8srVJ,,,Deklinierte_Form,0.673458,Zierats,Zierat,zierat,N,verwaist_lemma_id
204893,Zierats,Zierats,LEX_A,LEX_A,SIjshKLpXi,,,Deklinierte_Form,0.667614,Zierates,Zierat,zierat,N,verwaist_lemma_id
281103,kalvinistischsten,kalvinistischsten,LEX_A,LEX_A,8cB9B12ei0,,,Deklinierte_Form,2.283051,calvinistischsten,kalvinistisch,kalvinistisch,A,verwaist_lemma_id
281104,kalvinistischsten_1,kalvinistischsten,LEX_A,LEX_A,5qUQXSsu5e,,,Superlativ Adjektiv,0.081220,calvinistischsten,kalvinistisch,kalvinistisch,A,verwaist_lemma_id
654680,Fischgratparkettes,Fischgratparkettes,LEX_A,LEX_A,P2et0rDmni,,,Deklinierte_Form,0.842930,Fischgratparketts,Fischgratparkett,fischgratparkett,N,verwaist_lemma_id
654681,Fischgratparketts,Fischgratparketts,LEX_A,LEX_A,inOCtLnD0h,,,Deklinierte_Form,1.526227,Fischgratparkettes,Fischgratparkett,fischgratparkett,N,verwaist_lemma_id


In [462]:
# umformen
translate_lex = pak.rename_col( translate_lex, 'lex',        'data' )
translate_lex = pak.rename_col( translate_lex, 'nolemma_id', 'data_id' )
translate_lex['member'] = ('wiki ' + translate_lex['member'] ).str.strip()
translate_lex = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma)

update_col: 667539 cells written into existing column
update_col: 0 cells written into existing column
update_col: 0 cells written into new column
update_col: 667539 cells written into existing column
update_col: 667539 cells written into new column


In [463]:
# Müll
mask = translate_lex.lemma_tag == 'TODO'
check_mask(translate_lex,mask,0,5)

msg = 'TODO'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )

check_mask:  5 rows
Move 5 rows from 667539


In [464]:
# not any nan
assert not pak.any_nan(translate_lex)

In [465]:
pak.sample(translate_lex)

,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
0,lieben_1,lieben,,0.311824,wiki Deklinierte_Form,lieb,lieb,ADJA,A,82.432930,lieb
1,Polen_1,Polen,,0.113761,wiki Deklinierte_Form,Pole,Pole,NN,N,473.835083,pole
1491,Häuser,Häuser,,0.812983,wiki Deklinierte_Form,Haus,Haus,NN,N,108960.914062,haus
4690,bedeckte,bedeckte,,1.726641,wiki Deklinierte_Form,bedeckt,bedeckt,ADJA,A,222.012573,bedeckt
188180,überführte,überführte,,1.812539,wiki Deklinierte_Form,überführt_4,überführt,ADJA,A,0.032591,überführt
188314,α-Zerfällen,α-Zerfällen,,0.441083,wiki Deklinierte_Form,α-Zerfall,α-Zerfall,NN,N,345.928619,α-zerfall
342561,beizubehalten,beizubehalten,,0.492736,wiki Erweiterter_Infinitiv Verb,beibehalten,beibehalten,VVFIN,V,86.246506,beibehalten
363206,abfliegende,abfliegende,,1.843308,wiki Deklinierte_Form,abfliegend,abfliegend,ADJA,A,1.230273,abfliegend
528508,anstach,anstach,,0.724141,wiki Konjugierte_Form Verb,anstechen,anstechen,VVFIN,V,293.354034,anstechen
574405,-abelste,-abelste,,1.736897,wiki Deklinierte_Form,-abel,-abel,ZFIX,Z,225.476135,-abel


In [466]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

3 rows


,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
13851,Euros,Euros,,0.908025,wiki Deklinierte_Form,Euro,Euro,NN,N,1403.598999,euro
60437,tugendhaftestes,tugendhaftestes,,1.194590,wiki Deklinierte_Form,tugendhaft,tugendhaft,ADJA,A,84.057236,tugendhaft
667520,zu_viele,zu viele,,1.020573,wiki Wortverbindung,zu_viel,zu viel,ADV,A,51.254711,zu viel


In [467]:
# anschauen
#pak.analyse_freqs(translate_lex,['member','lemma_tag','lex'])

## Partizipien ==========
Für fast jedes Partizip gibt es zwei Datensätze: <br> 
1) in wiktionary_merkmal.data: Die Partizip-Flexion des Verbs. merkmal ist 'Ptz1' oder 'Ptz2'.<br> 
2) in wiktionary_lemma.lemma:  Als Adjektiv mit member 'Partizip'. Es gibt ca. 320 Ausnahmen, nicht alle Partizipien sind auch Adjektive.

In [468]:
# Falsche Partizipien löschen
falsche_partizipien = ['sehen','mögen','hören','sollen','können','lassen']
mask1 = wiktionary_merkmal.merkmal.isin(['Ptz1','Ptz2'])
mask2 = wiktionary_merkmal.data.isin(falsche_partizipien)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0,6)
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal,wiktionary_merkmal_trash, mask, 'Falsche_Partizipien')
#wiktionary_merkmal[mask]

check_mask:  6 rows
Move 6 rows from 4855367


In [469]:
# verben_ptz1: Alle Ptz1-Merkmale der Verben
spalten = ['lemma_id','data_id','data','data_score','merkmal']
mask = wiktionary_merkmal.merkmal == 'Ptz1'
verben_ptz1 = wiktionary_merkmal[mask][spalten]
verben_ptz1 = pak.update_col(df_to=verben_ptz1, df_from=wiktionary_lemma, on='lemma_id', col='lemma', verbose=False)
verben_ptz1 = pak.move_cols(       verben_ptz1,'lemma','lemma_id')

# was anzeigen
a = search_str(verben_ptz1, suche_debug)
if len(a) == 0:
    a = verben_ptz1.sample(5)
a

,lemma_id,lemma,data_id,data,data_score,merkmal
1828516,hereinmarschieren,hereinmarschieren,hereinmarschierend,hereinmarschierend,0.585711,Ptz1
1829687,funktionalisieren,funktionalisieren,funktionalisierend,funktionalisierend,0.577446,Ptz1
1818427,verwachsen,verwachsen,verwachsend,verwachsend,0.492736,Ptz1
1816897,walzen,walzen,walzend,walzend,0.455545,Ptz1
1817932,veruntreuen,veruntreuen,veruntreuend,veruntreuend,0.439017,Ptz1


In [470]:
# Kein leeres data_id
mask = verben_ptz1.data_id.isnull()  |  (verben_ptz1.data_id == '')
check_mask(verben_ptz1,mask,0)

check_mask:  0 rows


In [471]:
# alle enden mit 'nd'
mask = ~verben_ptz1.data.str.endswith('nd')
check_mask(verben_ptz1,mask,0)
#verben_ptz[mask]

check_mask:  0 rows


In [472]:
# verben_ptz2: Alle Ptz2-Merkmale der Verben
spalten = ['lemma_id','data_id','data','data_score','merkmal']
mask = wiktionary_merkmal.merkmal == 'Ptz2'
verben_ptz2 = wiktionary_merkmal[mask][spalten]
verben_ptz2 = pak.update_col(df_to=verben_ptz2, df_from=wiktionary_lemma, on='lemma_id', col='lemma', verbose=False)
verben_ptz2 = pak.move_cols(       verben_ptz2,'lemma','lemma_id')

# was anzeigen
a = search_str(verben_ptz2, suche_debug)
if len(a) == 0:
    a = verben_ptz2.sample(5)
a

,lemma_id,lemma,data_id,data,data_score,merkmal
1813373,e-mailen,e-mailen,geemailt_1,geemailt,0.061075,Ptz2
1816511,vorübergehen,vorübergehen,vorübergegangen,vorübergegangen,0.544388,Ptz2
1814904,tomatisieren,tomatisieren,tomatisiert,tomatisiert,0.505132,Ptz2
1806186,beklauen,beklauen,beklaut,beklaut,0.457612,Ptz2
1814274,wurschteln,wurschteln,gewurschtelt,gewurschtelt,0.422488,Ptz2


In [473]:
# Kein leeres data_id
mask = verben_ptz2.data_id.isnull()  |  (verben_ptz2.data_id == '')
check_mask(verben_ptz2,mask,0)

check_mask:  0 rows


In [474]:
# adjektive_ptz1: Adjektive mit Ptz1-Markierung
spalten = ['lemma_id','lemma','lemma_score','tag','member','data_id']
mask1 = wiktionary_lemma.tagZ == 'ADJ'
mask2 = wiktionary_lemma.member.str.contains('Ptz1')
mask = mask1  &  mask2 
adjektive_ptz1 = wiktionary_lemma[mask][spalten]

# was anzeigen
a = search_str(adjektive_ptz1, suche_debug)
if len(a) == 0:
    a = adjektive_ptz1.sample(5)
a

,lemma_id,lemma,lemma_score,tag,member,data_id
113223,absorbierend,absorbierend,1.509198,ADJA,Ptz1,absorbieren
138297,herumkrakeelend,herumkrakeelend,1.403826,ADJA,Ptz1,herumkrakeelen
161915,versaubeutelnd,versaubeutelnd,1.379033,ADJA,Ptz1,versaubeuteln
139687,hocharbeitend,hocharbeitend,1.341843,ADJA,Ptz1,hocharbeiten
132208,gasend,gasend,1.651760,ADJA,Ptz1,gasen


In [475]:
# Kein leeres data_id
# data_id verweist auf das Verb
mask = adjektive_ptz1.data_id.isnull()  |  (adjektive_ptz1.data_id == '')
check_mask(adjektive_ptz1,mask,0)

check_mask:  0 rows


In [476]:
# adjektive_ptz2: Adjektive mit Ptz2-Markierung
spalten = ['lemma_id','lemma','lemma_score','tag','member','data_id']
mask1 = wiktionary_lemma.tagZ == 'ADJ'
mask2 = wiktionary_lemma.member.str.contains('Ptz2')
mask = mask1  &  mask2 
adjektive_ptz2 = wiktionary_lemma[mask][spalten]

# was anzeigen
a = search_str(adjektive_ptz2, suche_debug)
if len(a) == 0:
    a = adjektive_ptz2.sample(5)
a

,lemma_id,lemma,lemma_score,tag,member,data_id
113820,akkommodiert,akkommodiert,0.511331,ADJA,Ptz2,akkommodieren
139181,hingekritzelt,hingekritzelt,0.443149,ADJA,Ptz2,hinkritzeln
161253,verkürzt,verkürzt,0.476207,ADJA,Ptz2,verkürzen
134448,geprangt,geprangt,0.453479,ADJA,Ptz2,prangen
167350,zueinandergefunden,zueinandergefunden,1.521595,ADJA,Ptz2,zueinanderfinden


In [477]:
# Kein leeres data_id
# data_id verweist auf das Verb
# Das sind Kandidaten für die Aufnahme in lexeme_manuell, Reiter Partizipien
nöö = ['befangen','betreten','durchwachsen','erfahren','erlesen','erwachsen','wollen']
mask1 = adjektive_ptz2.data_id.isnull()  |  (adjektive_ptz2.data_id == '')
mask2 = ~adjektive_ptz2.lemma.isin(nöö)
mask = mask1  &  mask2
#grid(adjektive_ptz2,mask)
check_mask(adjektive_ptz2,mask,0)

check_mask:  0 rows


In [478]:
#for p in adjektive_ptz2[mask].lemma:
#    print(p)

In [479]:
# adjektive_noptz: Adjektive ohne Ptz-Markierung
spalten = ['lemma_id','lemma','lemma_score','tag','member','data_id']
mask1 =  wiktionary_lemma.tagZ == 'ADJ'
mask2 = ~wiktionary_lemma.member.str.contains('Ptz1')
mask3 = ~wiktionary_lemma.member.str.contains('Ptz2')
mask = mask1  &  mask2  &  mask3  
adjektive_noptz = wiktionary_lemma[mask][spalten]

# was anzeigen
a = search_str(adjektive_noptz, suche_debug)
if len(a) == 0:
    a = adjektive_noptz.sample(5)
a

,lemma_id,lemma,lemma_score,tag,member,data_id
118698,außenseiterisch,außenseiterisch,43.479572,ADJA,noSteig,
112355,abgelaufen,abgelaufen,169.345657,ADJA,noSteig,
147336,nationalistisch,nationalistisch,113.636696,ADJA,,
152499,san-marinesisch,san-marinesisch,54.822594,ADJA,noSteig,
150459,punktförmig,punktförmig,43.178097,ADJA,noSteig,


In [480]:
# Nur leeres data_id
mask = adjektive_noptz.data_id.isnull()  |  (adjektive_noptz.data_id == '')
#grid(adjektive_noptz,~mask)
check_mask(adjektive_noptz,~mask,0)

check_mask:  0 rows


In [481]:
# Das sind vermutlich eigentlich Partizipien 
# Partizip (geraten)
mask1 = adjektive_noptz.lemma.str.endswith('nd')
mask2 = ~adjektive_noptz.lemma.str.contains(' ')
mask = mask1  &  mask2
kandidaten = adjektive_noptz[mask].copy()

# Kandidaten aufbereiten
nöö = ['dutzend','flektierend','frappierend','spannend']
kandidaten['verb'] = kandidaten.lemma.str[:-1]
mask =  wiktionary_lemma.tagZZ == 'V'
kandidaten = pak.update_col(kandidaten, 
                            wiktionary_lemma[mask],
                            left_on=['verb'],
                            right_on=['lemma'],
                            col='lemma_id',
                            col_rename='data_id',
                            func='min'
                           )

# und filtern
maskA = kandidaten.data_id.isnull()  |  (kandidaten.data_id == '')
maskB = kandidaten.lemma.isin(nöö)
mask = maskA  |  maskB
kandidaten = pak.drop_rows(kandidaten,mask)

spalten = ['lemma','verb']
grid(kandidaten[spalten])

update_col: func min applied, 382 records less!
update_col: 8 cells written into existing column
Delete 297 rows from 301
4 rows


,lemma,verb
121264,bestimmend,bestimmen
129409,erregend,erregen
147265,naheliegend,naheliegen
147271,nahestehend,nahestehen


### Gibt es Verben ohne Partizip-Lexeme?
* ggf. Partizipen manuell nachtragen

In [482]:
# Verben ohne Ptz1
spalten = ['lemma_id','lemma','lemma_score','tag','member']
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 = ~wiktionary_lemma.lemma.isin(verben_ptz1.lemma)
mask = mask1  &  mask2 
check_mask(wiktionary_lemma,mask,26)
verben_ohne_ptz1 = wiktionary_lemma[mask][spalten]
#verben_ohne_ptz1

check_mask:  33 rows


In [483]:
# Kandidaten für die manuelle Aufnahme
# - entweder als Ptz1-Lexem in lexeme_manuell (Reiter Partizipien)
# - oder als lemma in die löschliste

nöö = ['angedeihen','erstinken','gegenanargumentieren','grundsanieren','interdependieren','mittagessen','umfragen','urformen','voranmelden','zusammenreiten']
mask1 = verben_ohne_ptz1.lemma.str.endswith('en')
mask2 = ~verben_ohne_ptz1.lemma.isin(nöö)
mask = mask1  &  mask2
kandidaten = verben_ohne_ptz1[mask][['lemma']]
kandidaten = kandidaten.drop_duplicates().sort_values('lemma')
kandidaten['lex'] = kandidaten.lemma + 'd'
kandidaten = pak.move_cols(kandidaten,'lex')
kandidaten

,lex,lemma
115159,anguckend,angucken
132411,geborend,geboren
2100,handstoppend,handstoppen
145638,milbend,milben
146584,möchtend,möchten
154965,sprützend,sprützen
169161,zustande kommend,zustande kommen


In [484]:
# Verben ohne Ptz2
nöö = ['Schlange stehen', 'gehn', 'stehn', 'umsehn','vergleiche']
spalten = ['lemma_id','lemma','lemma_score','tag','member']
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 = ~wiktionary_lemma.lemma.isin(verben_ptz2.lemma)
mask3 = ~wiktionary_lemma.lemma.isin(nöö)
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_lemma,mask,0,11)
verben_ohne_ptz2 = wiktionary_lemma[mask][spalten]
verben_ohne_ptz2

check_mask:  6 rows


,lemma_id,lemma,lemma_score,tag,member
115159,angucken_M,angucken,6.718384,VVFIN,manuell
117142,aufrütteln_M,aufrütteln,6.718384,VVFIN,manuell
132411,geboren_M,geboren,2.239461,VVFIN,manuell
138322,herumpimpeln_M,herumpimpeln,6.718384,VVFIN,manuell
146584,möchten,möchten,7.470430,VVFIN,
169161,zustande_kommen,zustande kommen,12.605865,VVFIN,Wortverbindung


In [485]:
del verben_ohne_ptz1
del verben_ohne_ptz2

In [486]:
#mask = wiktionary_merkmal.lemma_id == 'betreffen'
#wiktionary_merkmal[mask]

In [487]:
#mask = wiktionary_lemma.lemma == 'betreffend'
#grid(wiktionary_lemma[mask])

### Partizipien und Adjektive vergleichen

In [488]:
# merge für Ptz1
a = pd.merge(verben_ptz1, adjektive_ptz1, left_on='data', right_on='lemma', how='outer',indicator=True, suffixes=('_v', '_a'))

In [489]:
# Übersicht
assert a.shape[0] > 13000
pak.analyse_freqs(a,'_merge')

,_merge,_merge_count,_merge_percent,graph
0,both,14180,100.0,##################################################
1,left_only,0,0.0,
2,right_only,0,0.0,


In [490]:
# Als Partizipialadjektiv gekennzeichnet, hat aber kein passendes Partizip
# >> Kandidaten für die Löschliste

mask = a._merge == 'right_only'
check_mask(a,mask,0,20)
a[mask][['lemma_a']]

check_mask:  0 rows


,lemma_a


In [491]:
# Partizip ohne passendes Partizipialadjektiv
mask = a._merge == 'left_only'
check_mask(a,mask,0,1)
#a[mask] 

check_mask:  0 rows


In [492]:
# merge für Ptz2
a = pd.merge(verben_ptz2, adjektive_ptz2, left_on='data', right_on='lemma', how='outer',indicator=True, suffixes=('_v', '_a'))
a = a.drop_duplicates()

In [493]:
# Übersicht
assert a.shape[0] > 14000
pak.analyse_freqs(a,'_merge')

,_merge,_merge_count,_merge_percent,graph
0,both,14197,98.9,#################################################
1,left_only,165,1.1,
2,right_only,0,0.0,


In [494]:
# Als Partizipialadjektiv gekennzeichnet, hat aber kein passendes Partizip
# >> Kandidaten für die Löschliste

mask = a._merge == 'right_only'
check_mask(a,mask,0,30)
a[mask][['lemma_id_a','lemma_a','tag','member']]

check_mask:  0 rows


,lemma_id_a,lemma_a,tag,member


In [495]:
# entkennzeichnen
entkennzeichnen = a[mask].lemma_id_a
mask_L = wiktionary_lemma.lemma_id.isin(entkennzeichnen)
wiktionary_lemma.loc[mask_L,'member'] = pak.remove_str(wiktionary_lemma[mask_L].member, 'Ptz2')
wiktionary_lemma[mask_L]

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score


In [496]:
# Partizip ohne passendes Partizipialadjektiv
mask = a._merge == 'left_only'
check_mask(a,mask,0,350)
fehlende_adjektive = a[mask]
fehlende_adjektive = pak.update_col(df_to=fehlende_adjektive, df_from=adjektive_noptz, left_on='lemma_v', right_on='lemma',col='lemma_id',col_rename='adj_id')
#fehlende_adjektive

check_mask:  165 rows
update_col: 7 cells written into new column


In [497]:
# Das Partizipialadjektiv fehlt. Das kann zwei Ursachen haben:
# (1) Es gibt ein Adjektiv, das ist aber nicht als Partizipialadjektiv markiert
# (2) Es gibt kein Adjektiv.
# Zuerst kümmern wir uns um Fall (1).
# Die in adj_id genannten Adjektive müssen noch als Partizipialadjektiv markiert werden
mask_a = ~fehlende_adjektive.adj_id.isnull()
check_mask(fehlende_adjektive,mask_a,0,20)
fehlende_adjektive[mask_a]

check_mask:  7 rows


,lemma_id_v,lemma_v,data_id_v,data,data_score,merkmal,lemma_id_a,lemma_a,lemma_score,tag,member,data_id_a,_merge,adj_id
14080,erfahren,erfahren,erfahren_1,erfahren,163.108917,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,erfahren_1
14140,betreten_1,betreten,betreten,betreten,11.072776,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,betreten
14150,beschlagen_1,beschlagen,beschlagen,beschlagen,18.211857,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,beschlagen
14186,befangen_1,befangen,befangen,befangen,8.290561,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,befangen
14188,erwachsen_1,erwachsen,erwachsen,erwachsen,6.950320,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,erwachsen
14299,erlesen_1,erlesen,erlesen,erlesen,4.263538,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,erlesen
14313,durchwachsen_2,durchwachsen,durchwachsen,durchwachsen,1.615978,Ptz2,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only,durchwachsen


In [498]:
# markieren
mask = wiktionary_lemma.lemma_id.isin(fehlende_adjektive[mask_a].adj_id)
check_mask(wiktionary_lemma,mask,0,20)
wiktionary_lemma.loc[mask,'member'] += ' Ptz2'
#wiktionary_lemma[mask]

check_mask:  7 rows


In [499]:
# Fall (2)
# Dies sind Kandidaten dafür, manuell aufgenommen zu werden.
# Und zwar als Partizipien (Ptz2) in lexeme_manuell, dann werden die Adjektive automatisch erstellt und passend verlinkt.
# Das geschieht in d01_tag_und_member.
#

mask_a = fehlende_adjektive.adj_id.isnull()
kandidaten = fehlende_adjektive[mask_a][['data']].drop_duplicates().sort_values('data')

nöö = ['abbegeben','abbehalten','abbekommen','abgetan','anbekommen','angetan','aufbekommen','aufgetan','ausgetan','begeben','behalten','beibehalten','bekommen',
       'benachrangt','bestoßen','betan','betragen','dabehalten','dargetan','dazugetan','dickgetan','dortbehalten',
       'durchbekommen','durchhauen','durchmarschiert','entraten','entschlagen','erfangen','ermessen','erschlafen','erschollen',
       'ersehen','ertragen','fallenlassen','fortwollen','ge- -zt','gegenanargumentiert','geheimgetan',
       'gemecht','genuggetan','geraidet','getan','gleichgetan','großgetan','gutgetan','hartgetan','hervorgetan','hinbekommen','hindurchmarschieren',
       'hineinbegeben','hineingetan','hinterlaufen','hinunterbekommen','hinzugetan','hinüberrannte','hängen lassen','leichtgetan','leidgetan',
       'lieb behalten','missfallen','mitbekommen','mitwollen','nachbekommen','nachgeraten','nachgetan','notgetan','schwergetan','schöngetan','stehenlassen',
       'umeinandergehetzt','umeinandergelegen','umeinandergestanden','umgefragt','umgetan','umtreten','unterbekommen','unterfangen','unterwaschen',
       'verhebt','verladen','verschleißt','versehen','vertan','vertragen',
       'vorgetan','weggetan','wegwollen','weh getan','wehgetan','widerfahren','widerraten','wiederwollen','wohlgetan','zubekommen','zugetan',
       'zugutegetan','zurechtbekommen','zurückbegeben','zurückbekommen','zurückerhalten','zurückgetan','zurückwollen','zusammenbekommen','zusammengetan','zuvorgetan',
       'überlassen','überschlafen','betroffen','geschehen','umschlagen']
print('len nöö =',len(nöö))
mask1 = ~kandidaten.data.str.startswith('-')
mask2 = ~kandidaten.data.isin(nöö)
mask = mask1  &  mask2  
check_mask(kandidaten,mask, 0,20)
grid(kandidaten,mask)

len nöö = 110
check_mask:  7 rows
7 rows out of 118


,data
14187,fortbegeben
14077,geadert
14194,gesprützt
14078,geädert
14302,herumbekommen
14320,photogeshoppt
14131,zusammengeraten


### Aufräumen

In [500]:
del verben_ptz1
del verben_ptz2
del adjektive_ptz1
del adjektive_ptz2
del adjektive_noptz

## wiktionary_lemma ==========

In [501]:
# lemma_id vorhanden und eindeutig?
mask = wiktionary_lemma.lemma_id == ''
check_mask(wiktionary_lemma,mask,0)

assert wiktionary_lemma.lemma_id.is_unique

check_mask:  0 rows


In [502]:
pak.nnan(translate_lex)

Series([], dtype: int64)

In [503]:
# Dups?
mask = wiktionary_lemma.duplicated(subset=['lemma_id'])
check_mask(wiktionary_lemma,mask,0)

check_mask:  0 rows


In [504]:
# nach Priorität sortieren (laut Steuertabellen)
wiktionary_lemma['tag'] = tag_sortieren(wiktionary_lemma.tag)

In [505]:
# Spalten wiederherstellen
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1
wiktionary_lemma = ergänze_felder(wiktionary_lemma)                     # lemma_lower, isupper, lemma_len

# Fehler?
mask = (wiktionary_lemma.tagZZ.str.len() > 1)
check_mask(wiktionary_lemma,mask,0,10) # ein paar können wir tolerieren

check_mask:  303 rows
check_mask:  0 rows


In [506]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
453,Euro,11127,0,Euro,euro,True,1,NN,NN,6Y69DrZ3KB,NOM,N,Einheit manuell,,1403.598999
2607,zu_viel,89575,0,zu viel,zu viel,False,2,ADV,ADV,JGmS0Ezhw2,ADV,A,Wortverbindung manuell,,51.254711
29704,Euro_1,11127,1,Euro,euro,True,1,NN,NN,ZoNMYpSagd,NOM,N,,,36.994484


## Abkürzungen ==========
* abspalten
* belöschen
* korrigieren
* variieren
* pflegen

In [507]:
geschütze_abkürzungen = ['H2O','DEU','deu','RAF','UNO','CDU','SPD','FDP',
                         'SED','UTC','ORF','RKI',
                         'Mo','Di','Mi','Do','Fr','Sa','So',  'MO','DI','MI','DO','FR','SA','SO',  'Mo.','Di.','Mi.','Do.','Fr.','Sa.','So.', 'NS','US','VR','IR','UV','9/11','OP','KI','AI','IT','IS',
                         'Dax','FDJ','MDR','USB','2D','3D','OEM','VW',
                         'u.','ca.','ev.','PKK','USK','dpi','SUV', 'Fa.','od.','u.','CO2','SO2','VR China','Meck-Pomm','Hbf',
]

In [508]:
bsp_abkürzungen_kurz = ['altisländ.', 'EU','Krh', 'Krh.', 'Krhs', 'Krhs.', 'Krkhs', 'Krkhs.']
bsp_abkürzungen_lang = ['altisländisch', 'Europäische Union','Krankenhaus']

In [509]:
# wiktionary_lemma kann sowohl die Langform als auch die Kurzform als eigenständiges lemma enthalten.
# Eine Kurzform ist dabei mit member 'Abkürzung' gekennzeichnet.
# TODO: Beide müssen nicht unbedingt den gleichen tag haben, siehe 'EU' vs. 'Europäische Union': NN / NE
# TODO: Nicht jede Kurzform hat einen solchen Datensatz.

search_str(wiktionary_lemma, bsp_abkürzungen_kurz + bsp_abkürzungen_lang)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
25535,EU,2594,0,EU,eu,True,1,NN,NN,ogiDPNaXBP,NOM,N,Abkürzung noPl,,636.565918
29756,Europäische_Union,12532,0,Europäische Union,europäische union,True,2,NE,NE,uybq13EMS2,NAM,N,adjNN Wortverbindung adjNN,,108.882866
55175,Krankenhaus,732,0,Krankenhaus,krankenhaus,True,1,NN,NN,3vYP0v96YO,NOM,N,,,52026.234375
55532,Krh.,885631,0,Krh.,krh.,True,1,TODO,TODO,MmYrohybl3,X_todo,X,Abkürzung,,5.775660
55533,Krhs.,885632,0,Krhs.,krhs.,True,1,TODO,TODO,mu9IOuR8OR,X_todo,X,Abkürzung,,6.462754
55855,Krkhs.,885633,0,Krkhs.,krkhs.,True,1,TODO,TODO,AvPGfbWu3x,X_todo,X,Abkürzung,,5.818603
114147,altisländ.,21498,0,altisländ.,altisländ.,False,1,TODO,TODO,KhhNGA29QI,X_todo,X,Abkürzung,,34.896648
114148,altisländisch,609806,0,altisländisch,altisländisch,False,1,ADJA,ADJA,oOi64beFse,ADJ,A,noSteig,,104.247162


In [510]:
# wiktionary_merkmal enthält für Kurzformen ein Merkmal 'def', das die Langform nennt.

mask1 = wiktionary_merkmal.lemma_id.isin(bsp_abkürzungen_kurz)
mask2 = wiktionary_merkmal.merkmal == 'def'
mask = mask1  &  mask2
wiktionary_merkmal[mask]

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
53844,EU,def,Europäische_Union,Europäische Union,,NN,1,,212.188629,False
197919,altisländ.,def,altisländisch,altisländisch,,TODO,1,,34.896648,False
1451747,Krh.,def,Krankenhaus,Krankenhaus,,TODO,1,,5.775660,False
1451748,Krhs.,def,Krankenhaus,Krankenhaus,,TODO,1,,6.462754,False
1451749,Krkhs.,def,Krankenhaus,Krankenhaus,,TODO,1,,5.818603,False


In [511]:
# wiktionary_merkmal enthält für Langformen Merkmale 'abk', das eine Kurzform nennt.

mask1 = wiktionary_merkmal.lemma_id.isin(bsp_abkürzungen_lang)
mask2 = wiktionary_merkmal.merkmal == 'abk'
mask = mask1  &  mask2
wiktionary_merkmal[mask]

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
19432,Krankenhaus,abk,Krankenhaus~Krh,Krh,,NN,1,,5780.692871,True
19433,Krankenhaus,abk,Krankenhaus~Krhs,Krhs,,NN,1,,5780.692871,True
19434,Krankenhaus,abk,Krankenhaus~Krkhs,Krkhs,,NN,1,,5780.692871,True
1106271,altisländisch,abk,altisländisch~non,non,,ADJA,1,,34.749054,True
1106272,altisländisch,abk,altnordisch,altnordisch,,ADJA,1,,34.749054,True


### wiktionary_merkmal_abk abspalten

In [512]:
#suche_debug = ['au','ei','hm','qm','ui','mg','km','cm','mm','dm','ft','nm','pt','m²','m³','kg','lb','db']

In [513]:
# abk-Merkmale abspalten
mask = (wiktionary_merkmal.merkmal == 'abk')
wiktionary_merkmal, wiktionary_merkmal_abk = pak.move_rows( wiktionary_merkmal, mask, msg=None )
wiktionary_merkmal_abk = wiktionary_merkmal_abk.sort_values(['data_score','data'], ascending=[False,True])

Move 5107 rows from 4855361


In [514]:
#TODO: Direkt durch die Kurzformen in wiktionary_lemma ergänzen?

In [515]:
# suche_debug wiktionary_merkmal_abk
print(bsp_abkürzungen_lang)
search_str(wiktionary_merkmal_abk, bsp_abkürzungen_lang)

['altisländisch', 'Europäische Union', 'Krankenhaus']


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
19432,Krankenhaus,abk,Krankenhaus~Krh,Krh,,NN,1,,5780.692871,True
19433,Krankenhaus,abk,Krankenhaus~Krhs,Krhs,,NN,1,,5780.692871,True
19434,Krankenhaus,abk,Krankenhaus~Krkhs,Krkhs,,NN,1,,5780.692871,True
1106272,altisländisch,abk,altnordisch,altnordisch,,ADJA,1,,34.749054,True
1106271,altisländisch,abk,altisländisch~non,non,,ADJA,1,,34.749054,True


In [516]:
# suche_debug wiktionary_merkmal_abk
search_str(wiktionary_merkmal_abk, suche_debug)

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
129305,Euro,abk,Euro~EUR,EUR,,NN,1,Währungszeichen,153.715988,True
1830843,Euro,abk,Euro~EUR,EUR,,NN,,manuell,153.715988,True


### wiktionary_merkmal_abk belöschen

In [517]:
# belöschen: Manuelle Löschungen aus data oder lemma_id
löschmich =  ['Objekt','ECO-Codes','Bibo','Kilom','Land','Basche','DV-Vertrag','Igel','Dolle','Ampere','Baum',
              'Rat','Reibe','Heinrich_1','Diastema mediale','Divisionär','Soda-Zitron','Antillen-Gulden',
              "'H'B","'H'GW","'H'H","'H'L","'H'ST","'H'RO","'H'WI",'gel','gel.','Reg.','Imp.','Reg','Imp','AUR',
             ]

mask = wiktionary_merkmal_abk.data.isin(löschmich)
check_mask(wiktionary_merkmal_abk,mask, 48)

msg1 = 'löschmich_data'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg1)

mask = wiktionary_merkmal_abk.lemma_id.isin(löschmich)
check_mask(wiktionary_merkmal_abk,mask, 65)

msg2 = 'löschmich_id'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg2)

# anschauen
spalten = ['data', 'meta', 'lemma_id',  'lemma_tag',   'merkmal', 'msg']
mask = wiktionary_merkmal_trash.msg.isin([msg1, msg2])
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  48 rows
Move 48 rows from 5107
check_mask:  65 rows
Move 65 rows from 5059
113 rows out of 125


,data,meta,lemma_id,lemma_tag,merkmal,msg
95044,Bibo,umgangssprachliche,Bibliothek,NN,abk,löschmich_data
116869,Reg,,Regierung,NN,abk,löschmich_data
434639,Reg,,Register,NN,abk,löschmich_data
32407,Kilom,,Kilometer,NN,abk,löschmich_data
45837,Land,,Landung,NN,abk,löschmich_data
...,...,...,...,...,...,...
206228,Heyns,,Heinrich_1,NE,abk,löschmich_id
206229,Heynsen,,Heinrich_1,NE,abk,löschmich_id
206230,Hinner,,Heinrich_1,NE,abk,löschmich_id
350072,ANG,,Antillen-Gulden,NN,abk,löschmich_id


In [518]:
# belöschen: zu lang
mask1 =  wiktionary_merkmal_abk.data.str.len() > 6                    # zu lang
mask2 = ~wiktionary_merkmal_abk.data.str.contains('-')
mask3 = ~wiktionary_merkmal_abk.data.str.contains(' ')
mask4 = ~wiktionary_merkmal_abk.data.str.contains('.', regex=False)
mask5 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask6 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask7 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7
check_mask(wiktionary_merkmal_abk,mask, 338)

msg = 'zulang_single'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  338 rows
Move 338 rows from 4994
338 rows out of 463


,data,meta,lemma_id,lemma_tag,merkmal,msg
46514,Energie,,Arbeit,NN,abk,zulang_single
46513,englisch,,Arbeit,NN,abk,zulang_single
76349,Flächeneinheit,,Morgen,NN,abk,zulang_single
76351,Großbritannien,,Morgen,NN,abk,zulang_single
87656,Landsch,,Landschaft,NN,abk,zulang_single
...,...,...,...,...,...,...
272235,Landkreis,,Hann._Münden,NE,abk,zulang_single
587177,unverbr,,unverbrüchlich,ADJA,abk,zulang_single
295492,Aquilin,,Aquilinus,NE,abk,zulang_single
162351,Kraftfahrzeugkennzeichen,,Halle_1,NE,abk,zulang_single


In [519]:
# belöschen: zu lang
mask1 =  wiktionary_merkmal_abk.data.str.len() > 15                   # zu lang
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask4 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal_abk,mask, 10)

msg = 'zulang'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  10 rows
Move 10 rows from 4656
10 rows out of 473


,data,meta,lemma_id,lemma_tag,merkmal,msg
97032,Ohmscher Widerstand,Größenzeichen,Widerstand,NN,abk,zulang
67844,Doctor theologiae,,Doktor,NN,abk,zulang
1100563,Muster-ADV-Vertrag,,Muster-Auftragsdatenverarbeitungsvertrag,NN,abk,zulang
201556,Sacer Ordo Cisterciensis,,Zisterzienser,NN,abk,zulang
161836,Residual Current Device,englisch,Fehlerstromschutzschalter,NN,abk,zulang
199212,Ordo Cartusiensis,,Kartäuser,NN,abk,zulang
1152351,Kfz-Haftpflichtversicherung,,Kraftfahrzeug-Haftpflichtversicherung,NN,abk,zulang
331594,chemische Formel,,Natriumsulfat,NN,abk,zulang
245749,Niederländischen Antillen,Internet,Aruba,NE,abk,zulang
903360,chemische Formel,,Natriumnitrit,NN,abk,zulang


In [520]:
# belöschen: Namen
mask1 =  wiktionary_merkmal_abk.lemma_tag == 'NE'                  # ist Name
mask2 = ~wiktionary_merkmal_abk.data.str.isupper()                 # nicht nur groß
mask3 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask4 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask5 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(wiktionary_merkmal_abk,mask, 330)

msg = 'Namen'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  329 rows
Move 329 rows from 4646
329 rows out of 802


,data,meta,lemma_id,lemma_tag,merkmal,msg
4903,Dtl,,Deutschland,NE,abk,Namen
4904,Dtld,,Deutschland,NE,abk,Namen
73358,Bln,,Berlin,NE,abk,Namen
73357,Kfz-Kennzeichen,,Berlin,NE,abk,Namen
151798,Johs,,Johannes,NE,abk,Namen
...,...,...,...,...,...,...
213586,hau,ISO,Hausa_2,NE,abk,Namen
248970,bhw,ISO,Biak_1,NE,abk,Namen
250413,Dore,,Doris_1,NE,abk,Namen
230543,bgk,ISO,Bit_1,NE,abk,Namen


In [521]:
# belöschen: Nur Ziffern
mask1 = wiktionary_merkmal_abk.data.str.isdigit() 
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask4 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal_abk,mask, 20)

msg = 'nur_Ziffern'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  20 rows
Move 20 rows from 4317
20 rows out of 822


,data,meta,lemma_id,lemma_tag,merkmal,msg
525716,200,arabische,zweihundert,CARD,abk,nur_Ziffern
661540,400,Arabische,vierhundert,CARD,abk,nur_Ziffern
544347,102,,hundertundzweite,ADJA CARD,abk,nur_Ziffern
1701770,700,arabische,siebenhundert,CARD,abk,nur_Ziffern
1701511,900,arabische,neunhundert,CARD,abk,nur_Ziffern
635132,102,,hundertzweite,ADJA CARD,abk,nur_Ziffern
542226,101,,einhunderterste,ADJA CARD,abk,nur_Ziffern
525682,300,arabische,dreihundert,CARD,abk,nur_Ziffern
544350,200,,zweihundertste,ADJA CARD,abk,nur_Ziffern
542239,101,,hunderterste,ADJA CARD,abk,nur_Ziffern


In [522]:
# belöschen: Endet mit Ziffer
mask1 =  wiktionary_merkmal_abk.data.str[-1].str.isdigit() 
mask2 = ~wiktionary_merkmal_abk.data.str.endswith(('²','³','₂')) 
mask3 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask4 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask5 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5 
check_mask(wiktionary_merkmal_abk,mask, 162)

msg = 'Endet_mit_Ziffer'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  162 rows
Move 162 rows from 4297
162 rows out of 984


,data,meta,lemma_id,lemma_tag,merkmal,msg
20801,E 175,,Gold,NN,abk,Endet_mit_Ziffer
16159,E 949,E-Nummer,Wasserstoff,NN,abk,Endet_mit_Ziffer
126543,E 174,,Silber,NN,abk,Endet_mit_Ziffer
125462,E 948,,Sauerstoff,NN,abk,Endet_mit_Ziffer
126089,E 173,,Aluminium,NN,abk,Endet_mit_Ziffer
...,...,...,...,...,...,...
181742,E 953,,Isomalt,NN,abk,Endet_mit_Ziffer
305638,0-0,,kurze_Rochade,TODO,abk,Endet_mit_Ziffer
305798,0-0-0,,lange_Rochade,TODO,abk,Endet_mit_Ziffer
358976,0-0,,klein_rochieren,TODO,abk,Endet_mit_Ziffer


In [523]:
# belöschen: Bibel
mask1 =  wiktionary_merkmal_abk.lemma_id.str[0].str.isdigit() 
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask4 =  wiktionary_merkmal_abk.meta != 'manuell'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal_abk,mask, 12)

msg = 'Bibel'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash[spalten], mask )

check_mask:  12 rows
Move 12 rows from 4135
12 rows out of 996


,data,meta,lemma_id,lemma_tag,merkmal,msg
1773844,"2,5G",,"2,5G-Regel",NN,abk,Bibel
663739,2 Kor,,2._Korintherbrief,NN,abk,Bibel
663720,1 Kor,,1._Korintherbrief,NN,abk,Bibel
1217629,1 Thess,,1._Thessalonicherbrief,NN,abk,Bibel
1217648,1 Tim,,1._Timotheusbrief,NN,abk,Bibel
1217654,2 Tim,,2._Timotheusbrief,NN,abk,Bibel
1217633,2 Thess,,2._Thessalonicherbrief,NN,abk,Bibel
1219019,2 Petr,,2._Petrusbrief,NN,abk,Bibel
1219017,1 Petr,,1._Petrusbrief,NN,abk,Bibel
1219084,1 Joh,,1._Johannesbrief,NN,abk,Bibel


In [524]:
# belöschen: meta
löschmich_meta = ['Postleitzahl','Elementsymbol','seltener','selten','Ordenskürzel','Österreich','Makrosprache',
                  'Kirmanjki','Schweiz','ISO','Kfz','Internet','ISO-3166','Kfz-Kennzeichen','fachsprachlich','Dreibuchstabencode',
                  'international','Deutschland','ISO-4217-Code','ISO-639-3','chemisches','Zentral-Quiche','Komi',
                  'Kraftfahrzeugkennzeichen','englisch','veraltet','Dimli','Shimaore','bisweilen',
                  'Katalogbezeichnungen','ISO-639-1','inoffiziell','olympisch','chemische','Zusammensetzungen',
                  'Netzjargon','ISO-4217','bundesdeutsch','Währungscodes','UN LOCODE','römische','schweizerisch',
                  'Währungscode','Römische','bedeutungsgleiche','Summenformel',
                  'Rumänien','Lüftungstechnik','Lebensmittelfarbstoff','internationaler','volkstümlich',
                  'Maya-Sprachen','ISO-3166-1','ISO-3166-Code','üblich','wissenschaftlich','ehem',
                  'Polizeijargon','IOC-Ländercode','Moldawien','taktisches','postalisch','Abkürzung','Römisches',
                  'fachsprachlich schriftsprachlich','Schifffahrt','Codierung','SIL-Code']
mask1 =  wiktionary_merkmal_abk.meta.isin(löschmich_meta)
mask2 = ~wiktionary_merkmal_abk.lemma_id.str.endswith('_M')        # nicht aus lexeme_manuell
mask3 = ~wiktionary_merkmal_abk.data.isin(geschütze_abkürzungen)
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal_abk,mask, 1035)

msg = 'meta'
wiktionary_merkmal_abk, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal_abk, wiktionary_merkmal_trash, mask, msg)

# anschauen
mask = wiktionary_merkmal_trash.msg == msg
a = wiktionary_merkmal_trash[mask].sort_values(['meta','data_score'],ascending=[True,False])
grid(a)

check_mask:  1035 rows
Move 1035 rows from 4123
1035 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg
749186,Sperrzone,abk,Sperrzone~SpZo,SpZo,,NN,1,Abkürzung,54.319572,True,meta
628703,Asthenie,abk,ICD,ICD,,NN,1,Codierung,83.081894,True,meta
103075,Rangierbahnhof,abk,Rbf,Rbf,,NN,,Deutschland,124.814873,True,meta
1520458,Trinkwasserverordnung,abk,Trinkwasserverordnung~TWV,TWV,,NN,1,Deutschland,97.237305,True,meta
1133574,Gesundheitsministerium,abk,Gesundheitsministerium~BMG,BMG,,NN,1,Deutschland,59.766357,True,meta
...,...,...,...,...,...,...,...,...,...,...,...
655098,Kubikzentimeter,abk,cbcm,cbcm,,NN,1,veraltet,36.631935,True,meta
655097,Kubikzentimeter,abk,ccm,ccm,,NN,1,veraltet,36.631935,True,meta
175355,Bauernhof,abk,Hof,Hof,,NN,1,volkstümlich,198.280060,True,meta
335119,Wisent,abk,Wisent~B. bonasus,B. bonasus,,NN,1,wissenschaftlich,110.699867,True,meta


In [525]:
# Übriggebliebene nach meta
a = pak.analyse_freqs( wiktionary_merkmal_abk, ['meta','lemma_tag','data'] )
grid(a)

16 rows


,meta,meta_count,meta_percent,lemma_tag,lemma_tag_count,data,data_count
0,,2953,95.6,"[NN, ADJA, NE, ADV, TODO, VVFIN, APPR, , KOMBI, CARD, KON, NN NE, PDAT, ZFIX, PTKANT, ADV APPR, PIAT, KOUS, APPO APPR, ITJ]","[1997, 609, 123, 98, 36, 23, 18, 10, 8, 5, 4, 3, 3, 3, 3, 2, 2, 2, 2, 1]","[Str, inf, Pfl, Konv, ind, Ins, Gen, imp, Chemie, erk, Ztr, Adj, Chr, Konj, ged, get, Pat, NaCl, Ver, Erl]","[16, 9, 7, 7, 7, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4]"
1,manuell,112,3.6,"[NN, KOMBI, ADJA, ADV, KON, $(, NN NE, VVFIN, KOUS, ADV ADJD, APPR, CARD, NE]","[68, 17, 7, 4, 3, 3, 2, 2, 2, 1, 1, 1, 1]","[Bd., Feb., Jan., Str., FR, Nr., std., SO, OP, geschl., Abb., Hg., Jun., km, Abschn., m, DI, SA, MI, MO]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,umgangssprachlich,4,0.1,"[NN, PIAT]","[3, 1]","[was, Perso, Mini, Kelle]","[1, 1, 1, 1]"
3,kurz,3,0.1,NN,3,"[Mayo, Emu, Snooker-WM]","[1, 1, 1]"
4,international,2,0.1,NE,2,"[DEU, UNO]","[1, 1]"
5,physikalisch,2,0.1,NN,2,"[atm, atü]","[1, 1]"
6,chemische,2,0.1,NN,2,"[CO2, SO2]","[1, 1]"
7,Einheitenzeichen,2,0.1,NN,2,"[min, var]","[1, 1]"
8,Summenformel,1,0.0,NN,1,H2O,1
9,ISO,1,0.0,NN,1,deu,1


In [526]:
# meta ist jetzt sinnlos, bis auf 'manuell'
mask =  wiktionary_merkmal_abk.meta != 'manuell'
wiktionary_merkmal_abk.loc[mask,'meta'] = ''

In [527]:
# suche_debug wiktionary_merkmal_abk
search_str(wiktionary_merkmal_abk, suche_debug)

,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
129305,Euro,abk,Euro~EUR,EUR,,NN,1,,153.715988,True
1830843,Euro,abk,Euro~EUR,EUR,,NN,,manuell,153.715988,True


### Analysieren

In [528]:
# Gelöschte überprüfen, aus suche_debug?
print(suche_debug)
search_str(wiktionary_merkmal_trash, suche_debug)

['Euro', '€', 'tugendhaftestes', 'zu viele', 'zuviele', 'zu viel']


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg


In [529]:
# Gelöschte überprüfen, aus lexeme_manuell?
mask1 = wiktionary_merkmal_trash.lemma_id.str.endswith('_M') 
mask2 = ~wiktionary_merkmal_trash.msg.isin(['dup'])
mask = mask1  &  mask2
check_mask(wiktionary_merkmal_trash, mask, 0)
#wiktionary_merkmal_trash[mask]

check_mask:  0 rows


In [530]:
# Gelöschte überprüfen, aus geschütze_abkürzungen?
mask = wiktionary_merkmal_trash.data.isin(geschütze_abkürzungen)
check_mask(wiktionary_merkmal_trash, mask, 0)

check_mask:  0 rows


In [531]:
# Gelöschte anschauen
a = pak.analyse_freqs( wiktionary_merkmal_trash, ['msg','lemma_tag','data'] )
grid(a)

11 rows


,msg,msg_count,msg_percent,lemma_tag,lemma_tag_count,data,data_count
0,meta,1035,51.0,"[NN, NE, ADJA, CARD, ADV, VVFIN, ITJ]","[769, 247, 8, 5, 4, 1, 1]","[chi, zho, kom, nya, ber, ful, nbl, nde, diq, bel, smi, NGC, kpv, srn, roa, fur, ndo, tut, grc, spa]","[11, 11, 5, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2]"
1,zulang_single,338,16.6,"[NN, ADJA, NE, VVFIN, TODO, PIAT, CARD, APPR, ADV]","[192, 105, 30, 4, 3, 1, 1, 1, 1]","[Summenformel, Kraftfahrzeugkennzeichen, Linguistik, Hirnnerv, Wasserl, Ostubaf, Schussw, alphanum, Halbstrukturformel, Gruppenformel, Fernschr, Morphonem, unverbr, Energie, englisch, Flächeneinheit, Großbritannien, Landsch, Rechenzeichen, Vorzeichen]","[14, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1]"
2,Namen,329,16.2,NE,329,"[Kfz-Kennzeichen, Balder, UMa, Benz, UMi, Ofr, Chris, Bert, Baldo, Dtl, Dtld, Bln, Johs, Lzg, Bbg, Sachs, Hmb, Afr, Erzg, Erzgeb]","[20, 6, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,Endet_mit_Ziffer,162,8.0,"[NN, NE, TODO, VVFIN]","[103, 53, 4, 2]","[ICD-10, 0-0, 0-0-0, ISO 3166-2, OR-1, A-2384, A-2325, A-2351, E 507, ISO 639-3, E 175, E 949, E 174, E 948, E 173, E 939, E 501, A-2340, A-3390, R53]","[19, 6, 4, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,löschmich_id,65,3.2,"[NE, NN]","[61, 4]","[Sozi, Div, Hein, Heincke, Heindel, Heindl, Heine, Heinecke, Heineke, Heinel, Heinelt, Heinemann, Heinen, Heiner, Heinermann, Heinert, Heines, Heinicke, Heinig, Heink]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,löschmich_data,48,2.4,"[NN, ADJA, NE, VVFIN]","[34, 9, 3, 2]","[gel, Reg, Imp, Rat, Reibe, AUR, Bibo, Kilom, Land, Basche, Objekt, ECO-Codes, DV-Vertrag, Igel, Baum, Dolle, Ampere]","[11, 9, 9, 4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
6,nur_Ziffern,20,1.0,"[ADJA CARD, CARD, ADV]","[12, 6, 2]","[100, 200, 102, 101, 300, 1000, 400, 700, 900, 500, 9000, 10000, 100000]","[3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1]"
7,Bibel,12,0.6,NN,12,"[2,5G, 2 Kor, 1 Kor, 1 Thess, 1 Tim, 2 Tim, 2 Thess, 2 Petr, 1 Petr, 1 Joh, 2 Joh, 3 Joh]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
8,zulang,10,0.5,"[NN, NE]","[9, 1]","[chemische Formel, Ohmscher Widerstand, Doctor theologiae, Muster-ADV-Vertrag, Sacer Ordo Cisterciensis, Residual Current Device, Ordo Cartusiensis, Kfz-Haftpflichtversicherung, Niederländischen Antillen]","[2, 1, 1, 1, 1, 1, 1, 1, 1]"
9,Leerstring data Müll,6,0.3,"[NE, NN]","[4, 2]",,6


In [532]:
# Gelöschte mit dem höchsten score
mask = ~wiktionary_merkmal_trash.msg.isin(['dup'])
grid(wiktionary_merkmal_trash[mask].sort_values('data_score',ascending=False))

2031 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg
46513,Arbeit,abk,englisch,englisch,,NN,7,,3426.708008,True,zulang_single
46514,Arbeit,abk,Energie,Energie,,NN,7,,3426.708008,True,zulang_single
592835,Blindgänger,abk,Blindgänger~UXB,UXB,,NN,1,international,3357.089111,True,meta
592834,Blindgänger,abk,Blindgänger~MEC,MEC,,NN,1,international,3357.089111,True,meta
592836,Blindgänger,abk,Blindgänger~UXO,UXO,,NN,1,international,3357.089111,True,meta
...,...,...,...,...,...,...,...,...,...,...,...
186226,Heide_3,abk,Heide_3~HEI,HEI,,NE,1,Kfz-Kennzeichen,0.967372,True,meta
460570,geliefert,abk,gel,gel,,ADJA,,,0.901368,True,löschmich_data
249498,Iban_2,abk,Iban_2~iba,iba,,NE,1,ISO,0.889556,True,Namen
430967,gelandet,abk,gel,gel,,ADJA,,,0.854617,True,löschmich_data


In [533]:
# Verbliebene nach tag
pak.analyse_freqs(wiktionary_merkmal_abk,['lemma_tag','meta','data'])

,lemma_tag,lemma_tag_count,lemma_tag_percent,meta,meta_count,data,data_count
0,NN,2085,67.5,"[, manuell]","[2017, 68]","[Str, Pfl, Konv, Ins, Gen, Ztr, Adj, Konj, Pat, NaCl, Ver, Chr, Erl, Fut, StB, Demogr, Geb, Hdl, Anl, Verf]","[16, 7, 7, 6, 6, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3]"
1,ADJA,616,19.9,"[, manuell]","[609, 7]","[ind, ged, get, unv, imp, afr, inf, geh, med, vsl, gebr, ungen, soz, frdl, akt, isl, int, reg, pers, lat]","[7, 5, 5, 4, 4, 4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2]"
2,NE,126,4.1,"[, manuell]","[125, 1]","[LUX, RUS, BAD, ABG, MKD, HAL, GKW, DEU, L.E., BRB, ISO, CHN, AUS, MUC, CHE, AUT, AFG, FRA, GBR, WAR]","[2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,ADV,102,3.3,"[, manuell]","[98, 4]","[bes, pass, insb, insbes, ebd, östl, sod, dto, bisl, sof, eig, eigtl, ntto, zur, teilw, desgl, btto, vll, vllt, vlt]","[3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,TODO,36,1.2,,36,"[Chemie, ad lib, s. A., s.A., sel. A., sel.A., m. E., i. E., S. M., gültig, C.c., S. H., m. m., S. Em., HDTV, A. D., z. H., z. Hd., a. l., ad l]","[4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,VVFIN,25,0.8,"[, manuell]","[23, 2]","[erk, inf, imp, gez, abgez, rad, abn, zit, entf, herst, anschl, betr, vgl, vgl., beh, dest, vergl, abr, rat, geb.]","[4, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
6,KOMBI,25,0.8,"[manuell, ]","[17, 8]","[i. Allg., i. A., u. a., u.a., z. Z., z. Zt., z.Zt., s. o., s.o., s. u., s.u., s. a., s.a., Anm. d. Red., i.A., k.A., o. B., o. dgl., o.ä., u. dgl.]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
7,APPR,19,0.6,"[, manuell]","[18, 1]","[entg, innerh, bez, bezgl, bzgl, inf, mod, betr, hins, vs., anl, abzgl, zzgl, gem, inkl, exkl, südl, westl, östl]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
8,,10,0.3,,10,"[dies, jdm, jmdm, jdn, jmdn, jds, jmds, nen, inf]","[2, 1, 1, 1, 1, 1, 1, 1, 1]"
9,KON,7,0.2,"[, manuell]","[4, 3]","[&, bezw, bzw, bzw., resp, rsp, d.h.]","[1, 1, 1, 1, 1, 1, 1]"


In [534]:
# suche_debug wiktionary_merkmal_abk
print(suche_debug)
search_str(wiktionary_merkmal_abk, suche_debug)

['Euro', '€', 'tugendhaftestes', 'zu viele', 'zuviele', 'zu viel']


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
129305,Euro,abk,Euro~EUR,EUR,,NN,1,,153.715988,True
1830843,Euro,abk,Euro~EUR,EUR,,NN,,manuell,153.715988,True


In [535]:
# suche_debug wiktionary_merkmal
#mask = wiktionary_merkmal.merkmal.isin(['gm'])
#a = search_str(wiktionary_merkmal[mask], suche_debug)
#grid(a.sort_values('lemma_id'))

### wiktionary_merkmal_abk korrigieren

In [536]:
# Einzelne Worte löschen
r = ['(grad)',]
wiktionary_merkmal_abk['data'] = pak.remove_str(wiktionary_merkmal_abk.data, r)

In [537]:
# korrigieren: Punkt innen ohne Punkt hinten
# Das ist eigentlich schon weiter oben passiert
endet_nicht_mit = ( '.', 'Tag', 'Ort',)
mask1 =  wiktionary_merkmal_abk.data.str.contains('.', regex=False)  
mask2 = ~wiktionary_merkmal_abk.data.str.endswith(endet_nicht_mit)
mask3 = (wiktionary_merkmal_abk.data.str.split('.').str[-1].str.len() <= 5) # letztes Wort nicht zu lang
mask4 =  wiktionary_merkmal_abk.data.str.len() > 2
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask( wiktionary_merkmal_abk, mask, 0,136 )
wiktionary_merkmal_abk.loc[mask,'data'] += '.'
grid(wiktionary_merkmal_abk,mask)

check_mask:  2 rows
2 rows out of 3088


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
186271,Steuerberater,abk,Steuerberater~St.-Ber,St.-Ber.,,NN,1,,80.872543,True
313841,Steuernummer,abk,Steuernummer~St.-Nr,St.-Nr.,,NN,1,,24.318172,True


In [538]:
# korrigieren: Punkt hinten ergänzen
ausnahmen =  [ 'Admin','Schirm','Kappe','Symbol','Potenz','Schiri','Info','Bus','Kiga','Hof','Teen','Teenie','Teeny','Nitro','Steno','Perso','Navi',]
ausnahmen += [ 'Mayo','Hasch','Britta','Tusse','Tussi','Jäger','Reha','Latte','Pkw','Escape','Spasti','Proll','Prolo','Traufe','Qbit','Qubit',]
ausnahmen += [ 'Rose','Kino','Alster','Hafen','Psycho','Dame','Kelle','Kopie','FeWo','Chemie','Knobi','Makro','Majo','Tief']
mask1 = ~wiktionary_merkmal_abk.data.isin(ausnahmen)
mask2 = ~wiktionary_merkmal_abk.data.str.contains('-')                                                       # Kein Bindestrich
mask3 = ~wiktionary_merkmal_abk.data.str.contains(' ')                                                       # kein Space
mask4 = ~wiktionary_merkmal_abk.data.str.contains('.', regex=False)                                          # kein Punkt
mask5 = ~wiktionary_merkmal_abk.meta.isin(['ISO','kurz','umgangssprachlich','Netzjargon'])
mask6 = (wiktionary_merkmal_abk.data.str.len() > 2)   |   wiktionary_merkmal_abk.data.isin(['Dr','No'])
mask7 =  wiktionary_merkmal_abk.data.str.contains('^[A-Za-zöüäÖÄÜ]')                                         # startswith Buchstabe
mask8 =  wiktionary_merkmal_abk.data.str.count('[A-ZÜÖÄ]')  <=    1                                          # Nicht mehr als 1 Großbuchstabe
mask9 = ~wiktionary_merkmal_abk.data.str.contains('[^A-Za-zöüäÖÄÜß]')                                        # irgendwelche Sonderzeichen

mask  = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7  &  mask8  &  mask9
check_mask( wiktionary_merkmal_abk, mask, 1815 )

wiktionary_merkmal_abk.loc[mask,'data'] += '.'  
grid(wiktionary_merkmal_abk,mask)

check_mask:  1815 rows
1815 rows out of 3088


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
19432,Krankenhaus,abk,Krankenhaus~Krh,Krh.,,NN,1,,5780.692871,True
19433,Krankenhaus,abk,Krankenhaus~Krhs,Krhs.,,NN,1,,5780.692871,True
19434,Krankenhaus,abk,Krankenhaus~Krkhs,Krkhs.,,NN,1,,5780.692871,True
138965,Garten,abk,Garten~Gart,Gart.,,NN,1,,4551.003906,True
21871,Sprache,abk,Sprache~Spr,Spr.,,NN,2,,3765.383057,True
...,...,...,...,...,...,...,...,...,...,...
145125,östlich_1,abk,östlich_1~östl,östl.,,ADV,1,,0.779431,True
457318,infiziert,abk,infiziert~inf,inf.,,ADJA,,,0.769881,True
479907,getilgt,abk,get_M,get.,,ADJA,,,0.711443,True
457320,infiziert_1,abk,infiziert_1~inf,inf.,,,,,0.174397,True


In [539]:
# variieren: Punkt mit Space
#mask = wiktionary_merkmal_abk.data.str.contains('.', regex=False)
mask = wiktionary_merkmal_abk.data.str.count('\.') > 1
df = wiktionary_merkmal_abk[mask].copy()
r = {'.':'. ',   '-':''}
df['data']    = pak.replace_str(df.data, r).str.strip()
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

162 rows added, now a total of 3250


In [540]:
# variieren: Bindestriche weglassen
mask1 =  wiktionary_merkmal_abk.data.str.contains('-')
mask2 = ~wiktionary_merkmal_abk.data.str.startswith('-')
mask3 = ~wiktionary_merkmal_abk.data.str.endswith('-')
mask = mask1  &  mask2  &  mask3
df = wiktionary_merkmal_abk[mask].copy()
r = {'-':' ', '  ':' '}
df['data']    = pak.replace_str(df.data, r)
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

123 rows added, now a total of 3373


In [541]:
# variieren: Punkte weglassen
mask = wiktionary_merkmal_abk.data.str.contains('.', regex=False)
df = wiktionary_merkmal_abk[mask].copy()
r = {'.':'',}
df['data']    = pak.replace_str(df.data, r)
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

2237 rows added, now a total of 5610


In [542]:
# variieren: Spaces weglassen
mask = wiktionary_merkmal_abk.data.str.contains(' ')
df = wiktionary_merkmal_abk[mask].copy()
r = {' ':'',}
df['data']    = pak.replace_str(df.data, r)
wiktionary_merkmal_abk = pak.add_rows(wiktionary_merkmal_abk, df)
#grid(df)

697 rows added, now a total of 6307


In [543]:
# pflegen: Überfüssige Spaces streichen
r = {'  ':' ', '  ':' ',}
wiktionary_merkmal_abk['data'] = pak.replace_str(wiktionary_merkmal_abk.data, r)
wiktionary_merkmal_abk['data'] = wiktionary_merkmal_abk.data.str.strip()

In [544]:
# data_id neu vergeben
wiktionary_merkmal_abk['data_id'] = (wiktionary_merkmal_abk.lemma_id + '~' + wiktionary_merkmal_abk.data).str.strip()

# data_id vorhanden und nicht widerspüchlich?
mask = wiktionary_merkmal_abk.data_id == ''
check_mask(wiktionary_merkmal_abk,mask,0)

a = pak.same_but_different(wiktionary_merkmal_abk,same='data_id',different='data')
assert a.shape[0] == 0

check_mask:  0 rows


In [545]:
grid(wiktionary_merkmal_abk.sort_values('data'))

6307 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
711,Prozent,abk,Prozent~%,%,,NN,,manuell,143.270859,True
374,und,abk,und~&,&,,KON,,manuell,245.027344,True
2901,-prozentig,abk,-prozentig~-%ig,-%ig,,ZFIX,1,,14.858729,True
1250,-schaft,abk,-schaft~-sch,-sch,,ZFIX,1,,78.539078,True
1251,-schaft,abk,-schaft~-schft,-schft,,ZFIX,1,,78.539078,True
...,...,...,...,...,...,...,...,...,...,...
2771,überschüssig,abk,überschüssig~übsch.,übsch.,,ADJA,1,,23.642769,True
3056,Microcontroller_M,abk,Microcontroller_M~μC,μC,,NN,,manuell,2.239461,True
2701,Mikrosievert,abk,Mikrosievert~μSv,μSv,,NN,1,,26.903440,True
1377,Mikrometer,abk,Mikrometer~μm,μm,,NN,,manuell,71.704559,True


In [546]:
# pflegen: Dups entfernen

wiktionary_merkmal_abk = pak.group_and_agg( wiktionary_merkmal_abk, 
              ['lemma_id',  'lemma_tag',  'num',  'merkmal',   'meta',  'data',   'data_id',  'data_score',   ],
              [ 'group',    'group',      'min',  'group',     'max',   'group',  'group',    'max',     ],
              ['lemma_id',  'lemma_tag',  'num',  'merkmal',   'meta',  'data',   'data_id',  'data_score',   ],        
             )


696 rows less, now 5611 rows


In [547]:
# suche_debug wiktionary_merkmal_abk
search_str(wiktionary_merkmal_abk, suche_debug)

,lemma_id,lemma_tag,num,merkmal,meta,data,data_id,data_score
1022,Euro,NN,,abk,manuell,EUR,Euro~EUR,153.715988


In [548]:
# anschauen
grid(wiktionary_merkmal_abk.sort_values('data_score',ascending=False))

5611 rows


,lemma_id,lemma_tag,num,merkmal,meta,data,data_id,data_score
3562,Wasser,NN,1,abk,,H2O,Wasser~H2O,6229.017090
1957,Krankenhaus,NN,1,abk,,Krkhs.,Krankenhaus~Krkhs.,5780.692871
1956,Krankenhaus,NN,1,abk,,Krkhs,Krankenhaus~Krkhs,5780.692871
1955,Krankenhaus,NN,1,abk,,Krhs.,Krankenhaus~Krhs.,5780.692871
1954,Krankenhaus,NN,1,abk,,Krhs,Krankenhaus~Krhs,5780.692871
...,...,...,...,...,...,...,...,...
4274,getilgt,ADJA,,abk,,get.,getilgt~get.,0.711443
4410,infiziert_1,,,abk,,inf.,infiziert_1~inf.,0.174397
4409,infiziert_1,,,abk,,inf,infiziert_1~inf,0.174397
4210,gedeckt_1,ADJA,,abk,,ged,gedeckt_1~ged,0.086008


### wiktionary_lemma pflegen

In [549]:
# suche_debug 
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
453,Euro,11127,0,Euro,euro,True,1,NN,NN,6Y69DrZ3KB,NOM,N,Einheit manuell,,1403.598999
2607,zu_viel,89575,0,zu viel,zu viel,False,2,ADV,ADV,JGmS0Ezhw2,ADV,A,Wortverbindung manuell,,51.254711
29704,Euro_1,11127,1,Euro,euro,True,1,NN,NN,ZoNMYpSagd,NOM,N,,,36.994484


In [550]:
# Alle auflösbaren Abkürzungen mit eigenem Eintrag in wiktionary_lemma
mask_K = wiktionary_merkmal_abk.lemma_id != wiktionary_merkmal_abk.data_id
wiktionary_merkmal_abk[mask_K].tail(3)

,lemma_id,lemma_tag,num,merkmal,meta,data,data_id,data_score
5608,üblich,ADJA,1,abk,,übl.,üblich~übl.,75.100830
5609,üblicherweise,ADV,1,abk,,üblw,üblicherweise~üblw,22.103651
5610,üblicherweise,ADV,1,abk,,üblw.,üblicherweise~üblw.,22.103651


In [551]:
# Obsolet
# wiktionary_lemma.member nachtragen
# Nicht jede Abkürzung hat einen eigenen Eintrag in wiktionary_lemma.
# Aber wenn, dann ist dieser mit member 'Abkürzung' gekennzeichnet.

mask1 = ~wiktionary_lemma.member.str.contains('Abkürzung')                       # nicht als Abkürzung gekennzeichnet
mask2 = wiktionary_lemma.lemma_id.isin(wiktionary_merkmal_abk[mask_K].data_id)   # ist aber eine auflösbare Abkürzung
mask = mask1  &  mask2
check_mask( wiktionary_lemma, mask, 0, 169 )

wiktionary_lemma.loc[mask,'member'] = (wiktionary_lemma[mask].member + ' Abkürzung').str.strip()
grid( wiktionary_lemma, mask )

check_mask:  0 rows
No rows, mask filters them all away


### translate_abk
* translate_abk1: Nimmt die in wiktionary_lemma gelisteten Abkürzungen und löst sie über ihr def-Merkmal auf
* translate_abk2: Basiert auf wiktionary_merkmal_abk

#### translate_abk1

In [552]:
# translate_abk1: Kurzformen aus wiktionary_lemma
mask_l = wiktionary_lemma.member.str.contains('Abkürzung')
spalten = ['lemma_id','lemma','tag','member','lemma_score',]
translate_abk1 = wiktionary_lemma[mask_l][spalten]
translate_abk1 = pak.rename_col(translate_abk1,'lemma',       'kurz')
translate_abk1 = pak.rename_col(translate_abk1,'lemma_id',    'kurz_id')
translate_abk1 = pak.rename_col(translate_abk1,'tag',         'kurz_tag')
translate_abk1 = pak.rename_col(translate_abk1,'member',      'kurz_member')
translate_abk1 = pak.rename_col(translate_abk1,'lemma_score', 'kurz_score')
translate_abk1 = pak.reset_index(translate_abk1)
pak.sample(translate_abk1)

,kurz_id,kurz,kurz_tag,kurz_member,kurz_score
0,24/7,24/7,ADJA,Abkürzung manuell ergänzt,14.942436
1,2D,2D,ADJA,manuell ergänzt Abkürzung,21.063148
22,http,http,TRUNC,Abkürzung manuell ergänzt,19.930384
33,&c.,&c.,TODO,Abkürzung,4.049606
455,BH,BH,NN,Abkürzung,189.630875
667,BerlinFG,BerlinFG,TODO,Abkürzung,5.788859
2975,Stasi,Stasi,NN,Abkürzung noPl,9214.994141
4463,z._A.,z. A.,TODO,Abkürzung,0.102266
4529,μm,μm,TODO,Abkürzung,41.927307


In [553]:
# Langform verknüpfen
mask_m = (wiktionary_merkmal.merkmal == 'def') # alle Definitionen, nicht nur die der Abkürzungen

# lang_id
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_merkmal[mask_m], 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'data_id',
                                              col_rename = 'lang_id',
                                              func =       'first',
                                            )
# lang
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_merkmal[mask_m], 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'data',
                                              col_rename = 'lang',
                                              func =       'first',
                                            )

# lang_tag
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'tag',
                                              col_rename = 'lang_tag',
                                            )


# lang_member
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'member',
                                              col_rename = 'lang_member',
                                            )

# lang_score
translate_abk1 = pak.update_col(              df_to =      translate_abk1, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'lemma_score',
                                              col_rename = 'lang_score',
                                            )

translate_abk1['todo'] = 'lemma' 
translate_abk1['rank'] = 1.8 
print()
pak.sample(translate_abk1)

update_col: func first applied, 178789 records less!
update_col: 2711 cells written into new column
update_col: func first applied, 178789 records less!
update_col: 2711 cells written into new column
update_col: 2616 cells written into new column
update_col: 2616 cells written into new column
update_col: 2616 cells written into new column



,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank
0,24/7,24/7,ADJA,Abkürzung manuell ergänzt,14.942436,Stunde,Stunde,NN,Einheit manuell,6160.494141,lemma,1.8
22,http,http,TRUNC,Abkürzung manuell ergänzt,19.930384,<NA>,<NA>,<NA>,<NA>,NaN,lemma,1.8
31,on,on,FM,manuell ergänzt Abkürzung,21.076181,online,online,ADJA,noSteig noFlex,319.479553,lemma,1.8
33,&c.,&c.,TODO,Abkürzung,4.049606,et_cetera,et cetera,ADV,Wortverbindung manuell,30.591106,lemma,1.8
40,1_Joh,1 Joh,TODO,Abkürzung,15.758732,1._Johannesbrief,1. Johannesbrief,NN,adjNN Wortverbindung adjNN noPl,74.711502,lemma,1.8
455,BH,BH,NN,Abkürzung,189.630875,Büstenhalter,Büstenhalter,NN,manuell,1189.476318,lemma,1.8
605,BVerfGG,BVerfGG,TODO,Abkürzung,11.272240,Gesetz,Gesetz,NN,,15998.839844,lemma,1.8
667,BerlinFG,BerlinFG,TODO,Abkürzung,5.788859,<NA>,<NA>,<NA>,<NA>,NaN,lemma,1.8
2179,MCU,MCU,TODO,Abkürzung,19.673538,Mikrocontroller,Mikrocontroller,NN,,356.384735,lemma,1.8
2975,Stasi,Stasi,NN,Abkürzung noPl,9214.994141,Geheimpolizei,Geheimpolizei,NN,,4278.938477,lemma,1.8


#### translate_abk2

In [554]:
# translate_abk2: aus wiktionary_merkmal_abk
translate_abk2 = wiktionary_merkmal_abk.copy()
translate_abk2 = pak.drop_cols(translate_abk2,['num','merkmal'])
translate_abk2 = pak.rename_col(translate_abk2,'lemma_id',  'lang_id')
translate_abk2 = pak.rename_col(translate_abk2,'lemma_tag', 'lang_tag')
translate_abk2 = pak.rename_col(translate_abk2,'data_score','lang_score')
translate_abk2 = pak.rename_col(translate_abk2,'data',      'kurz')
translate_abk2 = pak.rename_col(translate_abk2,'data_id',   'kurz_id')
translate_abk2 = pak.rename_col(translate_abk2,'meta',      'kurz_member')
translate_abk2 = pak.reset_index(translate_abk2)
translate_abk2 = pak.move_cols(translate_abk2,['kurz_id','kurz'])
pak.sample(translate_abk2)

,kurz_id,kurz,lang_id,lang_tag,kurz_member,lang_score
0,-prozentig~-%ig,-%ig,-prozentig,ZFIX,,14.858729
194,Anmerkung_der_Redaktion_M~Anm d Red,Anm d Red,Anmerkung_der_Redaktion_M,KOMBI,manuell,2.239461
1248,Geburtstag~Geb,Geb,Geburtstag,NN,,1581.322998
2691,Prozent~%,%,Prozent,NN,manuell,143.270859
2722,Quadratmikrometer~μm²,μm²,Quadratmikrometer,NN,,41.616608
3182,Strafe~Str,Str,Strafe,NN,,411.752258
3562,Wasser~H2O,H2O,Wasser,NN,,6229.017090
4021,dieselbe~dies,dies,dieselbe,,,1.211093
4210,gedeckt_1~ged,ged,gedeckt_1,ADJA,,0.086008
5608,üblich~übl.,übl.,üblich,ADJA,,75.100830


In [555]:
# keine eigene kurz_id
mask = translate_abk2.kurz_id == translate_abk2.lang_id
check_mask(translate_abk2,mask, 0) 
translate_abk2.loc[mask,'kurz_id'] = ''

check_mask:  0 rows


In [556]:
# kurz_id ist nicht in wiktionary_lemma
#mask1 = ~translate_abk2.kurz_id.isin(wiktionary_lemma.lemma_id)
#mask2 =  translate_abk2.kurz_id != ''
#mask = mask1  &  mask2 
#check_mask(translate_abk2,mask,4503) 
#translate_abk2.loc[mask,'kurz_id'] = ''

In [557]:
# War das richtig?
bgc('orange')

#### translate_abk

In [558]:
# kurz_tag
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'tag',
                                              col_rename = 'kurz_tag',
                                              func =       'first',
                                            )


# kurz_member
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'member',
                                              col_rename = 'kurz_member',
                                              cond =       'null'
                                            )

# kurz_score
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'kurz_id',
                                              right_on =   'lemma_id',
                                              col =        'lemma_score',
                                              col_rename = 'kurz_score',
                                            )

# lang
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'lemma',
                                              col_rename = 'lang',
                                            )


# lang_member
translate_abk2 = pak.update_col(              df_to =      translate_abk2, 
                                              df_from =    wiktionary_lemma, 
                                              left_on =    'lang_id',
                                              right_on =   'lemma_id',
                                              col =        'member',
                                              col_rename = 'lang_member',
                                            )


translate_abk2 = pak.move_cols(translate_abk2,['kurz_tag','kurz_member','kurz_score'],'kurz')
translate_abk2 = pak.move_cols(translate_abk2,['lang','lang_tag','lang_member'],'lang_id')
translate_abk2['rank'] = 1.0 
translate_abk2['todo'] = 'merkmal' 
print()
pak.sample(translate_abk2)

update_col: func first applied, but it was pointless!
update_col: 0 cells written into new column
update_col: 0 cells written into existing column
update_col: 0 cells written into new column
update_col: 5591 cells written into new column
update_col: 5591 cells written into new column



,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,rank,todo
0,-prozentig~-%ig,-%ig,<NA>,,NaN,-prozentig,-prozentig,ZFIX,Gebundenes_Lexem,14.858729,1.0,merkmal
194,Anmerkung_der_Redaktion_M~Anm d Red,Anm d Red,<NA>,manuell,NaN,Anmerkung_der_Redaktion_M,Anmerkung der Redaktion,KOMBI,Wortverbindung manuell,2.239461,1.0,merkmal
1248,Geburtstag~Geb,Geb,<NA>,,NaN,Geburtstag,Geburtstag,NN,,1581.322998,1.0,merkmal
2691,Prozent~%,%,<NA>,manuell,NaN,Prozent,Prozent,NN,Einheit manuell,143.270859,1.0,merkmal
2722,Quadratmikrometer~μm²,μm²,<NA>,,NaN,Quadratmikrometer,Quadratmikrometer,NN,,41.616608,1.0,merkmal
3182,Strafe~Str,Str,<NA>,,NaN,Strafe,Strafe,NN,,411.752258,1.0,merkmal
3562,Wasser~H2O,H2O,<NA>,,NaN,Wasser,Wasser,NN,,6229.017090,1.0,merkmal
4021,dieselbe~dies,dies,<NA>,,NaN,dieselbe,<NA>,,<NA>,1.211093,1.0,merkmal
4210,gedeckt_1~ged,ged,<NA>,,NaN,gedeckt_1,gedeckt,ADJA,Ptz2,0.086008,1.0,merkmal
4858,okay~O K,O K,<NA>,,NaN,okay,okay,ADJA,noSteig noFlex,39.506165,1.0,merkmal


In [559]:
# translate_abk erstellen
translate_abk = pd.concat([translate_abk1,translate_abk2], ignore_index=True)
translate_abk = pak.change_datatype(translate_abk)
translate_abk_trash = translate_abk.head(0)
translate_abk_trash['msg'] = None

change_datatype 
todo                 --> pd.string 
rank                 --> np.float32
change_datatype before: 5.7 MB after: 5.7 MB



In [560]:
# fillna

translate_abk['kurz_id']     = translate_abk.kurz_id.fillna('')
translate_abk['lang_id']     = translate_abk.lang_id.fillna('')

translate_abk['kurz']        = translate_abk.kurz.fillna('')
translate_abk['lang']        = translate_abk.lang.fillna('')

translate_abk['kurz_tag']    = translate_abk.kurz_tag.fillna('').str.replace('TODO','').str.strip()
translate_abk['lang_tag']    = translate_abk.lang_tag.fillna('').str.replace('TODO','').str.strip()

translate_abk['kurz_member'] = translate_abk.kurz_member.fillna('')
translate_abk['lang_member'] = translate_abk.lang_member.fillna('')

translate_abk['kurz_score']  = translate_abk.kurz_score.fillna(0)
translate_abk['lang_score']  = translate_abk.lang_score.fillna(0)

#### Belöschen und ranken

In [561]:
# konkurrenz: Wie viele konkurrierende lang_id gibt es?
gruppiert = translate_abk.groupby(['kurz'])
translate_abk['konkurrenz'] = gruppiert.lang_id.transform( 'nunique' ) 

In [562]:
grid(translate_abk,pak.sample)

12 rows out of 10141


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
0,24/7,24/7,ADJA,Abkürzung manuell ergänzt,14.942436,Stunde,Stunde,NN,Einheit manuell,6160.494141,lemma,1.8,1
22,http,http,TRUNC,Abkürzung manuell ergänzt,19.930384,,,,,0.000000,lemma,1.8,1
31,on,on,FM,manuell ergänzt Abkürzung,21.076181,online,online,ADJA,noSteig noFlex,319.479553,lemma,1.8,1
33,&c.,&c.,,Abkürzung,4.049606,et_cetera,et cetera,ADV,Wortverbindung manuell,30.591106,lemma,1.8,1
2975,Stasi,Stasi,NN,Abkürzung noPl,9214.994141,Geheimpolizei,Geheimpolizei,NN,,4278.938477,lemma,1.8,1
2981,Str.,Str.,,Abkürzung,135.142532,Strafe,Strafe,NN,,3705.770508,lemma,1.8,16
3337,Ws.,Ws.,,Abkürzung,11.028746,Wasser,Wasser,NN,,56061.152344,lemma,1.8,1
4529,μm,μm,,Abkürzung,41.927307,,,,,0.000000,lemma,1.8,2
4530,-prozentig~-%ig,-%ig,,,0.000000,-prozentig,-prozentig,ZFIX,Gebundenes_Lexem,14.858729,merkmal,1.0,1
7221,Prozent~%,%,,manuell,0.000000,Prozent,Prozent,NN,Einheit manuell,143.270859,merkmal,1.0,1


In [563]:
# ID fehlt
maskA = translate_abk.kurz_id == ''
maskB = translate_abk.lang_id == ''
mask = maskA | maskB
check_mask(translate_abk,mask,1864)

translate_abk.loc[mask,'rank'] *= 0.5
translate_abk[mask].sort_values('kurz_score', ascending=False)

check_mask:  1819 rows


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
2323,NGO,NGO,NN,Abkürzung,2046.884277,,,,,0.0,lemma,0.9,2
17,USA,USA,NE,Geo Geo_land noSg Abkürzung,1161.254395,,,,,0.0,lemma,0.9,3
2840,SMS,SMS,NN,Abkürzung,870.798462,,,,,0.0,lemma,0.9,1
10,GmbH,GmbH,NN,manuell ergänzt Abkürzung,682.923218,,,,,0.0,lemma,0.9,1
206,ASCII,ASCII,NN,Abkürzung noPl,619.340088,,,,,0.0,lemma,0.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4370,umg.,umg.,,Abkürzung,0.185524,,,,,0.0,lemma,0.9,1
3930,k._A.,k. A.,,Abkürzung,0.178712,,,,,0.0,lemma,0.9,2
1445,FDA_1,FDA,,Abkürzung,0.151688,,,,,0.0,lemma,0.9,1
4477,zit._n.,zit. n.,,Abkürzung,0.137390,,,,,0.0,lemma,0.9,1


In [564]:
# Anfangsbuchstabe ungleich
mask1 = (translate_abk.lang.str[0].str.lower()  !=  translate_abk.kurz.str[0].str.lower())
mask2 = ~translate_abk.kurz.str[0].str.isdigit() 
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask = mask1  &  mask2  &  mask3
check_mask(translate_abk,mask,984)

translate_abk.loc[mask,'rank'] *= 0.8
translate_abk[mask]

check_mask:  887 rows


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
7,BDSM,BDSM,NN,Abkürzung manuell ergänzt,208.074417,Sammelbezeichnung,Sammelbezeichnung,NN,,277.477875,lemma,1.44,2
12,ISBN,ISBN,NN,manuell ergänzt Abkürzung,103.012543,Nummer,Nummer,NN,Einheit manuell,7274.298828,lemma,1.44,1
13,LGBT,LGBT,NN,Abkürzung manuell,134.721130,Sammelbezeichnung,Sammelbezeichnung,NN,,277.477875,lemma,1.44,1
33,&c.,&c.,,Abkürzung,4.049606,et_cetera,et cetera,ADV,Wortverbindung manuell,30.591106,lemma,1.44,1
70,A.C.A.B.,A.C.A.B.,,Abkürzung,59.133762,Parole,Parole,NN,,1149.086792,lemma,1.44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10085,Ölhafen~Hafen,Hafen,,,0.000000,Ölhafen,Ölhafen,NN,,27.923941,merkmal,0.80,1
10086,Österreich~AUT,AUT,,,0.000000,Österreich,Österreich,NE,Geo Geo_land,195.422760,merkmal,0.80,1
10087,Überhorizontradar~OTH Radar,OTH Radar,,,0.000000,Überhorizontradar,Überhorizontradar,NN,,34.843922,merkmal,0.80,1
10088,Überhorizontradar~OTH-Radar,OTH-Radar,,,0.000000,Überhorizontradar,Überhorizontradar,NN,,34.843922,merkmal,0.80,1


In [565]:
# nichtssagende_abk1
mask1 =  translate_abk.kurz.str.len() == 3
mask2 =  translate_abk.kurz.str.startswith('ge')
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask4 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(translate_abk,mask,20)

msg = 'nichtssagende_abk1'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  24 rows
Move 24 rows from 10141
24 rows


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
5810,Gegisch~geg,geg,,,0.0,Gegisch,Gegisch,NN,,52.228848,merkmal,1.0,nichtssagende_abk1,2
5828,Gemeingermanisch~gem,gem,,,0.0,Gemeingermanisch,Gemeingermanisch,NN,,31.228254,merkmal,1.0,nichtssagende_abk1,3
5865,Germanisch~gem,gem,,,0.0,Germanisch,Germanisch,NN,,59.976376,merkmal,1.0,nichtssagende_abk1,3
6915,Neuhochdeutsch~ger,ger,,,0.0,Neuhochdeutsch,Neuhochdeutsch,NN,,71.346214,merkmal,0.8,nichtssagende_abk1,1
8726,gebildet~geb,geb,,,0.0,gebildet,gebildet,ADJA,,43.955456,merkmal,1.0,nichtssagende_abk1,3
8728,geboren~geb,geb,,,0.0,geboren,geboren,ADJA,noSteig,38.656197,merkmal,1.0,nichtssagende_abk1,3
8738,gedeckt~ged,ged,,,0.0,gedeckt,gedeckt,ADJA,,20.119692,merkmal,1.0,nichtssagende_abk1,5
8740,gedeckt_1~ged,ged,,,0.0,gedeckt_1,gedeckt,ADJA,Ptz2,0.086008,merkmal,1.0,nichtssagende_abk1,5
8742,gediegen~ged,ged,,,0.0,gediegen,gediegen,ADJA,,63.110580,merkmal,1.0,nichtssagende_abk1,5
8744,gedient~ged,ged,,,0.0,gedient,gedient,ADJA,Ptz2,0.910134,merkmal,1.0,nichtssagende_abk1,5


In [566]:
# nichtssagende_abk2
mask1 =  translate_abk.kurz.str.len() == 4
mask2 =  translate_abk.kurz.str.startswith('ge')
mask3 =  translate_abk.kurz.str.endswith('.')
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_abk,mask,34)

msg = 'nichtssagende_abk2'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  39 rows
Move 39 rows from 10117
39 rows out of 63


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
3753,geb.,geb.,,Abkürzung,88.242928,geborener,geborener,,,0.000000,lemma,1.8,nichtssagende_abk2,4
3755,ged.,ged.,,Abkürzung,13.196012,gedämpft,gedämpft,ADJA,Ptz2,0.837086,lemma,1.8,nichtssagende_abk2,5
3756,gef.,gef.,,Abkürzung,34.923553,gefallen,gefallen,VVFIN,,101.318176,lemma,1.8,nichtssagende_abk2,1
3757,geg.,geg.,,Abkürzung,9.004710,gegeben,gegeben,ADJA,Ptz2,0.668355,lemma,1.8,nichtssagende_abk2,3
3759,geh.,geh.,,Abkürzung,21.829100,gehoben,gehoben,ADJA,,340.262726,lemma,1.8,nichtssagende_abk2,3
3760,gek.,gek.,,Abkürzung,18.374691,gekennzeichnet,gekennzeichnet,ADJA,Ptz2,0.453479,lemma,1.8,nichtssagende_abk2,1
3761,gel.,gel.,,Abkürzung,51.948780,geladen,geladen,ADJA,Ptz2,0.538190,lemma,1.8,nichtssagende_abk2,1
3763,gem.,gem.,,Abkürzung,115.115265,gemahlen,gemahlen,ADJA,Ptz2,1.769529,lemma,1.8,nichtssagende_abk2,4
3765,gen.,gen.,,Abkürzung,37.976196,genannt,genannt,ADJA,Ptz2,0.509264,lemma,1.8,nichtssagende_abk2,2
3774,gep.,gep.,,Abkürzung,13.351590,gepaart,gepaart,ADJA,Ptz2,0.807867,lemma,1.8,nichtssagende_abk2,1


In [567]:
# Abk_kürzt_nicht
mask1 =  translate_abk.lang.str.len() <= translate_abk.kurz.str.len() 
mask2 =  translate_abk.lang_id != ''
mask3 =  translate_abk.kurz_id != ''
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz.isin(['okay',]) 
mask6 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6
check_mask(translate_abk,mask,228)

msg = 'Abk_kürzt_nicht'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
gridt( translate_abk_trash, msg )

check_mask:  210 rows
Move 210 rows from 10078
210 rows out of 273


In [568]:
#mask = translate_abk_trash.msg == 'Abk_kürzt_nicht'
#gridt(translate_abk_trash,mask)

In [569]:
# zu_kurz_end_Vokal
mask1 =  translate_abk.kurz.str.len() == 3 
mask2 =  translate_abk.kurz.str.lower().str.endswith(('a.','e.','i.','o.','u.','ä.','ö.','ü.','y.'))
mask3 =  translate_abk.kurz_tag == ''
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_abk,mask,21)

msg = 'zu_kurz_end_Vokal'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  21 rows
Move 21 rows from 9868
21 rows out of 294


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
807,Co.,Co.,,Abkürzung,16.382223,Kompanie,Kompanie,NN,,1337.275879,lemma,1.44,zu_kurz_end_Vokal,1
1624,Ga.,Ga.,,Abkürzung,6.577270,Garage,Garage,NN,,4282.641602,lemma,1.80,zu_kurz_end_Vokal,1
2171,MA.,MA.,,Abkürzung,5.511693,Mittelalter,Mittelalter,NN,noPl,508.786438,lemma,1.80,zu_kurz_end_Vokal,1
2597,Pi.,Pi.,,Abkürzung,7.166071,Piaster,Piaster,NN,,437.788239,lemma,1.80,zu_kurz_end_Vokal,1
2665,Qu.,Qu.,,Abkürzung,13.877033,Quartal,Quartal,NN,,819.089478,lemma,1.80,zu_kurz_end_Vokal,1
2788,Ri.,Ri.,,Abkürzung,3.053587,Richter,Richter,NN,,5028.521484,lemma,1.80,zu_kurz_end_Vokal,1
3067,Ti.,Ti.,,Abkürzung,13.471220,Tiroler,Tiroler,ADJA,noSteig noFlex,41.235859,lemma,1.80,zu_kurz_end_Vokal,1
3313,Wa.,Wa.,,Abkürzung,22.748615,Waage,Waage,NN,,3058.229492,lemma,1.80,zu_kurz_end_Vokal,1
3401,Zi.,Zi.,,Abkürzung,2.227140,Zimmer,Zimmer,NN,,5176.409668,lemma,1.80,zu_kurz_end_Vokal,1
3457,ae.,ae.,,Abkürzung,4.236610,altenglisch,altenglisch,ADJA,noSteig,379.057709,lemma,1.80,zu_kurz_end_Vokal,1


In [570]:
# zu_kurz_start_Vokal
mask1 =  translate_abk.kurz.str.len() == 3 
mask2 =  translate_abk.kurz.str.endswith('.')
mask3 =  translate_abk.kurz.str.lower().str.startswith(('a','e','i','o','u','ä','ö','ü','y'))
mask4 =  translate_abk.kurz_tag == ''
mask5 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask6 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6
check_mask(translate_abk,mask,19)

msg = 'zu_kurz_start_Vokal'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  19 rows
Move 19 rows from 9847
19 rows out of 313


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
376,Az.,Az.,,Abkürzung,12.935077,Aktenzeichen,Aktenzeichen,NN,,429.356567,lemma,1.80,zu_kurz_start_Vokal,1
1373,Ed.,Ed.,,Abkürzung,8.791534,Edition,Edition,NN,,453.229187,lemma,1.80,zu_kurz_start_Vokal,1
1425,Ew.,Ew.,,Abkürzung,2.276727,Einwohner,Einwohner,NN,manuell,1193.954590,lemma,1.80,zu_kurz_start_Vokal,1
1428,Ez.,Ez.,,Abkürzung,6.786630,Einzahl,Einzahl,NN,,743.405518,lemma,1.80,zu_kurz_start_Vokal,1
2452,Ob.,Ob.,,Abkürzung,13.350096,Oberbefehlshaber,Oberbefehlshaber,NN,,618.614258,lemma,1.80,zu_kurz_start_Vokal,1
3361,Yd.,Yd.,,Abkürzung,2.251934,Yard,Yard,NN,,119.029610,lemma,1.80,zu_kurz_start_Vokal,1
3499,am.,am.,,Abkürzung,7.788982,amerikanisch,amerikanisch,ADJA,,974.893066,lemma,1.80,zu_kurz_start_Vokal,1
3506,an.,an.,,Abkürzung,2.342843,altnordisch,altnordisch,ADJA,noSteig,144.764542,lemma,1.80,zu_kurz_start_Vokal,1
3527,ar.,ar.,,Abkürzung,7.716775,arisch,arisch,ADJA,noSteig,157.932709,lemma,1.80,zu_kurz_start_Vokal,1
3543,as.,as.,,Abkürzung,5.475056,altsächsisch,altsächsisch,ADJA,noSteig,328.152374,lemma,1.80,zu_kurz_start_Vokal,1


In [571]:
# Ranking: Ist die Abkürzung in lexeme_manuell?
mask = translate_abk.kurz_member.str.contains('manuell')
translate_abk.loc[mask,'rank'] *= 2.0

mask = translate_abk.lang_member.str.contains('manuell')
translate_abk.loc[mask,'rank'] *= 2.0

In [572]:
# Ranking: Ist die Abkürzung beidseitig verknüpft?
gruppiert = translate_abk.groupby(['kurz_id'])
translate_abk['rank2'] = gruppiert.todo.transform( 'nunique' ) 

mask = translate_abk.rank2 > 1
translate_abk.loc[mask,'rank'] *= 5.0
translate_abk = pak.drop_cols(translate_abk,'rank2')

In [573]:
# Abkürzung hat eigenen Eintrag
mask = translate_abk.kurz_id != ''
translate_abk.loc[mask,'rank'] *= 2.0

In [574]:
# Ranking: lang_score
translate_abk['rank'] *= (translate_abk.lang_score + 0.2)

In [575]:
# Ranking: Kein tag
mask = translate_abk.lang_tag == ''
translate_abk.loc[mask,'rank'] *= 0.2
translate_abk[mask]

,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
4,5G,5G,NN,Abkürzung manuell ergänzt,8.974190,,,,,0.000000,lemma,0.144000,1
5,9/11,9/11,NN,Abkürzung manuell ergänzt,55.178623,Terroranschläge,Terroranschläge,,,0.000000,lemma,0.288000,2
10,GmbH,GmbH,NN,manuell ergänzt Abkürzung,682.923218,,,,,0.000000,lemma,0.144000,1
11,HMM,HMM,ITJ,Abkürzung manuell ergänzt,7.572477,,,,,0.000000,lemma,0.144000,1
14,Opa_1,Opa,NN,Abkürzung Person_pre manuell ergänzt,8.245752,,,,,0.000000,lemma,0.144000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10051,zu_Händen~zHd,zHd,,,0.000000,zu_Händen,zu Händen,,Wortverbindung,10.456056,merkmal,4.262422,1
10056,zum_Exempel~z E,z E,,,0.000000,zum_Exempel,zum Exempel,,Wortverbindung,4.680923,merkmal,1.952369,1
10057,zum_Exempel~z. E.,z. E.,,,0.000000,zum_Exempel,zum Exempel,,Wortverbindung,4.680923,merkmal,1.952369,1
10058,zum_Exempel~z.E.,z.E.,,,0.000000,zum_Exempel,zum Exempel,,Wortverbindung,4.680923,merkmal,1.952369,1


In [576]:
# sicherheitshalber einmal sortieren
translate_abk = translate_abk.sort_values(['kurz','lang_score'],ascending=[True,False]).reset_index(drop=True)

In [577]:
# Ranking betrachten
grid(translate_abk.sort_values(['konkurrenz','kurz','rank',],ascending=[False,True,False]))

9828 rows


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
5940,Straße~Str,Str,,manuell,0.000000,Straße,Straße,NN,manuell,1416.006226,merkmal,11329.649414,16
5939,Strand~Str,Str,,,0.000000,Strand,Strand,NN,,2786.176025,merkmal,5572.751953,16
5941,Strom~Str,Str,,,0.000000,Strom,Strom,NN,,627.335266,merkmal,1255.070557,16
5942,Struktur~Str,Str,,,0.000000,Struktur,Struktur,NN,,556.234680,merkmal,1112.869385,16
5943,Strafe~Str,Str,,,0.000000,Strafe,Strafe,NN,,411.752258,merkmal,823.904541,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9819,überschüssig~übsch,übsch,,,0.000000,überschüssig,überschüssig,ADJA,noSteig,23.642769,merkmal,47.685539,1
9820,übsch.,übsch.,,Abkürzung,2.284992,überschüssig,überschüssig,ADJA,noSteig,70.928307,lemma,256.061890,1
9821,überschüssig~übsch.,übsch.,,,0.000000,überschüssig,überschüssig,ADJA,noSteig,23.642769,merkmal,47.685539,1
9824,Mikrosievert~μSv,μSv,,,0.000000,Mikrosievert,Mikrosievert,NN,,26.903440,merkmal,43.365505,1


In [578]:
# rank!
translate_abk = pak.rank(translate_abk, col_score='rank', cols_group='kurz', on_conflict='first')

rank: 2063 rows less, now 7765 rows


In [579]:
?pak.replace_str

Signature: pak.replace_str(series, translationtable)
Docstring:
Replaces substrings from a Series of strings according to a translationtable.
* translationtable: Can be a dict, a list of tuples or a DataFrame with two columns.
  Example: {'President Trump':'Trump',   'HELLO':'Hello'}
  or      [('President Trump','Trump'), ('HELLO','Hello')]
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/string.py
Type:      function


In [580]:
# Verwaiste Abkürzungen in wiktionary_lemma kennzeichnen  
mask = translate_abk.lang_id == ''
check_mask(translate_abk,mask,1745)
verwaist_kurz_id = translate_abk[mask].kurz_id

mask = wiktionary_lemma.lemma_id.isin(verwaist_kurz_id) 
check_mask(wiktionary_lemma,mask,1745)

wiktionary_lemma.loc[mask,'member'] = pak.replace_str( wiktionary_lemma[mask].member, {'Abkürzung':'Abkürzung_verwaist' } )
gridt(wiktionary_lemma[mask],pak.sample)

check_mask:  1746 rows
check_mask:  1746 rows
15 rows out of 1746


In [581]:
# Verwaiste Abkürzungen in translate_abk löschen
mask = translate_abk.lang_id == ''
check_mask(translate_abk,mask,1745)

msg = 'Abkürzung_verwaist'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)
grid( translate_abk_trash, msg )

check_mask:  1746 rows
Move 1746 rows from 7765
1746 rows out of 2059


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
48,5G,5G,NN,Abkürzung manuell ergänzt,8.974190,,,,,0.0,lemma,0.144,Abkürzung_verwaist,1
3038,GmbH,GmbH,NN,manuell ergänzt Abkürzung,682.923218,,,,,0.0,lemma,0.144,Abkürzung_verwaist,1
3183,HMM,HMM,ITJ,Abkürzung manuell ergänzt,7.572477,,,,,0.0,lemma,0.144,Abkürzung_verwaist,1
4801,Opa_1,Opa,NN,Abkürzung Person_pre manuell ergänzt,8.245752,,,,,0.0,lemma,0.144,Abkürzung_verwaist,1
5032,PetA,Peta,CARD,manuell ergänzt Abkürzung,12.209723,,,,,0.0,lemma,0.144,Abkürzung_verwaist,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9750,ÜAG,ÜAG,,Abkürzung,4.995471,,,,,0.0,lemma,0.072,Abkürzung_verwaist,1
9751,ÜD,ÜD,,Abkürzung,5.416959,,,,,0.0,lemma,0.072,Abkürzung_verwaist,1
9752,ÜE,ÜE,,Abkürzung,7.028529,,,,,0.0,lemma,0.072,Abkürzung_verwaist,1
9774,ödp,ödp,,Abkürzung,27.600363,,,,,0.0,lemma,0.072,Abkürzung_verwaist,1


In [582]:
# zu häufig und Anfangsbuchstabe ungleich

erlaubt = ['Ingenieur','okay','per Adresse','Professor','Kalorie','insbesondere','insbesonders']

a = pak.analyse_freqs(translate_abk,'lang')
mask_a = a.lang_count >= 5 
mask1 = translate_abk.lang.isin(a[mask_a].lang)
mask2 = (translate_abk.lang.str[0].str.lower()  !=  translate_abk.kurz.str[0].str.lower())
mask3 = ~translate_abk.kurz.str[0].str.isdigit() 
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5A = translate_abk.kurz_member == 'Abkürzung'  # genau gleich 'Abkürzung', also nichts zusätzliches 
mask5B = translate_abk.kurz_member == ''           # oder ganz leer 
mask6 = ~translate_abk.lang.isin(erlaubt)
mask = mask1  &  mask2  &  mask3  &  mask4  &  (mask5A | mask5B)  &  mask6  
check_mask(translate_abk,mask,170)

msg = 'zu_häufig'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)
grid( translate_abk_trash, msg )

check_mask:  176 rows
Move 176 rows from 6019
176 rows out of 2235


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
180,AKH,AKH,NN,Abkürzung,66.509537,Krankenhaus,Krankenhaus,NN,,52026.234375,lemma,149836.125000,zu_häufig,1
1065,BauFG,BauFG,,Abkürzung,14.304992,Gesetz,Gesetz,NN,,15998.839844,lemma,46077.234375,zu_häufig,1
2166,EEAG,EEAG,,Abkürzung,10.641104,Gesetz,Gesetz,NN,,15998.839844,lemma,46077.234375,zu_häufig,1
2206,ELFG,ELFG,,Abkürzung,22.796688,Gesetz,Gesetz,NN,,15998.839844,lemma,46077.234375,zu_häufig,1
2220,EMVG,EMVG,,Abkürzung,35.377846,Gesetz,Gesetz,NN,,15998.839844,lemma,46077.234375,zu_häufig,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,BVerfGE,BVerfGE,,Abkürzung,10.220346,Entscheidungen,Entscheidungen,,,0.000000,lemma,0.115200,zu_häufig,1
1016,BVerwGE,BVerwGE,,Abkürzung,10.220346,Entscheidungen,Entscheidungen,,,0.000000,lemma,0.115200,zu_häufig,1
1080,BayObLGSt,BayObLGSt,,Abkürzung,12.903807,Entscheidungen,Entscheidungen,,,0.000000,lemma,0.115200,zu_häufig,1
1081,BayObLGZ,BayObLGZ,,Abkürzung,12.903807,Entscheidungen,Entscheidungen,,,0.000000,lemma,0.115200,zu_häufig,1


In [583]:
a = pak.analyse_freqs(translate_abk,['lang','kurz'])
gridt(a)

3106 rows


In [584]:
# erlaubt_und_verboten

verboten     = ['Bundesministerium','Brandmeister','Braumeister','Rundfunk','Ortsnamengrundwort','Kurzwort','Diminutiv','diminuendo']
doch_erlaubt = []

mask1 =  translate_abk.lang.isin(verboten) 
mask2 = ~translate_abk.lang.isin(doch_erlaubt) 
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask4 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4 
#grid(translate_abk,mask)
check_mask(translate_abk,mask,67)

msg = 'erlaubt_und_verboten'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  66 rows
Move 66 rows from 5843
66 rows out of 2301


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
5285,RBB,RBB,,Abkürzung,48.454773,Rundfunk,Rundfunk,NN,noPl,1879.257202,lemma,6766.045410,erlaubt_und_verboten,1
8898,rbb,rbb,,Abkürzung,32.147079,Rundfunk,Rundfunk,NN,noPl,1879.257202,lemma,6766.045410,erlaubt_und_verboten,1
3787,Kfz,Kfz,NN,Abkürzung,172.759323,Kurzwort,Kurzwort,NN,,1048.175781,lemma,3774.152588,erlaubt_und_verboten,2
3952,Kurzw.,Kurzw.,,Abkürzung,4.193744,Kurzwort,Kurzwort,NN,,1048.175781,lemma,3774.152588,erlaubt_und_verboten,1
1410,CSU,CSU,NN,Abkürzung noPl,100.792984,Kurzwort,Kurzwort,NN,,1048.175781,lemma,3019.322021,erlaubt_und_verboten,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1271,Brandmeister~Br.Mstr.,Br.Mstr.,,,0.000000,Brandmeister,Brandmeister,NN,,42.247726,merkmal,84.895454,erlaubt_und_verboten,2
1274,Brandmeister~BrM,BrM,,,0.000000,Brandmeister,Brandmeister,NN,,42.247726,merkmal,84.895454,erlaubt_und_verboten,2
1277,Brandmeister~BrMstr,BrMstr,,,0.000000,Brandmeister,Brandmeister,NN,,42.247726,merkmal,84.895454,erlaubt_und_verboten,2
7386,diminuendo~dim,dim,,,0.000000,diminuendo,diminuendo,ADV,,17.771036,merkmal,35.942074,erlaubt_und_verboten,1


In [585]:
# lang_tag_fehlt
mask1 = translate_abk.lang_tag == ''
mask2 = translate_abk.kurz_tag != ''
mask3 = translate_abk.lang_id  != ''
mask4 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask5 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
#grid(translate_abk,mask)
check_mask(translate_abk,mask,6)

msg = 'lang_tag_fehlt'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  5 rows
Move 5 rows from 5777
5 rows out of 2306


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
4019,LPG,LPG,NN,Abkürzung,376.507599,landwirtschaftliche,landwirtschaftliche,,,0.0,lemma,0.1440,lang_tag_fehlt,1
4854,PAK,PAK,NN,Abkürzung noSg,143.740311,polyzyklische,polyzyklische,,,0.0,lemma,0.1440,lang_tag_fehlt,1
1361,CD-ROM,CD-ROM,NN,Abkürzung,562.880005,scheibenförmiger,scheibenförmiger,,,0.0,lemma,0.1152,lang_tag_fehlt,1
2144,EAP,EAP,NN,Abkürzung,190.905014,Ämtern,Ämtern,,,0.0,lemma,0.1152,lang_tag_fehlt,1
6006,Supergau,Supergau,NN,Abkürzung,87.907776,anzunehmenden,anzunehmenden,,,0.0,lemma,0.1152,lang_tag_fehlt,1


In [586]:
# tag von lang nach kurz übertragen
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.lang_tag != ''
mask = mask1  &  mask2
check_mask(translate_abk,mask,5182)

translate_abk.loc[mask,'kurz_tag'] = translate_abk[mask].lang_tag
grid(translate_abk,mask)

check_mask:  5457 rows
5457 rows out of 5772


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
1091,Bd.,Bd.,NN,Abkürzung,20.965275,Band,Band,NN,manuell,41571.363281,lemma,299315.250000,1
6649,Ws.,Ws.,NN,Abkürzung,11.028746,Wasser,Wasser,NN,,56061.152344,lemma,201820.859375,1
3795,Kh.,Kh.,NN,Abkürzung,4.933620,Krankenhaus,Krankenhaus,NN,,52026.234375,lemma,187295.156250,1
3925,Krh.,Krh.,NN,Abkürzung,5.775660,Krankenhaus,Krankenhaus,NN,,52026.234375,lemma,187295.156250,1
3928,Krhs.,Krhs.,NN,Abkürzung,6.462754,Krankenhaus,Krankenhaus,NN,,52026.234375,lemma,187295.156250,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7494,erb.,erb.,ADJA,Abkürzung,6.305295,erbaut,erbaut,ADJA,Ptz2,0.449347,lemma,2.337650,1
7512,erschl.,erschl.,ADJA,Abkürzung,3.632099,erschlossen_1,erschlossen,ADJA,Ptz2,0.075022,lemma,0.990078,1
8223,latinis.,latinis.,ADJA,Abkürzung,2.740582,latinisiert_1,latinisiert,ADJA,Ptz2,0.062883,lemma,0.946380,1
9523,vj.,vj.,ADJA,Abkürzung,9.361846,verjährt_1,verjährt,ADJA,Ptz2,0.058751,lemma,0.931504,1


In [587]:
# Weiterer_Schrott
mask1 =  translate_abk.kurz_tag == ''
mask2 = ~translate_abk.lang_member.isin(['','Wortverbindung','Redewendung'])
mask3 = ~translate_abk.kurz.isin(geschütze_abkürzungen)
mask4 = ~translate_abk.kurz_member.str.contains('manuell')
mask = mask1  &  mask2  &  mask3  &  mask4  
#grid(translate_abk,mask)
check_mask(translate_abk,mask,2)

msg = 'Weiterer_Schrott'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  2 rows
Move 2 rows from 5772
2 rows out of 2308


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
76,AAA,AAA,,Abkürzung,168.779648,AFIS,AFIS,,Abkürzung,35.151310,lemma,25.452944,Weiterer_Schrott,1
1574,DPA,DPA,,Abkürzung,27.408615,DPMA,DPMA,,Abkürzung,6.408694,lemma,4.758260,Weiterer_Schrott,1


In [588]:
# KOMBI
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.lang_member.isin(['Wortverbindung','Redewendung'])
mask = mask1  &  mask2 
check_mask(translate_abk,mask,98)

translate_abk.loc[mask,'kurz_tag'] = 'KOMBI'
grid(translate_abk,mask)

check_mask:  102 rows
102 rows out of 5770


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
8999,s._A.,s. A.,KOMBI,Abkürzung,13.263565,seligen_Angedenkens,seligen Angedenkens,,Redewendung,139.364365,lemma,100.486343,1
8989,seligen_Angedenkens~s A,s A,KOMBI,,0.000000,seligen_Angedenkens,seligen Angedenkens,,Redewendung,139.364365,merkmal,55.825745,1
9017,seligen_Angedenkens~s.A.,s.A.,KOMBI,,0.000000,seligen_Angedenkens,seligen Angedenkens,,Redewendung,139.364365,merkmal,55.825745,1
9027,seligen_Angedenkens~sA,sA,KOMBI,,0.000000,seligen_Angedenkens,seligen Angedenkens,,Redewendung,139.364365,merkmal,55.825745,1
9065,seligen_Angedenkens~sel A,sel A,KOMBI,,0.000000,seligen_Angedenkens,seligen Angedenkens,,Redewendung,139.364365,merkmal,55.825745,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7267,per_Adresse~c/o,c/o,KOMBI,,0.000000,per_Adresse,per Adresse,,Wortverbindung,4.577855,merkmal,1.528913,2
9334,ut_supra~u s,u s,KOMBI,,0.000000,ut_supra,ut supra,,Wortverbindung,2.050718,merkmal,0.900287,1
9351,ut_supra~u. s.,u. s.,KOMBI,,0.000000,ut_supra,ut supra,,Wortverbindung,2.050718,merkmal,0.900287,1
9359,ut_supra~u.s.,u.s.,KOMBI,,0.000000,ut_supra,ut supra,,Wortverbindung,2.050718,merkmal,0.900287,1


In [589]:
# lang_member ist jetzt ausgeschöpft
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.lang_member != ''
mask = mask1  &  mask2 
check_mask(translate_abk,mask,0)

check_mask:  0 rows


In [590]:
# Großgeschrieben => NN
mask1 = translate_abk.kurz_tag == ''
mask2 = translate_abk.kurz.str[0].str.isupper()
mask3 = translate_abk.kurz.str[0].str.isalpha() 
mask4 = translate_abk.kurz.str.len() > 2
mask5 = translate_abk.kurz.str.len() >= 4 
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5 
check_mask(translate_abk, mask, 16)
translate_abk.loc[mask,'kurz_tag'] = 'NN'
grid(translate_abk,mask)

check_mask:  16 rows
16 rows out of 5770


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
1097,Bde.,Bde.,NN,Abkürzung,3.235405,Bände,Bände,,,0.0,lemma,0.1440,2
2795,GDPdU,GDPdU,NN,Abkürzung,27.685272,Grundsätze,Grundsätze,,,0.0,lemma,0.1440,1
3345,Hrn.,Hrn.,NN,Abkürzung,25.777845,Herrn,Herrn,,,0.0,lemma,0.1440,1
3597,Jhs.,Jhs.,NN,Abkürzung,5.906564,Jahrhunderts,Jahrhunderts,,,0.0,lemma,0.1440,1
4407,Mmes.,Mmes.,NN,Abkürzung,2.805653,Mesdames,Mesdames,,,0.0,lemma,0.1440,1
5485,RiStBV,RiStBV,NN,Abkürzung,13.247355,Richtlinien,Richtlinien,,,0.0,lemma,0.1440,1
6683,Yds.,Yds.,NN,Abkürzung,1.735439,Yards,Yards,,,0.0,lemma,0.1440,1
6784,ZÄPrO,ZÄPrO,NN,Abkürzung,26.821758,Zahnärzte,Zahnärzte,,,0.0,lemma,0.1440,1
1233,BodSchätzDB,BodSchätzDB,NN,Abkürzung,12.183882,Durchführungsbestimmungen,Durchführungsbestimmungen,,,0.0,lemma,0.1152,1
1855,E_163,E 163,NN,Abkürzung,1.642471,Anthocyane,Anthocyane,,,0.0,lemma,0.1152,1


In [591]:
# Großgeschrieben => NN
mask1  = translate_abk.kurz_tag == ''
mask2  = translate_abk.kurz.str[0].str.isupper()
mask3  = translate_abk.kurz.str[0].str.isalpha() 
mask4  = translate_abk.kurz.str[0] ==  translate_abk.lang.str[0]
mask = mask1  &  mask2  &  mask3  &  mask4 
check_mask(translate_abk, mask, 4)
translate_abk.loc[mask,'kurz_tag'] = 'NN'
grid(translate_abk,mask)

check_mask:  4 rows
4 rows out of 5770


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
5523,RÄ,RÄ,NN,Abkürzung,14.568176,Rechtsanwälte,Rechtsanwälte,,,0.0,lemma,0.144,1
5594,SJZ,SJZ,NN,Abkürzung,13.025398,Süddeutsche,Süddeutsche,,,0.0,lemma,0.144,1
6001,SuS,SuS,NN,Abkürzung,41.070152,Schülerinnen,Schülerinnen,,,0.0,lemma,0.144,1
6355,V4,V4,NN,Abkürzung,5.640099,Visegrad-Staaten,Visegrad-Staaten,,,0.0,lemma,0.144,1


In [592]:
# Aufzählungen => CARD
# nicht nur für kurz_tag == ''
mask1 = translate_abk.kurz.str[:-1].str.isnumeric()
mask2 = translate_abk.kurz.str.endswith('.')  
mask3 = translate_abk.kurz.str.len() <= 3 
mask = mask1  &  mask2  &  mask3
check_mask(translate_abk, mask, 11)
translate_abk.loc[mask,'kurz_tag'] = 'CARD'
grid(translate_abk,mask)

check_mask:  11 rows
11 rows out of 5770


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,konkurrenz
54,8.,8.,CARD,Abkürzung,34.177845,achter,achter,ADJA,,106.679688,lemma,384.766846,1
6,0.,0.,CARD,Abkürzung,29.465649,nullter,nullter,,,0.000000,lemma,0.144000,1
14,1.,1.,CARD,Abkürzung,32.793232,erster,erster,,,0.000000,lemma,0.144000,1
15,10.,10.,CARD,Abkürzung,33.962463,zehnter,zehnter,,,0.000000,lemma,0.144000,1
27,2.,2.,CARD,Abkürzung,55.384083,zweiter,zweiter,,,0.000000,lemma,0.144000,1
38,3.,3.,CARD,Abkürzung,33.316307,dritter,dritter,,,0.000000,lemma,0.144000,1
43,4.,4.,CARD,Abkürzung,34.424000,vierter,vierter,,,0.000000,lemma,0.144000,1
47,5.,5.,CARD,Abkürzung,35.716309,fünfter,fünfter,,,0.000000,lemma,0.144000,1
49,6.,6.,CARD,Abkürzung,34.670155,sechster,sechster,,,0.000000,lemma,0.144000,1
51,7.,7.,CARD,Abkürzung,53.208374,siebter,siebter,,,0.000000,lemma,0.144000,1


In [593]:
# Rest
mask  = translate_abk.kurz_tag == ''
check_mask(translate_abk, mask, 25)

grid(translate_abk,mask)
msg = 'Rest'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
grid( translate_abk_trash, msg )

check_mask:  26 rows
26 rows out of 5770
Move 26 rows from 5770
26 rows out of 2334


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
303,AWZ,AWZ,,Abkürzung,29.302071,ausschließliche,ausschließliche,,,0.0,lemma,0.1440,Rest,1
2509,FFW,FFW,,Abkürzung,19.173544,freiwillige,freiwillige,,,0.0,lemma,0.1440,Rest,1
2835,GOS,GOS,,Abkürzung,16.847281,gymnasiale,gymnasiale,,,0.0,lemma,0.1440,Rest,1
4912,PKV,PKV,,Abkürzung,21.444920,private,private,,,0.0,lemma,0.1440,Rest,1
7346,dass.,dass.,,Abkürzung,1.599041,dasselbe,dasselbe,,,0.0,lemma,0.1440,Rest,1
7354,deins,deins,,Abkürzung,10.623570,deines,deines,,,0.0,lemma,0.1440,Rest,1
7383,dies.,dies.,,Abkürzung,1.698215,dieselbe,dieselbe,,,0.0,lemma,0.1440,Rest,2
7435,eG,eG,,Abkürzung,22.990444,eingetragene,eingetragene,,,0.0,lemma,0.1440,Rest,1
8062,jmdm.,jmdm.,,Abkürzung,4.435301,jemandem,jemandem,,,0.0,lemma,0.1440,Rest,1
8063,jmdn.,jmdn.,,Abkürzung,4.435301,jemanden,jemanden,,,0.0,lemma,0.1440,Rest,1


In [594]:
# Mehr gibt es nicht
mask = translate_abk.kurz_tag == ''
check_mask(translate_abk, mask, 0)

#grid(translate_abk,mask)
#msg = 'Rest'
#translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
#grid( translate_abk_trash, msg )

check_mask:  0 rows


In [595]:
# Neue kurz_id vergeben
#mask = translate_abk.kurz_id == ''
#translate_abk.loc[mask,'kurz_id'] = translate_abk[mask].kurz.str.replace(' ','_') + '_A'
#check_mask(translate_abk,mask,2983)
#grid(translate_abk,mask)

In [596]:
# member Abkürzung nachtragen
mask = ~translate_abk.kurz_member.str.contains('Abkürzung')
translate_abk.loc[mask,'kurz_member'] = (translate_abk[mask].kurz_member + ' Abkürzung').str.strip()

#### rückübertragen

In [597]:
# Alle Kurzformen aus wiktionary_lemma rauslöschen, die nicht in translate_abk erwähnt werden
# und keinen echten tag haben
mask1 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask2 = ~wiktionary_lemma.member.str.contains('manuell')
mask3 = ~wiktionary_lemma.lemma_id.isin(translate_abk.kurz_id)
mask4 = ~wiktionary_lemma.lemma_id.isin(translate_abk.lang_id)
mask5 =  wiktionary_lemma.tag == 'TODO'
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
#grid(wiktionary_lemma,mask)
check_mask(wiktionary_lemma,mask,2015)

msg = 'translate_abk1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows( wiktionary_lemma, wiktionary_lemma_trash, mask, msg)

# anschauen
mask = wiktionary_lemma_trash.msg == msg
grid( wiktionary_lemma_trash[mask].sort_values('lemma_score', ascending=False).head(20) )

check_mask:  2078 rows
Move 2078 rows from 171220
20 rows


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg
70413,OP,66325,0,OP,op,True,1,TODO,TODO,FuYe8uWvQC,X_todo,X,Abkürzung,,294.044769,translate_abk1
46613,IKT,66042,0,IKT,ikt,True,1,TODO,TODO,kjCn5q7bqN,X_todo,X,Abkürzung_verwaist,,230.687836,translate_abk1
3504,AIDS,2598,0,AIDS,aids,True,1,TODO,TODO,FKRIzfD19u,X_todo,X,Abkürzung_verwaist,,185.285721,translate_abk1
101596,VOB,20157,0,VOB,vob,True,1,TODO,TODO,8Sf63vXpXN,X_todo,X,Abkürzung_verwaist,,169.586456,translate_abk1
3414,AAA,12177,0,AAA,aaa,True,1,TODO,TODO,ScDRJ4i3Di,X_todo,X,Abkürzung,,168.779648,translate_abk1
21686,DV,17392,0,DV,dv,True,1,TODO,TODO,nukAa6BdK6,X_todo,X,Abkürzung,,159.248001,translate_abk1
82756,SPS,65443,0,SPS,sps,True,1,TODO,TODO,3Kr7uASQpq,X_todo,X,Abkürzung_verwaist,,158.823608,translate_abk1
78452,RWA,548144,0,RWA,rwa,True,1,TODO,TODO,JvtGnQG3ua,X_todo,X,Abkürzung_verwaist,,153.982208,translate_abk1
11904,BO,34707,0,BO,bo,True,1,TODO,TODO,uJtBfB5IvL,X_todo,X,Abkürzung,,142.694153,translate_abk1
46638,IS,405627,0,IS,is,True,1,TODO,TODO,cxE9ZR1uLt,X_todo,X,Abkürzung,,140.964706,translate_abk1


In [598]:
# Einige Datensätze sind in translate_abk gelöscht, haben aber in wiktionary_lemma einen richtigen tag.
# Die bleiben drin.
mask1 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask2 = ~wiktionary_lemma.lemma_id.isin(translate_abk.kurz_id)
mask3 = ~wiktionary_lemma.lemma_id.isin(translate_abk.lang_id)
mask4 =  wiktionary_lemma.tag != 'TODO'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_lemma,mask,192)

# anschauen
grid( wiktionary_lemma, mask )

check_mask:  205 rows
205 rows out of 169142


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
4,5G,975075,0,5G,5g,True,1,NN,NN,JibWQRx7kV,NOM,N,Abkürzung_verwaist manuell ergänzt,,8.974190
5,9/11,697316,0,9/11,9/11,True,1,NN,NN,pVdoCJgGNp,NOM,N,Abkürzung manuell ergänzt,,55.178623
558,GmbH,3714,0,GmbH,gmbh,True,1,NN,NN,oJv2f2yoH1,NOM,N,manuell ergänzt Abkürzung_verwaist,,682.923218
588,HMM,975439,0,HMM,hmm,True,1,ITJ,ITJ,ZiHOc9k4pu,X_itj,X,Abkürzung_verwaist manuell ergänzt,,7.572477
1184,Opa_1,67456,1,Opa,opa,True,1,NN,NN,kTTezpuydR,NOM,N,Abkürzung_verwaist Person_pre manuell ergänzt,,8.245752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124589,dpa,102374,0,dpa,dpa,False,1,NN,NN,OzNJpbkjPu,NOM,N,Abkürzung_verwaist noPl,,95.411606
169761,ÖAMTC,274668,0,ÖAMTC,öamtc,True,1,NN,NN,R9jVdn0Hyy,NOM,N,Abkürzung_verwaist noPl,,93.568619
169762,ÖBB,274658,0,ÖBB,öbb,True,1,NN,NN,WCMGG6F78P,NOM,N,Abkürzung_verwaist noSg,,87.414108
169767,ÖPNV,14046,0,ÖPNV,öpnv,True,1,NN,NN,QeejouURdC,NOM,N,Abkürzung noPl,,65.180862


In [599]:
#a = pak.analyse_freqs(wiktionary_lemma,['tag','lemma'])
#gridt(a)

In [600]:
# kurz_tag => tag_new
wiktionary_lemma = pak.update_col( df_to =      wiktionary_lemma, 
                                   df_from =    translate_abk, 
                                   left_on =    'lemma_id',
                                   right_on =   'kurz_id',
                                   col =        'kurz_tag',
                                   col_rename = 'tag_new',
                                   func =       'first',
                                  )
wiktionary_lemma = pak.move_cols(wiktionary_lemma,'tag_new','lemma_len')

update_col: func first applied, but it was pointless!
update_col: 2244 cells written into new column


In [601]:
# prüfen: tag_new != tag
mask1 = wiktionary_lemma.tag != wiktionary_lemma.tag_new
mask2 = wiktionary_lemma.tag != 'TODO'
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask,0)
#gridt(wiktionary_lemma[mask].sort_values('lemma_score', ascending=False))

check_mask:  0 rows


In [602]:
# anschauen: tag_new != tag
mask1 = wiktionary_lemma.tag != wiktionary_lemma.tag_new
mask2 = wiktionary_lemma.tag == 'TODO'
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask,2152)

grid(wiktionary_lemma[mask].sort_values('lemma_score', ascending=False), pak.sample  )

check_mask:  2089 rows
11 rows out of 2089


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag_new,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
2698,&c.,240350,0,&c.,&c.,False,1,ADV,TODO,TODO,qlt4Ws57OB,X_todo,X,Abkürzung,,4.049606
6816,Alt_4,1538,4,Alt,alt,True,1,NN,TODO,TODO,2zgIqrAL8Q,X_todo,X,Abkürzung,,0.901200
25533,ETCS,127801,0,ETCS,etcs,True,1,NN,TODO,TODO,7bbXxDP8PB,X_todo,X,Abkürzung,,587.812317
41450,HI,164,0,HI,hi,True,1,NN,TODO,TODO,6zFf4QJfQH,X_todo,X,Abkürzung,,8.867126
77466,Psych.,13670,0,Psych.,psych.,True,1,NN,TODO,TODO,zzdhv7PEVr,X_todo,X,Abkürzung,,4.850154
128922,erf.,242872,0,erf.,erf.,False,1,VVFIN,TODO,TODO,NMwfXJjM19,X_todo,X,Abkürzung,,31.472126
131214,fm,1724,0,fm,fm,False,1,NN,TODO,TODO,v5mq3CPzDY,X_todo,X,Abkürzung,,24.073540
150267,prähist.,16113,0,prähist.,prähist.,False,1,ADJA,TODO,TODO,26ai7An0ft,X_todo,X,Abkürzung,,2.127967
152385,s._l.,102548,0,s. l.,s. l.,False,2,KOMBI,TODO,TODO,007bOWUBuU,X_todo,X,Abkürzung,,19.105988
164301,vrstl.,1114479,0,vrstl.,vrstl.,False,1,ADJA,TODO,TODO,I9ZwJZP1Al,X_todo,X,Abkürzung,,11.987206


In [603]:
# tag_new => tag
wiktionary_lemma.loc[mask,'tag'] = wiktionary_lemma[mask].tag_new
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'tag_new')

In [604]:
# pflege_tag_felder für die veränderten tags
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1

In [605]:
# lang_id => data_id_new
wiktionary_lemma = pak.update_col( df_to =      wiktionary_lemma, 
                                   df_from =    translate_abk, 
                                   left_on =    'lemma_id',
                                   right_on =   'kurz_id',
                                   col =        'lang_id',
                                   col_rename = 'data_id_new',
                                   func =       'first',
                                  )

update_col: func first applied, but it was pointless!
update_col: 2244 cells written into new column


In [606]:
# prüfen: data_id_new != data_id
mask1 = wiktionary_lemma.data_id != wiktionary_lemma.data_id_new
mask2 = wiktionary_lemma.data_id != ''
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask, 0)
#grid(wiktionary_lemma[mask].sort_values('lemma_score', ascending=False) )

check_mask:  0 rows


In [607]:
# anschauen: data_id_new 
mask1 = wiktionary_lemma.data_id != wiktionary_lemma.data_id_new
mask2 = wiktionary_lemma.data_id == ''
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask, 2492)
grid(wiktionary_lemma[mask].sort_values('lemma_score', ascending=False) )

check_mask:  2244 rows
2244 rows


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,data_id_new
91138,Stasi,396199,0,Stasi,stasi,True,1,NN,NN,9mD0nliMrv,NOM,N,Abkürzung noPl,,9214.994141,Geheimpolizei
19407,CO₂,119982,0,CO₂,co₂,True,1,NN,NN,GN3I5Ro2qO,NOM,N,Abkürzung noPl,,976.711426,chemisch
102897,WM,19318,0,WM,wm,True,1,NN,NN,DZ0F4GvXSb,NOM,N,Abkürzung,,967.495728,Weltmeisterschaft
45604,ICE,60577,0,ICE,ice,True,1,NN,NN,212XSE4kD0,NOM,N,Abkürzung,,874.731934,Intercity-Express
56064,Küfa,1112624,0,Küfa,küfa,True,1,NN,NN,gqEEWv5hAv,NOM,N,Abkürzung,,820.709839,öffentlich
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66209,NP_1,22234,1,NP,np,True,1,NN,NN,JMleexWkKy,NOM,N,Abkürzung,,1.163690,Problem
66189,NC_1,434053,1,NC,nc,True,1,NE,NE,ISXV3V1uf8,NAM,N,Abkürzung,,1.123401,North_Carolina
56188,L._A.,24118,0,L. A.,l. a.,True,2,NE,NE,8zCE8wgVKF,NAM,N,Abkürzung,,1.113570,Los_Angeles
60972,Man_2,116544,2,Man,man,True,1,NN,NN,BRwS3QSNCc,NOM,N,Abkürzung,,1.108692,Manual


In [608]:
# data_id_new => data_id
wiktionary_lemma.loc[mask,'data_id'] = wiktionary_lemma[mask].data_id_new
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'data_id_new')

In [609]:
# Obsolet
# Neue Datensätze für wiktionary_lemma
# (werden nicht angefügt)
mask_abk = wiktionary_lemma.member.str.contains('Abkürzung')
mask = ~translate_abk.kurz_id.isin(wiktionary_lemma[mask_abk].lemma_id)
check_mask(translate_abk,mask, 3031)

df = translate_abk[mask]

df = pak.rename_col(df, 'kurz_id',     'lemma_id')
df = pak.rename_col(df, 'kurz',        'lemma')
df = pak.rename_col(df, 'kurz_tag',    'tag')
df = pak.rename_col(df, 'kurz_member', 'member')
df = pak.rename_col(df, 'kurz_score',  'lemma_score')
df = pak.rename_col(df, 'lang_id',     'data_id')
df['page_id']   = -1
df['page_part'] = -1
df = pak.move_cols(df,['page_id','page_part'],'lemma_id')
df = pak.drop_cols(df,['konkurrenz','rank','todo','lang','lang_tag','lang_member','lang_score',])

df = ergänze_felder(df)
df = pflege_tag_felder(df, translate_tagZ)  
assert set(wiktionary_lemma.columns) ==  set(df.columns) 
grid(df)

check_mask:  3500 rows
check_mask:  11 rows
3500 rows


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,lemma_score,data_id
0,Band~Bd,-1,-1,Bd,bd,True,1,NN,NN,bikWZcL2QW,NOM,N,manuell Abkürzung,0.0,Band
1,Februar~Feb.,-1,-1,Feb.,feb.,True,1,NN,NN,Fg3ihv0HP1,NOM,N,manuell Abkürzung,0.0,Februar
2,Wasser~H2O,-1,-1,H2O,h2o,True,1,NN,NN,G1n9OQImdg,NOM,N,Abkürzung,0.0,Wasser
3,Januar~Jan,-1,-1,Jan,jan,True,1,NN,NN,hbDeAKHOZx,NOM,N,manuell Abkürzung,0.0,Januar
4,Krankenhaus~Krh,-1,-1,Krh,krh,True,1,NN,NN,mikAsHL6JD,NOM,N,Abkürzung,0.0,Krankenhaus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3495,per_Adresse~c/o,-1,-1,c/o,c/o,False,1,KOMBI,KOMBI,o7Ttp6APk4,X_kombi,X,Abkürzung,0.0,per_Adresse
3496,ut_supra~u s,-1,-1,u s,u s,False,2,KOMBI,KOMBI,RWFuuwz87O,X_kombi,X,Abkürzung,0.0,ut_supra
3497,ut_supra~u. s.,-1,-1,u. s.,u. s.,False,2,KOMBI,KOMBI,egluyxjYJE,X_kombi,X,Abkürzung,0.0,ut_supra
3498,ut_supra~u.s.,-1,-1,u.s.,u.s.,False,1,KOMBI,KOMBI,Bg1NX3Defp,X_kombi,X,Abkürzung,0.0,ut_supra


In [610]:
#wiktionary_lemma anfügen!
#wiktionary_lemma = pak.add_rows(wiktionary_lemma, df)

In [611]:
## lemma_id vorhanden und eindeutig?
#mask = wiktionary_lemma.lemma_id == ''
#check_mask(wiktionary_lemma,mask,0)
#
#assert wiktionary_lemma.lemma_id.is_unique

In [612]:
#mask = wiktionary_lemma.duplicated(subset='lemma_id', keep=False)
#grid(wiktionary_lemma[mask].sort_values('lemma_id'))

#### nochmal belöschen

In [613]:
# lang_id fehlt >> löschen (dürfte eigentlich nichts mehr dabei sein)
mask = translate_abk.lang_id == ''
check_mask(translate_abk, mask, 0)

msg = 'lang_id fehlt'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
#grid( translate_abk_trash, msg )

check_mask:  0 rows
No rows moved


In [614]:
# lang_id verwaist 
mask = ~translate_abk.lang_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_abk,mask,0,40)

msg = 'lang_id verwaist'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)
grid( translate_abk_trash, msg )

check_mask:  30 rows
Move 30 rows from 5744
30 rows out of 2364


,kurz_id,kurz,kurz_tag,kurz_member,kurz_score,lang_id,lang,lang_tag,lang_member,lang_score,todo,rank,msg,konkurrenz
6,0.,0.,CARD,Abkürzung,29.465649,nullter,nullter,,,0.0,lemma,0.1440,lang_id verwaist,1
14,1.,1.,CARD,Abkürzung,32.793232,erster,erster,,,0.0,lemma,0.1440,lang_id verwaist,1
15,10.,10.,CARD,Abkürzung,33.962463,zehnter,zehnter,,,0.0,lemma,0.1440,lang_id verwaist,1
27,2.,2.,CARD,Abkürzung,55.384083,zweiter,zweiter,,,0.0,lemma,0.1440,lang_id verwaist,1
38,3.,3.,CARD,Abkürzung,33.316307,dritter,dritter,,,0.0,lemma,0.1440,lang_id verwaist,1
43,4.,4.,CARD,Abkürzung,34.424000,vierter,vierter,,,0.0,lemma,0.1440,lang_id verwaist,1
47,5.,5.,CARD,Abkürzung,35.716309,fünfter,fünfter,,,0.0,lemma,0.1440,lang_id verwaist,1
49,6.,6.,CARD,Abkürzung,34.670155,sechster,sechster,,,0.0,lemma,0.1440,lang_id verwaist,1
51,7.,7.,CARD,Abkürzung,53.208374,siebter,siebter,,,0.0,lemma,0.1440,lang_id verwaist,1
56,9.,9.,CARD,Abkürzung,34.424000,neunter,neunter,,,0.0,lemma,0.1440,lang_id verwaist,1


In [615]:
# Bedeutungsloses ggf. löschen (dürfte eigentlich nichts mehr dabei sein)
maskA = translate_abk.kurz_id == translate_abk.lang_id
maskB = translate_abk.kurz    == translate_abk.lang
mask = maskA | maskB
check_mask(translate_abk,mask,0)

msg = 'bedeutungslos'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg)

# anschauen
#grid( translate_abk_trash, msg )

check_mask:  0 rows
No rows moved


In [616]:
# Keine neuen Datensätze für translate_lex
# mask1 = translate_abk.kurz_id.str.endswith('_A')
# mask2 = translate_abk.lang_id.isin(translate_lex.data_id)
# mask = mask1  &  mask2  
# check_mask(translate_abk,mask,0)

### wiktionary_merkmal belöschen
* alle abk-Merkmale sind schon gelöscht

In [617]:
# alle kurzformen
mask_abk =  wiktionary_lemma.member.str.contains('Abkürzung')
kurzformen = wiktionary_lemma[mask_abk].lemma_id

In [618]:
# Alle def-Merkmale aller Kurzformen löschen
mask1 = wiktionary_merkmal.merkmal == 'def'
mask2 = wiktionary_merkmal.lemma_id.isin(kurzformen)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,4867)

msg = 'def einer Abkürzung'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)

mask = wiktionary_merkmal_trash.msg == msg
grid( wiktionary_merkmal_trash, mask )

check_mask:  4313 rows
Move 4313 rows from 4850254
4313 rows out of 6344


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg
3437,HI,def,Herzinfarkt,Herzinfarkt,,TODO,1,Med,2.955709,False,def einer Abkürzung
3438,HI,def,Herzinsuffizienz,Herzinsuffizienz,,TODO,3,Med,2.955709,False,def einer Abkürzung
17957,Pkw,def,Personenkraftwagen,Personenkraftwagen,,NN,1,,90.900574,False,def einer Abkürzung
17985,PKW,def,Personenkraftwagen,Personenkraftwagen,,NN,1,,23.277538,False,def einer Abkürzung
18914,BP,def,Bundespost,Bundespost,,TODO,2,,9.253415,False,def einer Abkürzung
...,...,...,...,...,...,...,...,...,...,...,...
1794149,Küfa,def,Kochen,Kochen,,NN,1,,91.189987,False,def einer Abkürzung
1794855,EnSikuMaV,def,Kurzfristenergieversorgungssicherungsmaßnahmenverordnung,Kurzfristenergieversorgungssicherungsmaßnahmenverordnung,,TODO,1,,8.665539,False,def einer Abkürzung
1794859,EnSimiMaV,def,Mittelfristenergieversorgungssicherungsmaßnahmenverordnung,Mittelfristenergieversorgungssicherungsmaßnahmenverordnung,,TODO,1,,8.706564,False,def einer Abkürzung
1796003,BRAK,def,Bundesrechtsanwaltskammer,Bundesrechtsanwaltskammer,,TODO,1,,3.028793,False,def einer Abkürzung


### Abkürzungen überprüfen

In [619]:
# wiktionary_lemma: Viele Abkürzungen sind in translate_abk
mask1 = wiktionary_lemma.member.str.contains('Abkürzung')
mask2 = wiktionary_lemma.lemma_id.isin(translate_abk.kurz_id)
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask,2492)
#wiktionary_lemma[mask]

check_mask:  2214 rows


In [620]:
# wiktionary_lemma: Einige Abkürzungen sind nicht in translate_abk
# (weil es keine Auflösung gibt)
mask1 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask2 = ~wiktionary_lemma.lemma_id.isin(translate_abk.kurz_id)
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask,194)
wiktionary_lemma[mask].sort_values('lemma_score',ascending=False).head(12)

check_mask:  238 rows


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
66198,NGO,17771,0,NGO,ngo,True,1,NN,NN,yEWuwnb1ex,NOM,N,Abkürzung,,2046.884277
21172,DDR,3264,0,DDR,ddr,True,1,NE,NE,KT1XXtOB9k,NAM,N,Geo Abkürzung,,1806.537842
1551,USA,5687,0,USA,usa,True,1,NE,NE,wrAXX06aI5,NAM,N,Geo Geo_land noSg Abkürzung,,1161.254395
81253,SMS,1818,0,SMS,sms,True,1,NN,NN,vltqWIufRi,NOM,N,Abkürzung,,870.798462
558,GmbH,3714,0,GmbH,gmbh,True,1,NN,NN,tmxPM2D6DT,NOM,N,manuell ergänzt Abkürzung_verwaist,,682.923218
3472,ASCII,23980,0,ASCII,ascii,True,1,NN,NN,XlRQaviYkP,NOM,N,Abkürzung_verwaist noPl,,619.340088
3438,AG,3266,0,AG,ag,True,1,NN,NN,N6ANjCYBcF,NOM,N,Abkürzung,,615.005676
19361,CD-ROM,2693,0,CD-ROM,cd-rom,True,1,NN,NN,ZPF4x8ltdM,NOM,N,Abkürzung,,562.880005
98512,U-Bahnhof,47752,0,U-Bahnhof,u-bahnhof,True,1,NN,NN,k6zDwEQXFu,NOM,N,Abkürzung,,491.036011
99945,VBIED,452712,0,VBIED,vbied,True,1,NN,NN,bIPtvjYLin,NOM,N,Abkürzung_verwaist,,427.921753


In [621]:
# translate_abk: Alle lang_id sind in wiktionary_lemma
mask = ~translate_abk.lang_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_abk,mask,0)
#translate_abk[mask] #.sort_values('score',ascending=False).head(12)

check_mask:  0 rows


In [622]:
# translate_abk: Viele kurz_id sind in wiktionary_lemma
mask =  translate_abk.kurz_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_abk,mask,2732)
#translate_abk[mask] #.sort_values('score',ascending=False).head(12)

check_mask:  2214 rows


In [623]:
# ... aber viele auch nicht
mask = ~translate_abk.kurz_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_abk,mask,3031)
#translate_abk[mask] #.sort_values('score',ascending=False).head(12)

check_mask:  3500 rows


## wiktionary_lemma ==========

### pflegen

In [624]:
# manuelle_kürzel erstellen: Welche Sorten Sonder-IDs gibt es?
# alt: ('_V', '_M', '_N', '_Z', '_O', '_NV')
# ('_V', '_A', '_M', '_N', '_MADJ', '_Z', '_T')

wiktionary_lemma['kürzel'] = wiktionary_lemma.lemma_id.str.split('_').str[-1]

mask = wiktionary_lemma.lemma_id.str.contains('_')
a = pak.analyse_freqs(wiktionary_lemma[mask], ['kürzel','lemma_id'])

mask1 = a.kürzel_count > 30
mask2 = a.kürzel.str.isalpha() 
mask3 = a.kürzel.str.len() < 5 
mask  = mask1  &  mask2  &  mask3
manuelle_kürzel = tuple(['_'+kürzel for kürzel in list(a[mask].kürzel)])  # um manuelle lemma_id erkennen zu können 
print(manuelle_kürzel)
assert len(manuelle_kürzel) == 6
grid(a,mask)

wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'kürzel')

('_V', '_M', '_N', '_MADJ', '_Z', '_T')
6 rows out of 2246


### tag

In [625]:
# sichten
mask = wiktionary_lemma.tag == 'TODO'
pak.analyse_freqs( wiktionary_lemma[mask], ['member','lemma_id'], splits=[' ', None] )

,member,member_count,member_percent,lemma_id,lemma_id_count
0,Redewendung,268,64.4,"[A-a_machen, Aa_machen, Abbitte_leisten, Abbitte_tun, Achtung_gebietend, Anstalten_machen, Anstalten_treffen, Asbach_uralt, Atem_schöpfen, Aufsehen_erregen, Ausschau_halten, Bacchus_huldigen, Bauklötze_staunen, Bauklötzer_staunen, Bedenken_tragen, Bindfäden_regnen, Blut_lecken, Bände_sprechen, Chomez_batteln, Dampf_ablassen]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,Wortverbindung,146,35.1,"[Anno_Domini, Artes_liberales, Besorgnis_erregend, Druck_ausüben, Furcht_erregend, I_bims, Klein_Lieschen, Seine_Eminenz, Seine_Heiligkeit, Seine_Majestät, Wäller_Platt, a_la, a_lallemande, a_lalsacienne, a_lespagnole, a_secco, a_tempo, ab_irato, ad_libitum, ad_spectatores]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,,2,0.5,"[Tempi_passati, variatio_delectat]","[1, 1]"


In [626]:
# NE-Datensätze, die mit hoher Wahrscheinlichkeit was anderes als einen Namen bezeichnen
geschützte_namen = ['Martin','Baum','Junge','Klein','Keller','Mann','Peter','Schwarz','Simon',
                    'Thomas','Weiß','Winter','Sommer','Wolf','Stein','Henry','Frank','Müller',
                    'Meier','Huber','Schneider','Bauer','Conrad','Kramer','Grimm',
                    'Heinrich','Wagner','Pfeifer','Zimmer','Bach','Krämer','Fischer','Arnold',
                    'Lorenz','Fiedler','Weber','Walther','Busch','Schäfer','Michel','Ritter',
                    'Beck',
                   ]
mask1 =  wiktionary_lemma.member.str.contains('Person_nachname')
mask2 =  wiktionary_lemma.tag == 'NE'
mask3 =  wiktionary_lemma.page_part > 0
mask4 = ~wiktionary_lemma.lemma.isin(geschützte_namen)
mask5 = ~wiktionary_lemma.lemma.str.endswith(('mann','haus','meier','lein','berg','bruch','bach',))
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5

spalten = ['lemma_id','lemma','tag','member','lemma_score']
a = wiktionary_lemma[mask].sort_values('lemma_score').head(200).sort_values('lemma_score',ascending=False)[spalten]
gridt(a)

200 rows


In [627]:
# löschen
mask = wiktionary_lemma.lemma_id.isin(a.lemma_id)
check_mask( wiktionary_lemma, mask, 200 )

msg = 'NE1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(wiktionary_lemma, wiktionary_lemma_trash, mask, msg)
grid(wiktionary_lemma_trash,msg)

check_mask:  200 rows
Move 200 rows from 169142
200 rows out of 2278


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg
3559,Aalst_1,35489,1,Aalst,aalst,True,1,NE,NE,M3X46jj8dW,NAM,N,Person_nachname,,4.795716,NE1
3573,Aar_2,2510,2,Aar,aar,True,1,NE,NE,4FXmtiAk0B,NAM,N,Person_nachname,,3.321496,NE1
3629,Abakus_1,704,1,Abakus,abakus,True,1,NE,NE,L81y4NkDuj,NAM,N,Person_nachname,,2.555865,NE1
3639,Abart_1,2937,1,Abart,abart,True,1,NE,NE,lsO6Lgbzvu,NAM,N,Person_nachname,,2.852306,NE1
3649,Abate_2,14859,2,Abate,abate,True,1,NE,NE,6tgNSjUO8T,NAM,N,Person_nachname,,0.654998,NE1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108270,Zeuge_1,25035,1,Zeuge,zeuge,True,1,NE,NE,EYAZ4VE20G,NAM,N,Person_nachname,,0.941703,NE1
108581,Zinn_1,10873,1,Zinn,zinn,True,1,NE,NE,6UiYedi51x,NAM,N,Person_nachname,,0.208227,NE1
108791,Zoll_2,20685,2,Zoll,zoll,True,1,NE,NE,PA1uGNLV3d,NAM,N,Person_nachname,,0.085637,NE1
109652,Zwölfer_1,241161,1,Zwölfer,zwölfer,True,1,NE,NE,HtFAmGeipM,NAM,N,Person_nachname,,3.994555,NE1


In [628]:
# Prüfen: Alle Person_nachname sind NE
mask1 = wiktionary_lemma.member.str.contains('Person_nachname')
mask2 = wiktionary_lemma.tag == 'NE'
mask = mask1  &  ~mask2
check_mask(wiktionary_lemma,mask,0)

check_mask:  0 rows


In [629]:
# anschauen
mask = wiktionary_lemma.member.str.contains('Person_nachname')
a = pak.analyse_freqs( wiktionary_lemma[mask].sort_values('lemma_score',ascending=False), ['member','lemma'], splits=[' ',None] )
grid(a)

4 rows


,member,member_count,member_percent,lemma,lemma_count
0,Person_nachname,1405,96.1,"[Raab, Groß, Eder, Steinbach, Gottschalk, Rothe, Ansorge, Auer, Echtermeyer, Bichler, Nicolaus, Herrmann, Baak, Jauch, Blüml, Baumert, Schäuble, Stoiber, Crecelius, Rommel]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
1,Person_vorname,44,3.0,"[Jordan, Timm, Paul, Erdmann, Götz, Oswald, Wichmann, Reinhardt, Wilhelm, Gottschalk, Barthel, Hein, Hermann, Hinz, Klaus, Ernst, Norman, Wulf, Sander, Urban]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,Geo,12,0.8,"[Jordan, Raab, Neuhaus, Bode, Eder, Steinbach, Geyer, Rothe, Metz, Offenbach, Lauer, Hagen]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,Geo_stadt,1,0.1,Offenbach,1


In [630]:
#grid(pak.analyse_cols(wiktionary_lemma))

In [631]:
# KOMBI: Datensätze umtaggen, die eine Wortverbindung oder eine Redewendung beinhalten
mask1  = wiktionary_lemma.tag == 'TODO'
mask2a = wiktionary_lemma.member.str.contains('Redewendung')
mask2b = wiktionary_lemma.member.str.contains('Wortverbindung')
mask = mask1  &  (mask2a | mask2b)
check_mask( wiktionary_lemma, mask, 412 )

change_tag( wiktionary_lemma, mask, 'KOMBI', manuelle_kürzel )

check_mask:  412 rows
412 Datensätze


In [632]:
# Rest sichten
mask = wiktionary_lemma.tag == 'TODO'
pak.analyse_freqs( wiktionary_lemma[mask], ['member', 'lemma_id'], splits=[' ', None] )

,member,member_count,member_percent,lemma_id,lemma_id_count
0,,2,100.0,"[Tempi_passati, variatio_delectat]","[1, 1]"


In [633]:
#mask = wiktionary_lemma.member.str.contains('Abkürzung')
#pak.analyse_freqs( wiktionary_lemma[mask], ['member', 'tag'],  )

In [634]:
# Rest löschen
mask = wiktionary_lemma.tag == 'TODO'
check_mask( wiktionary_lemma, mask, 0, 10 )

msg = 'TODO-Rest'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(wiktionary_lemma, wiktionary_lemma_trash, mask, msg)
grid(wiktionary_lemma_trash,msg)

check_mask:  2 rows
Move 2 rows from 168942
2 rows out of 2280


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg
95241,Tempi_passati,484152,0,Tempi passati,tempi passati,True,2,TODO,TODO,M7YuJdBCVs,X_todo,X,,,4.542139,TODO-Rest
157772,variatio_delectat,747185,0,variatio delectat,variatio delectat,False,2,TODO,TODO,Q4dCFOaUid,X_todo,X,,,19.675831,TODO-Rest


In [635]:
# ADJD >> ADJA
wiktionary_lemma['tag'] = wiktionary_lemma.tag.str.replace('ADJD','ADJA')

In [636]:
# Einzelkorrekturen        
einzelkorrekturen = { 'sämtlich'       : 'PIAT ADJA',  
                      'besitzergreifen': 'VVFIN',    
                    }
for i,t in einzelkorrekturen.items():
    mask = (wiktionary_lemma.lemma_id == i)
    wiktionary_lemma.loc[mask,'tag'] = t

In [637]:
# Hilvsverben auf VAFIN
hilfsverben = [ 'haben', 'sein', 'werden', 'tun' ]
mask1 = wiktionary_lemma.tagZZ == 'V'
mask2 = wiktionary_lemma.lemma.isin(hilfsverben)
mask3 = wiktionary_lemma.tag != 'VAFIN'
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_lemma, mask, 1)

#grid(wiktionary_lemma,mask)
change_tag( wiktionary_lemma, mask, 'VAFIN', manuelle_kürzel )

check_mask:  1 rows
1 Datensätze


In [638]:
# Modalverben auf VMFIN
modalverben = [ 'dürfen', 'können', 'mögen', 'müssen', 'sollen', 'wollen', 'brauchen' ]
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 =  wiktionary_lemma.lemma.isin(modalverben)
mask3 =  wiktionary_lemma.tag != 'VMFIN'
mask4 =  wiktionary_lemma.page_part == 0
mask  = mask1  &  mask2  &  mask3  &  mask4
check_mask( wiktionary_lemma, mask, 7)

#grid(wiktionary_lemma,mask)
change_tag( wiktionary_lemma, mask, 'VMFIN', manuelle_kürzel )

check_mask:  7 rows
7 Datensätze


In [639]:
# Obsolet: Alle anderen auf VVFIN 
#TODO vgl. und vergleiche müssten hier verschwinden
sonderverben = [ 'dürfen', 'können', 'mögen', 'müssen', 'sollen', 'wollen', 'brauchen', 'haben', 'sein', 'werden', 'tun' ]
mask1 =  wiktionary_lemma.tagZZ == 'V'
mask2 = ~wiktionary_lemma.lemma.isin(sonderverben)
mask3 =  wiktionary_lemma.tag != 'VVFIN'
mask  = mask1  &  mask2  &  mask3  
check_mask( wiktionary_lemma, mask, 0)

#change_tag( wiktionary_lemma, mask, 'VVFIN', manuelle_kürzel, manuelle_quellen_schützen=False )
#grid(wiktionary_lemma,mask)

check_mask:  0 rows


In [640]:
# Verben analysieren
mask = wiktionary_lemma.tagZZ == 'V'
pak.analyse_freqs(wiktionary_lemma[mask],['tag','lemma_id'])

,tag,tag_count,tag_percent,lemma_id,lemma_id_count
0,VVFIN,13853,99.9,"[Schlange_stehen, abbeißen, abberufen, abfahren, abküssen, abpassen, abreißen, abschrecken, abstoßen, abtörnen, abwerten, achten, adstringieren, affigieren, agglutinieren, alarmieren, alleinstehen, alternieren, amtieren, anbehalten]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,VMFIN,7,0.1,"[brauchen, dürfen, können, mögen, müssen, sollen, wollen]","[1, 1, 1, 1, 1, 1, 1]"
2,VAFIN,5,0.0,"[haben, sein, tun, werden, tun_1]","[1, 1, 1, 1, 1]"


### Affix und Suffix

In [641]:
# AFIX verschieben
mask1  = wiktionary_lemma.lemma.str.endswith('-')
mask2 = ~wiktionary_lemma.tag.isin(['CARD','$('])
mask = mask1  &  mask2
check_mask( wiktionary_lemma, mask, 554)

msg = 'AFIX'
wiktionary_lemma, wiktionary_lemma_fix = pak.move_rows( wiktionary_lemma, mask, msg ) 
grid(wiktionary_lemma_fix,msg)

check_mask:  548 rows
Move 548 rows from 168940
548 rows


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg
6449,Allerwelts-,749499,0,Allerwelts-,allerwelts-,True,1,AFIX,AFIX,cxHAcBm0mJ,FIX,Z,Präfixoid Präfix,,24.288923,89
6526,Allround-,668309,0,Allround-,allround-,True,1,AFIX,AFIX,7ONzK5S9Ja,FIX,Z,Gebundenes_Lexem,,18.702240,89
10021,Atto-,86413,0,Atto-,atto-,True,1,AFIX,AFIX,qDRTJ9m2zw,FIX,Z,Gebundenes_Lexem Einheit,,11.136205,89
15534,Bilderbuch-,737083,0,Bilderbuch-,bilderbuch-,True,1,AFIX,AFIX,e9jHlBnsBG,FIX,Z,Präfixoid Präfix,,31.893124,89
17664,Breitband-,684488,0,Breitband-,breitband-,True,1,AFIX,AFIX,tkRig7fsFU,FIX,Z,Gebundenes_Lexem,,16.201553,89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168187,äqui-,71842,0,äqui-,äqui-,False,1,AFIX,AFIX,IN18uOGtyN,FIX,Z,Gebundenes_Lexem,,49.882153,89
168257,öko-,714607,0,öko-,öko-,False,1,AFIX,AFIX,2UPf2mFptH,FIX,Z,Präfix,,35.807617,89
168279,öno-,129534,0,öno-,öno-,False,1,AFIX,AFIX,EhBeLWhTux,FIX,Z,Gebundenes_Lexem,,21.613848,89
168316,über-,62111,0,über-,über-,False,1,AFIX,AFIX,TlRAeFyCW7,FIX,Z,Präfix,,148.673889,89


In [642]:
# ZFIX verschieben
mask1 = wiktionary_lemma.lemma.str.startswith('-')
mask2 = ~wiktionary_lemma.tag.isin(['CARD','$('])
mask = mask1  &  mask2
check_mask( wiktionary_lemma, mask, 389)

msg = 'ZFIX'
wiktionary_lemma, wiktionary_lemma_fix = pak.move_rows( wiktionary_lemma, wiktionary_lemma_fix, mask, msg ) 
grid(wiktionary_lemma_fix,msg)

check_mask:  384 rows
Move 384 rows from 168392
384 rows out of 932


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg
2707,-ab,48801,0,-ab,-ab,False,1,ZFIX,ZFIX,wkQjzA4mUW,FIX,Z,Suffix,,108.709518,ZFIX
2708,-abel,243516,0,-abel,-abel,False,1,ZFIX,ZFIX,4pI5yJDFtQ,FIX,Z,Adjektiv Adjektiv Suffix,,225.476135,ZFIX
2709,-aceae,232497,0,-aceae,-aceae,False,1,ZFIX,ZFIX,TdgYaB8j3X,FIX,Z,Suffix,,14.212000,ZFIX
2710,-ach,217804,0,-ach,-ach,False,1,ZFIX NE,ZFIX,NE,FIX,Z,Geo_ort Suffix,,45.883427,ZFIX
2711,-ade,714445,0,-ade,-ade,False,1,ZFIX,ZFIX,ooxdhfS1Nn,FIX,Z,Suffix,,104.712921,ZFIX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3086,-är,713795,0,-är,-är,False,1,ZFIX,ZFIX,ZbPlnMesw7,FIX,Z,Suffix,,387.161377,ZFIX
3087,-ärmelig,232479,0,-ärmelig,-ärmelig,False,1,ZFIX,ZFIX,4IvXDDnn91,FIX,Z,Gebundenes_Lexem,,17.031048,ZFIX
3088,-ästhesie,717017,0,-ästhesie,-ästhesie,False,1,ZFIX,ZFIX,rA3NOU37Ms,FIX,Z,Gebundenes_Lexem,,76.092667,ZFIX
3089,-äugig,690975,0,-äugig,-äugig,False,1,ZFIX,ZFIX,k1BjkFPbOq,FIX,Z,Suffix,,156.270752,ZFIX


In [643]:
# Sonstige Fixe
maskA = wiktionary_lemma.lemma.str.startswith('-')
maskB = wiktionary_lemma.lemma.str.endswith('-')
maskC = wiktionary_lemma.tag.str.contains('AFIX')
maskD = wiktionary_lemma.tag.str.contains('ZFIX')
maskE = wiktionary_lemma.member.str.contains('Affix')
maskF = wiktionary_lemma.member.str.contains('Suffix')
mask2 = ~wiktionary_lemma.tag.isin(['CARD','$('])
mask = (maskA | maskB | maskC | maskD | maskE | maskF)  &  mask2
check_mask( wiktionary_lemma, mask, 2)

msg = 'DIVFIX'
wiktionary_lemma, wiktionary_lemma_fix = pak.move_rows( wiktionary_lemma, wiktionary_lemma_fix, mask, msg ) 
grid(wiktionary_lemma_fix,msg)

check_mask:  1 rows
Move 1 rows from 168008
1 rows out of 933


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score,msg
121468,dar,50798,0,dar,dar,False,1,AFIX,AFIX,fC6QSwEjkp,FIX,Z,Präfix,,1.5212,DIVFIX


### member

In [644]:
# member Aufzählung bei CARD
mask1 = wiktionary_lemma.lemma.str[:-1].str.isnumeric()
mask2 = wiktionary_lemma.lemma.str.endswith('.')  
mask3 = wiktionary_lemma.lemma.str.len() <= 3 
mask4 = wiktionary_lemma.tag == 'CARD'
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_lemma, mask, 11)

wiktionary_lemma.loc[mask,'member'] += ' Aufzählung'
grid(wiktionary_lemma,mask)

check_mask:  11 rows
11 rows out of 168007


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
3098,0.,92317,0,0.,0.,True,1,CARD,CARD,gQ6Qc0jrf2,X_zahl,X,Abkürzung Aufzählung,nullter,29.465649
3128,1.,53737,0,1.,1.,True,1,CARD,CARD,BlO9C4oEB0,X_zahl,X,Abkürzung Aufzählung,erster,32.793232
3152,10.,53904,0,10.,10.,True,1,CARD,CARD,VwiG3OgZKN,X_zahl,X,Abkürzung Aufzählung,zehnter,33.962463
3228,2.,53711,0,2.,2.,True,1,CARD,CARD,dYHut6q89g,X_zahl,X,Abkürzung Aufzählung,zweiter,55.384083
3272,3.,53739,0,3.,3.,True,1,CARD,CARD,YIzybci5zZ,X_zahl,X,Abkürzung Aufzählung,dritter,33.316307
3292,4.,53891,0,4.,4.,True,1,CARD,CARD,CBIAgaYPg7,X_zahl,X,Abkürzung Aufzählung,vierter,34.424000
3315,5.,53892,0,5.,5.,True,1,CARD,CARD,1eCudHQMHf,X_zahl,X,Abkürzung Aufzählung,fünfter,35.716309
3339,6.,53894,0,6.,6.,True,1,CARD,CARD,sy0mCLqZIS,X_zahl,X,Abkürzung Aufzählung,sechster,34.670155
3352,7.,53897,0,7.,7.,True,1,CARD,CARD,hbcY0tvVSn,X_zahl,X,Abkürzung Aufzählung,siebter,53.208374
3365,8.,53901,0,8.,8.,True,1,CARD,CARD,OiDm0zw9Zr,X_zahl,X,Abkürzung Aufzählung,achter,34.177845


In [645]:
# member noSg und noPl nur bei Nomen
mask1a = wiktionary_lemma.member.str.contains('noPl')
mask1b = wiktionary_lemma.member.str.contains('noSg')
mask2  = wiktionary_lemma.tagZZ != 'N'
mask  = (mask1a | mask1b)  &  mask2
check_mask( wiktionary_lemma, mask, 0, 17 )

wiktionary_lemma.loc[mask,'member'] =  pak.remove_words( wiktionary_lemma[mask].member, ['noPl', 'noSg'] )
#grid(wiktionary_lemma,mask)

check_mask:  16 rows


### Allgemein

In [646]:
# Redundante Spalten
wiktionary_lemma = pflege_tag_felder(wiktionary_lemma, translate_tagZ)  # tagZ, tagZZ, tag_0, tag_1, tagZ_0, tagZ_1
wiktionary_lemma = ergänze_felder(wiktionary_lemma)                     # lemma_lower, isupper, lemma_len

# Fehler?
mask = (wiktionary_lemma.tagZZ.str.len() > 1)
check_mask(wiktionary_lemma,mask,0) # ein paar können wir tolerieren

check_mask:  291 rows
check_mask:  0 rows


In [647]:
# Keine Dups
mask = wiktionary_lemma.duplicated(subset='lemma_id', keep=False)
check_mask( wiktionary_lemma, mask, 0)

check_mask:  0 rows


In [648]:
# not any_nan
assert not pak.any_nan(wiktionary_lemma)
#pak.analyse_cols(wiktionary_lemma)

In [649]:
# suche_debug wiktionary_lemma
search_str(wiktionary_lemma, suche_debug)

,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
453,Euro,11127,0,Euro,euro,True,1,NN,NN,T14XKG7S29,NOM,N,Einheit manuell,,1403.598999
2607,zu_viel,89575,0,zu viel,zu viel,False,2,ADV,ADV,qIy6RLdzhn,ADV,A,Wortverbindung manuell,,51.254711
28450,Euro_1,11127,1,Euro,euro,True,1,NN,NN,4NiZv4vtOd,NOM,N,,,36.994484


In [650]:
a = pak.analyse_freqs(wiktionary_lemma,['tag','member'], splits=[None,' '])
grid(a)

61 rows


,tag,tag_count,tag_percent,member,member_count
0,NN,96705,57.6,"[, noPl, adjNN, Abkürzung, Wortverbindung, noSg, manuell, Spezies, Name, lexAlt, Zahlwort, Abkürzung_verwaist, Einheit, ergänzt, Person_pre, Redewendung, Zahlklassifikator]","[80032, 12078, 3179, 1457, 1135, 1081, 673, 360, 305, 297, 217, 128, 80, 79, 31, 11, 5]"
1,ADJA,41839,24.9,"[Ptz1, Ptz2, noSteig, , Abkürzung, manuell, noFlex, Wortverbindung, ergänzt, lexAlt, Abkürzung_verwaist]","[13796, 13719, 7980, 5695, 645, 520, 380, 77, 48, 48, 1]"
2,VVFIN,13853,8.2,"[, manuell, lexAlt, Wortverbindung, ergänzt, Abkürzung, Redewendung]","[13308, 495, 84, 42, 11, 10, 2]"
3,NE,12118,7.2,"[Geo, Person_vorname, , Person_nachname, Wortverbindung, adjNN, Geo_stadt, noSg, Geo_land, Abkürzung, Geo_straße, Spezies, manuell, lexAlt, Abkürzung_verwaist, Geo_ort, ergänzt, Person_pre]","[6169, 5274, 1507, 1405, 613, 326, 293, 212, 103, 81, 46, 11, 6, 3, 3, 2, 2, 1]"
4,ADV,1562,0.9,"[, Wortverbindung, Lokaladverb, Abkürzung, manuell, Pronominaladverb, ergänzt, Gradpartikel, Temporaladverb, Modalpartikel, Konjunktionaladverb, Fokuspartikel, Modaladverb, Redewendung, Interrogativadverb, lexAlt]","[1272, 71, 57, 55, 30, 29, 18, 16, 14, 7, 7, 5, 5, 4, 3, 2]"
...,...,...,...,...,...
56,APPR DIV,1,0.0,"[Kontraktion, manuell, ergänzt]","[1, 1, 1]"
57,PRELS PDS,1,0.0,"[manuell, ergänzt]","[1, 1]"
58,PIAT ADJA,1,0.0,,1
59,PRELS PWAT,1,0.0,manuell,1


## wiktionary_merkmal ==========

### pflegen

In [651]:
# merkmal_order: Sichert die Reihenfolge der gleichnamigen Merkmale, z.B. mehrere Defs
wiktionary_merkmal = wiktionary_merkmal.reset_index()
wiktionary_merkmal = pak.rank( wiktionary_merkmal, 
                               cols_group = ['lemma_id','merkmal'], 
                               col_score = 'index',
                               find = 'min',
                               col_target = 'merkmal_order')
wiktionary_merkmal = pak.rename_col(wiktionary_merkmal,'index','i')
wiktionary_merkmal = pak.force_index(wiktionary_merkmal,'i')
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'merkmal_order','lemma_id')

rank: 0 rows less, now 4845941 rows


In [652]:
wiktionary_merkmal.head(1)

,lemma_id,merkmal_order,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
i,,,,,,,,,,,
0,Hallo,1,syn,Gejohle,Gejohle,,NN,1,,34.813919,False


### Schrott löschen 1
* ohne die Felder lemma und lemma_tag

In [653]:
# zu viele Sonderzeichen
mask1 =  wiktionary_merkmal.data.str.count('[A-ZÖÄÜa-zöäüß0-9]') < wiktionary_merkmal.data.str.count('[^A-ZÖÄÜa-zöäüß0-9]')-1
mask2 =  wiktionary_merkmal.data.str.len() > 2
mask3 = ~wiktionary_merkmal.merkmal.isin(['ipa','rhymes'])
mask4 = ~wiktionary_merkmal.data.isin(['u. s. w.','V. I. P.'])
mask  = mask1  & mask2  &  mask3  &  mask4
check_mask( wiktionary_merkmal, mask, 17)
#grid(wiktionary_merkmal, mask)

msg = 'zu viele Sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg)

check_mask:  17 rows
Move 17 rows from 4845941
17 rows out of 6361


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order
64801,a._D.,alt_2,a._D.~a. d. D,a. d. D.,,,3,,11.459572,True,zu viele Sonderzeichen,1.0
64861,a._O.,alt_2,a._O.~a. d. O,a. d. O.,,TODO,2,,6.504342,True,zu viele Sonderzeichen,1.0
64862,a._O.,alt_2,a._O.~a. a. O,a. a. O.,,TODO,3,,6.504342,True,zu viele Sonderzeichen,2.0
68942,GmbH,alt_1,GmbH~G. m. b. H,G. m. b. H.,,NN,,,73.640892,True,zu viele Sonderzeichen,1.0
141173,BfA,up,,K. d. ö. R.,,NN,1,,18.144148,False,zu viele Sonderzeichen,1.0
176090,MdB,alt_2,MdB~M. d. B,M. d. B.,,TODO,,,7.672363,True,zu viele Sonderzeichen,1.0
176094,MdL,alt_2,MdL~M. d. L,M. d. L.,,TODO,1,,13.429291,True,zu viele Sonderzeichen,1.0
178302,aaO,alt_2,aaO~a. a. O,a. a. O.,,TODO,1,,12.276679,True,zu viele Sonderzeichen,1.0
268801,RAF,alt_2,RAF~R. A. F,R. A. F.,,TODO,3,,36.033504,True,zu viele Sonderzeichen,1.0
322500,u._V.,alt_1,u._V.~u. ü. V,u. ü. V.,,TODO,,,1.937145,True,zu viele Sonderzeichen,1.0


In [654]:
# fehlende Klammern
mask1 = wiktionary_merkmal.data.str.count('\(') != wiktionary_merkmal.data.str.count('\)')
mask2 = wiktionary_merkmal.data.str.len() > 1
mask = mask1  &  mask2
check_mask( wiktionary_merkmal, mask, 0, 10)

msg = 'fehlende_Klammern'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [655]:
# Fugenelemente
mask1 =  wiktionary_merkmal.data != '-'
mask2 =  wiktionary_merkmal.data.str.startswith('-')
mask3 =  wiktionary_merkmal.data.str.endswith('-')
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0)

msg = 'Fugenelemente'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


### aufsplitten
wiktionary_merkmal wird in drei Teile aufgesplittet:
* wiktionary_merkmal wurzelt in wiktionary_lemma
* wiktionary_lexmerkmal wurzelt in translate_lex.<br>
  translate_lex enthält ja Lexeme (z.B. die Deklinierten Formen). <br>
  wiktionary_lexmerkmal enthält die Merkmale dieser Lexeme.<br>
* translate_gm enthält alle gm-Merkmale, also die Verweise von einer deklinierten Form auf das Lemma.<br>

In [656]:
# Spalten löschen
wiktionary_merkmal = pak.drop_cols( wiktionary_merkmal, ['lemma_lower'])

In [657]:
# anschauen: verwaistes und verschollenes lemma_id
mask1 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
mask2 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma_trash.lemma_id)
mask3 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma_fix.lemma_id)
mask4 = ~wiktionary_merkmal.lemma_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
#grid(wiktionary_merkmal,mask)

In [658]:
# verwaistes lemma_id
# Ursache: Belöschen von wiktionary_lemma
mask1 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
mask2 = ~wiktionary_merkmal.lemma_id.isin(translate_lex.data_id)
mask = mask1  &  mask2

# mask nur für die Anzeige
mask_a1 = ~wiktionary_merkmal.lemma_id.str.startswith('-')
mask_a2 = ~wiktionary_merkmal.lemma_id.str.endswith('-')
mask_a = mask  &  mask_a1  &  mask_a2

a = pak.analyse_freqs(wiktionary_merkmal[mask_a].sort_values('data_score',ascending=False),['merkmal','lemma_id'])
grid(a)

30 rows


,merkmal,merkmal_count,merkmal_percent,lemma_id,lemma_id_count
0,ipa,2650,27.6,"[OP, IKT, Fachleute, AIDS, IS, MD, UTC, USBV, BENELUX, sprich, Pegida, MDR, KGV, BUND, AT, NP, XML, RKI, AC, multikulti]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,syllables,2529,26.3,"[Fachleute, UTC, sprich, Pegida, MSc, XML, QmP, multikulti, PISA, HDGDL, KIP, hdgdl, Hi-Fi, MP3, sollen_1, nachgetan, verspeist, getan, beim_Teutates!, betroffen_1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,def,1501,15.6,"[gel., o._B., cui_bono?, MD, BN, gem., ges., Wa., HDGDL, gef., gew., BB, LGBTQIA, AAA, gen., Ges., BNA, geb., LGBTQ, gest.]","[23, 18, 14, 13, 11, 10, 10, 10, 10, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8]"
3,up,1320,13.7,"[E_450, E_452, E_451, E_339, E_341, E_340, E_504, E_503, E_501, E_966, E_967, E_515, E_479b, E_300, E_461, E_516, E_466, E_470a, E_468, E_464]","[16, 16, 15, 12, 12, 12, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 7, 7, 7, 7]"
4,rhymes,468,4.9,"[Fachleute, AIDS, sprich, Pegida, Legida, LEGIDA, PEGIDA, MERS, betan, mechend, gesamt_1, versiert_2, sa!, unterminiert_1, getan, sehen_1, wollen_1, lassen_1, betroffen_1, sollen_1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,alt_1,390,4.1,"[Dipl.-Kff., as., PersAuswG, kartle, kartele, odle, anheimle, anheimele, odele, kartel, Fexen, odel, ab-_1, heimle_an, heimele_an, Baumgarten_1, anheimel, heimel_an, YOLO, EBIT]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1]"
6,alt_2,263,2.7,"[Az., gef., bez., am., Ges., geb., MSc, A.C.A.B., B_42, Qu., Hi-Fi, hapax_legomenon, Hapaxlegomenon, Lo-Fi, it., FUBAR, me., HiFi, Hifi, s._Br.]","[6, 4, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
7,gm,96,1.0,"[weiterverhandelt_1, unwirschen_1, Dice, wohlgetan, schwergetan, schleufend, geschleuft, leichtgetan, schöngetan, weh_getan, leidgetan, geheimgetan, lassen_1, gebitten_1, aufgetan, hineingetan, hinzugetan, dazugetan, umgetan, weggetan]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
8,gegen,78,0.8,"[QbA, QmP, 2_Mos, 4_Mos, 3_Mos, 5_Mos, MDR, Hi-Fi, Lo-Fi, 3_Makk, 2_Makk, 4_Makk, 1_Makk, OOD, OOA, FWDL, SaZ, GWDL, OLAP, B2C]","[9, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 1, 1]"
9,familie,66,0.7,"[be-…-t, Ge-…-e, top-_1, multikulti, zer-…-t, OP, AIDS, DV, IS, Pegida, ASL, NP, EEG, MP3, Hiat, FDJ, VAE, GEMA, DAX, BRICS]","[24, 13, 7, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [659]:
# verwaistes lemma_id löschen
check_mask(wiktionary_merkmal,mask,17724) 

msg = 'verwaist_lemma_id'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)
#grid( wiktionary_merkmal_trash, msg )

check_mask:  19225 rows
Move 19225 rows from 4845924


In [660]:
# anschauen
mask = wiktionary_merkmal_trash.msg == msg
a = pak.analyse_freqs( wiktionary_merkmal_trash[mask], ['merkmal','lemma_tag','data',] )
grid(a)

43 rows


,merkmal,merkmal_count,merkmal_percent,lemma_tag,lemma_tag_count,data,data_count
0,familie,4885,25.4,"[AFIX, ZFIX, ZFIX NE, , TODO, NE]","[2355, 2344, 96, 71, 18, 1]","[Schizophrenie, Logopädie, Homöopathie, Empathie, Autograph, Autokrat, Autokratie, Orthopädie, desinfizieren, klassifizieren, zertifizieren, Hypophyse, Hypotonie, Heliostat, Hygrostat, Mesopotamien, Mikronesien, Ichthyophage, fernab, herab]","[3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
1,ipa,3552,18.5,"[, AFIX, ZFIX, TODO, NE, ZFIX NE, ZFIX PPER]","[2258, 538, 276, 197, 189, 93, 1]","[-ɛk, ap, at, ak, an, ˈloʊˌfaɪ, ˈhaɪˌfi, -dʁʊf, -fʊʁt, -føːʁdə, kalviˈnɪstɪʃstn, daːl, -fɔʁt, baːɐ, ɡʁaːf, ɪŋən, -aʊ, -ʁɪc, -maːʁ, daks]","[4, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
2,syllables,3420,17.8,"[, AFIX, ZFIX, NE, ZFIX NE, TODO, ZFIX PPER]","[2290, 522, 281, 197, 93, 36, 1]","[in, gon, an, top, ab, Hi⇀Fi, Lo⇀Fi, tel, bar, therm, at, ar, al, ant, phil, en, ier, arch, coel, Hu⇀ken]","[4, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
3,def,2594,13.5,"[TODO, AFIX, ZFIX, NE, ZFIX NE, , ZFIX PPER]","[1218, 665, 296, 218, 131, 64, 2]","[Familienname, Ortsnamengrundwort, Nachname, Gesetz, Kraftfahrzeugkennzeichen, Bundeswehr, Dienstgrad, deutsch, Bundesministerium, Landkreis, Abkürzung, Siedlung, Verordnung, ohne, gegen, bei, deutschsprachig, weg, Kfz-Kennzeichen, hinzu]","[129, 49, 47, 39, 35, 33, 32, 30, 24, 23, 21, 17, 17, 13, 13, 13, 12, 11, 11, 11]"
4,up,1330,6.9,"[TODO, ZFIX, , AFIX]","[1317, 8, 4, 1]","[Chemie, Lebensmittelzusatzstoff, chemisch, Verbindung, chemische Verbindung, Konservierungsmittel, Farbstabilisator, Trennmittel, Emulgator, Antioxidans, Stabilisator, Säureregulator, Trägerstoff, Füllstoff, Geschmacksverstärker, Komplexbildner, Verdickungsmittel, Mehlbehandlungsmittel, Festigungsmittel, Überzugsmittel]","[210, 176, 113, 110, 92, 36, 36, 36, 33, 32, 24, 23, 23, 23, 22, 20, 19, 18, 15, 14]"
5,alt_1,750,3.9,"[, AFIX, ZFIX, TODO, ZFIX NE, NE]","[355, 288, 53, 30, 22, 2]","[glyco, sur, glyk, glyc, glyz, gluco, gluko, glyko, kom, suf, sug, suk, sup, sub, kol, kor, kon, -ation, -ition, -tion]","[6, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3]"
6,lateral,686,3.6,"[AFIX, ZFIX, TODO, ZFIX NE, ]","[459, 159, 32, 25, 11]","[weg, nicht, über, sehr, gegen, ober, nieder, vor, völlig, hinein, miss, unter, -isch, -lich, zusammen, fort, runter, -er, herunter, anti]","[12, 11, 9, 8, 8, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5]"
7,rhymes,479,2.5,"[, NE, TODO, AFIX, ZFIX, ZFIX NE]","[300, 143, 24, 7, 4, 1]","[iːda, asəstn, iːɐ, aːɐ, aːl, iːɐtn, iːɐtə, ɛːsɐ, iːɐtəst, iːʁəst, iːɐst, iːʁət, iːɐtət, iːʁə, aːzn, aːn, aks, eː, aɪtə, ɛlə]","[4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]"
8,alt_2,323,1.7,"[, TODO, AFIX, ZFIX, NE]","[141, 120, 43, 15, 4]","[nördl. Br., nördl. Breite, südl. Br., südl. Breite, s. Br., n. Br., Hapax Legomenon, Hapax legomenon, rasser, rassesten, rässer, rässesten, Gld, Hifi, HiFi, Hi-Fi, Lofi, LoFi, Lo-Fi, P. T.]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1]"
9,gegen,256,1.3,"[AFIX, TODO, ZFIX, ]","[161, 76, 18, 1]","[weg, raus, herauf, hinein, herunter, fremd, pro, vor, hinauf, rauf, rein, hoch, 1 Mos, hetero, unter, nach, ober, gegen, -phob, 2 Makk]","[10, 6, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3]"


In [661]:
# verwaiste data_id (früher wurden die gelöscht) 
mask1 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask2 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask3 =  wiktionary_merkmal.data_id != ''
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask, 49440)
grid(wiktionary_merkmal,mask)

check_mask:  49492 rows
49492 rows out of 4826699


,lemma_id,merkmal_order,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
i,,,,,,,,,,,
96,Dezember,1,alt,Dezember~December,December,,NN,,,212.693924,True
345,August_1,1,alt_abk,August_1~Gust,Gust,,NE,1,,9.259675,True
346,August_1,1,alt_klein,August_1~Gustel,Gustel,,NE,1,,9.259675,True
399,Oktober,1,alt,Oktober~October,October,,NN,,,240.864517,True
785,Litschi,1,alt_1,Litschi~Lychee,Lychee,,NN,,,39.416626,True
...,...,...,...,...,...,...,...,...,...,...,...
3178147,hinter_1,1,Superlativ,hinter_1~hinterst,hinterst,,ADJA,,,3.623495,True
3178633,ober-_1,1,Superlativ,ober-_1~oberst,oberst,,ADJA,,,30.302841,True
4315886,deins_1,1,syllables,deins,deins,,,,,0.201430,False


In [662]:
# msg = 'verwaist_data_id'
# wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg)
# grid( wiktionary_merkmal_trash, msg )

In [663]:
# # anschauen
# mask = wiktionary_merkmal_trash.msg == msg
# a = pak.analyse_freqs( wiktionary_merkmal_trash[mask], ['merkmal','lemma_tag','data',] )
# grid(a)

In [664]:
# translate_gm abspalten
mask = wiktionary_merkmal.merkmal == 'gm'
check_mask(wiktionary_merkmal, mask, 694604)
wiktionary_merkmal, translate_gm = pak.move_rows(wiktionary_merkmal, mask, msg=None)

check_mask:  694604 rows
Move 694604 rows from 4826699


In [665]:
# wiktionary_lexmerkmal abspalten
mask1 = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
mask2 =  wiktionary_merkmal.lemma_id.isin(translate_lex.data_id)
mask = mask1  &  mask2
check_mask(wiktionary_merkmal, mask, 1714508)

wiktionary_merkmal, wiktionary_lexmerkmal = pak.move_rows(wiktionary_merkmal, mask, msg=None)
wiktionary_lexmerkmal = pak.rename_col(wiktionary_lexmerkmal, 'lemma_id','lex_id')

check_mask:  1714508 rows
Move 1714508 rows from 4132095


### pflegen

In [666]:
# suche_debug wiktionary_merkmal
a = search_str(wiktionary_merkmal, suche_debug)
grid(a)

86 rows


,lemma_id,merkmal_order,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex
i,,,,,,,,,,,
640,Milliarde,2,kombi,Euro,Euro,,NN,1,Milliarden,231.778000,False
25446,Ei,34,def,Euro,Euro,,NN,9,synonym,2585.854980,False
25553,Ei,33,down,Euro,Euro,,NN,9,,2585.854980,False
38565,verstehen,29,kombi,Euro,Euro,,VVFIN,1,verstehen,1248.589844,False
41835,kriegen,3,kombi,zu_viel,zu viel,,VVFIN,1,kriegen,337.578461,False
...,...,...,...,...,...,...,...,...,...,...,...
2951779,Euro,2,Dat Pl,Euro,Euro,,NN,,,153.715988,True
3033696,Euro,1,Akk Pl,Euros,Euros,,NN,,,153.715988,True
3036860,Euro,2,Akk Pl,Euro,Euro,,NN,,,153.715988,True


In [667]:
# lemma nach wiktionary_merkmal übertragen
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='lemma')

update_col: 2417587 cells written into new column


In [668]:
# lemma_tag nach wiktionary_merkmal übertragen
wiktionary_merkmal['lemma_tag'] =  wiktionary_merkmal.lemma_tag.fillna('')
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='tag', col_rename='lemma_tag')
wiktionary_merkmal['lemma_tag'] =  wiktionary_merkmal.lemma_tag.fillna('')

update_col: 2417587 cells written into existing column


In [669]:
# sicherstellen: lemma_tag ist gesetzt
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag == ''
mask  =  mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


### Schrott löschen 2
* diesmal mit den Feldern lemma und lemma_tag

In [670]:
# beginnt mit Sonderzeichen
erlaubt = ["'nen"]
mask0 = ~wiktionary_merkmal.data.isin(erlaubt)
mask1 =  wiktionary_merkmal.data.str.len() > 1
mask2 = ~wiktionary_merkmal.data.str[0].str.isalpha()  
mask3 = ~wiktionary_merkmal.data.str[0].str.isnumeric()    
mask4 = ~wiktionary_merkmal.lemma_tag.isin([ '$(', '$.', '$,', 'ZFIX NE', 'ZFIX', 'CARD' ])
mask5 =  wiktionary_merkmal.is_lex == True
mask  =  mask0  &  mask1  &  mask2  &  mask3  &  mask4  &  mask5    
check_mask( wiktionary_merkmal, mask, 88)

msg = 'beginnt mit Sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg ) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  88 rows
Move 88 rows from 2417587
88 rows out of 25674


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
17926,wa_1,alt_2,wa_1~-wa,-wa,,PPER,,,1.455715,True,beginnt mit Sonderzeichen,1.0,wa
841743,Ache,alt_3,-ach,-ach,,NE,1,,26.709286,True,beginnt mit Sonderzeichen,5.0,Ache
1549949,m-RNA,alt_2,m-RNA~(m)RNA,(m)RNA,,NN,,,48.221127,True,beginnt mit Sonderzeichen,2.0,m-RNA
1598833,mRNA,alt_2,mRNA~(m)RNA,(m)RNA,,NN,,,7.196318,True,beginnt mit Sonderzeichen,2.0,mRNA
2273934,Doarschdich,Nom Sg,Doarschdich~(der) Doarschdich,(der) Doarschdich,,NN,,,14.964546,True,beginnt mit Sonderzeichen,1.0,Doarschdich
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007583,Zaischdich,Akk Pl,Zaischdich~(die) Zaischdich,(die) Zaischdich,,NN,,,30.504677,True,beginnt mit Sonderzeichen,1.0,Zaischdich
3008623,Faidich,Akk Pl,Faidich~(die) Faidich,(die) Faidich,,NN,,,15.196615,True,beginnt mit Sonderzeichen,1.0,Faidich
3008712,Sonndich,Akk Pl,Sonndich~(die) Sonndich,(die) Sonndich,,NN,,,13.335077,True,beginnt mit Sonderzeichen,1.0,Sonndich
3008791,Werdich,Akk Pl,Werdich~(die) Werdich,(die) Werdich,,NN,,,14.827385,True,beginnt mit Sonderzeichen,1.0,Werdich


In [671]:
# nicht_erlaubte_sonderzeichen 
mask1 = wiktionary_merkmal.data.str.contains(nicht_erlaubte_sonderzeichen)   
mask2 = ~wiktionary_merkmal.lemma_tag.isin(['$(','$.','$,'])
mask  =  mask1  &  mask2 
check_mask( wiktionary_merkmal, mask, 2)

msg = 'nicht_erlaubte_sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg)

check_mask:  2 rows
Move 2 rows from 2417499
2 rows out of 25676


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
1830809,umeinanderhetzen,Befehl du,umeinanderhetzen~hetz umeinander!,hetz umeinander!,,VVFIN,,,23.235119,True,nicht_erlaubte_sonderzeichen,1.0,umeinanderhetzen
1830810,umeinanderhetzen,Befehl ihr,umeinanderhetzen~hetzt umeinander!,hetzt umeinander!,,VVFIN,,,23.235119,True,nicht_erlaubte_sonderzeichen,1.0,umeinanderhetzen


In [672]:
# zu viele Wörter 
mask1 =  wiktionary_merkmal.data.str.split().str.len() > 3
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.merkmal.str.startswith('alt') 
mask  =  mask1  &  mask2  &  mask3 
check_mask( wiktionary_merkmal, mask, 0)

msg = 'zu viele Wörter'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [673]:
# Verbotene Fragmente in data, die nicht im lemma auftreten
verbotene_fragmente = ['Konjugation','Genitiv','Dativ','(',')',':','»','<','!','#',',',';','!','?']
wiktionary_merkmal['error'] = None
for verboten in verbotene_fragmente:
    mask1 =  wiktionary_merkmal.data.str.contains(verboten, regex=False)    
    mask2 = ~wiktionary_merkmal.lemma.str.contains(verboten, regex=False)  
    mask3 =  wiktionary_merkmal.data.str.len() > 1       
    mask4 =  wiktionary_merkmal.is_lex == True   
    mask5 = ~wiktionary_merkmal.merkmal.str.startswith('alt') 
    mask  =  mask1  &  mask2  &  mask3  &  mask4  &  mask5
    wiktionary_merkmal.loc[ mask, 'error'] = 'Verbotenes Fragment ' + verboten

In [674]:
mask = ~wiktionary_merkmal.error.isnull()  
check_mask( wiktionary_merkmal, mask, 0, 5)
#grid( wiktionary_merkmal, mask)

wiktionary_merkmal = pak.drop_cols( wiktionary_merkmal, 'error' )
msg = 'Verbotene Fragmente'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [675]:
# Überflüssige lemmagleiche Lexeme löschen
mask1  = wiktionary_merkmal.is_lex == True
mask2  = wiktionary_merkmal.data == wiktionary_merkmal.lemma
mask3A = wiktionary_merkmal.merkmal.str.startswith('alt') 
mask3B = wiktionary_merkmal.merkmal.isin( ['lex'] )
mask  = mask1  &  mask2  &  (mask3A | mask3B) 
check_mask( wiktionary_merkmal, mask, 210)

msg = 'lemmagleiche Lexeme'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  209 rows
Move 209 rows from 2417497
209 rows out of 25885


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
40468,Dojinshi,alt_2,Dojinshi,Dojinshi,,NN,,,47.115898,True,lemmagleiche Lexeme,2.0,Dojinshi
60510,Eiscreme,alt_2,Eiscreme,Eiscreme,,NN,,,71.216179,True,lemmagleiche Lexeme,1.0,Eiscreme
69528,Piezokeramik,alt_2,Piezokeramik,Piezokeramik,,NN,,,20.034588,True,lemmagleiche Lexeme,1.0,Piezokeramik
74078,Abbe_1,alt_3,Abbe_1,Abbe,,NE,1,,2.571619,True,lemmagleiche Lexeme,1.0,Abbe
77529,Chisinau,alt_2,Chisinau,Chisinau,,NE,,,48.291943,True,lemmagleiche Lexeme,1.0,Chisinau
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1761767,Enquete-Kommission,alt_2,Enquete-Kommission,Enquete-Kommission,,NN,,,40.098053,True,lemmagleiche Lexeme,1.0,Enquete-Kommission
1763657,Derrieregarde,alt_2,Derrieregarde,Derrieregarde,,NN,,,79.929085,True,lemmagleiche Lexeme,1.0,Derrieregarde
1788509,Schuhcreme,alt_2,Schuhcreme,Schuhcreme,,NN,,,47.697083,True,lemmagleiche Lexeme,1.0,Schuhcreme
1792201,Rasiercreme,alt_2,Rasiercreme,Rasiercreme,,NN,,,36.922779,True,lemmagleiche Lexeme,1.0,Rasiercreme


In [676]:
# Überflüssige lemmagleiche Nichtlexeme löschen
mask1 =  wiktionary_merkmal.is_lex == False
mask2 =  wiktionary_merkmal.data == wiktionary_merkmal.lemma
mask3 = ~wiktionary_merkmal.merkmal.isin( ['syllables','Hilfsverb'] )
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 1034)

msg = 'lemmagleiche Nichtlexeme'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  1034 rows
Move 1034 rows from 2417288
1034 rows out of 26919


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
400,November,herkunft,November,November,,NN,,,199.928970,False,lemmagleiche Nichtlexeme,1.0,November
426,ich,herkunft,ich,ich,,PPER,,,2407.714844,False,lemmagleiche Nichtlexeme,2.0,ich
456,lieben,herkunft,lieben,lieben,,VVFIN,,,173.774323,False,lemmagleiche Nichtlexeme,1.0,lieben
2919,klieben,herkunft,klieben,klieben,,VVFIN,,,106.576828,False,lemmagleiche Nichtlexeme,1.0,klieben
3362,See_1,gegen,See,See,,NN,1,,44.082108,False,lemmagleiche Nichtlexeme,6.0,See
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3854911,halt_1,ipa,halt_1,halt,,ITJ,,,1.191096,False,lemmagleiche Nichtlexeme,1.0,halt
3854915,halt,ipa,halt,halt,,ADV,,,21.113094,False,lemmagleiche Nichtlexeme,1.0,halt
3858856,pst,ipa,pst,pst,,ITJ,,,27.485693,False,lemmagleiche Nichtlexeme,1.0,pst
3897660,hm,ipa,hm,hm,,PTKANT,,,24.089207,False,lemmagleiche Nichtlexeme,1.0,hm


In [677]:
# Benannte Lexeme
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.merkmal != 'lex'
mask  = mask1  &  mask2
benannte_lexeme = wiktionary_merkmal[mask]

In [678]:
# Überflüssige lex: Die gibt es schon unter einem konkreteren Merkmal
# Das sind nicht so viele, weil die alle in translate_lex sind
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.merkmal == 'lex'
mask3 = pak.isin( wiktionary_merkmal, benannte_lexeme, on=['lemma','data'] )
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 14)

del benannte_lexeme
msg = 'Überflüssige lex'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  14 rows
Move 14 rows from 2416254
14 rows out of 26933


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
1830811,vergleichen,lex,vergleiche,vergleiche,VVIMP,VVFIN,,manuell,96.039993,True,Überflüssige lex,1.0,vergleichen
1831395,was,lex,wessen,wessen,,PWAT PRO,,manuell,144.917435,True,Überflüssige lex,1.0,was
1831408,dies,lex,diese,diese,,PRELS PDS,,manuell,55.655846,True,Überflüssige lex,1.0,dies
1831429,Fachmann,lex,Fachmann~Fachleute,Fachleute,,NN,,manuell,220.107452,True,Überflüssige lex,1.0,Fachmann
1831438,Versprechung,lex,Versprechungen,Versprechungen,,NN,,manuell,39.175423,True,Überflüssige lex,1.0,Versprechung
1831459,unser_1,lex,unser_1~unserm,unserm,,PPOSS,,manuell,9.851557,True,Überflüssige lex,1.0,unser
1831461,stören,lex,stört,stört,,VVFIN,,manuell,115.850243,True,Überflüssige lex,1.0,stören
1831474,Risikofaktor,lex,Risikofaktoren,Risikofaktoren,,NN,,manuell,69.324532,True,Überflüssige lex,1.0,Risikofaktor
1831488,Herr,lex,Herrn,Herrn,,NN,,manuell,1010.608398,True,Überflüssige lex,1.0,Herr
1831511,Liebkosung,lex,Liebkosungen,Liebkosungen,,NN,,manuell,45.781231,True,Überflüssige lex,1.0,Liebkosung


In [679]:
# TRUNC löschen
mask =  wiktionary_merkmal.lemma_tag == 'TRUNC'
check_mask( wiktionary_merkmal, mask, 0, 4)

msg = 'TRUNC'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

check_mask:  2 rows
Move 2 rows from 2416240
3 rows out of 26935


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
439905,www,def,WWW,WWW,,TRUNC,1,,2.239461,False,def einer Abkürzung,NaN,<NA>
569109,http,kombi,www,www,,TRUNC,1,http,4.404000,False,TRUNC,1.0,http
3794403,http,ipa,,haː teː teː ˈpeː,,TRUNC,,,4.404000,False,TRUNC,1.0,http


In [680]:
#search_str(wiktionary_lemma, 'http')

In [681]:
# suche_debug
maskA = wiktionary_merkmal_trash.data.isin(suche_debug)
maskB = wiktionary_merkmal_trash.lemma.isin(suche_debug)
mask = maskA | maskB
grid(wiktionary_merkmal_trash,mask, color='green')

3 rows out of 26935


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
1064204,TEuro,def,Euro,Euro,,TODO,1,,13.952158,False,def einer Abkürzung,NaN,<NA>
182024,EuroEG,def,Euro,Euro,,TODO,1,,3.685747,False,verwaist_lemma_id,3.0,<NA>
368994,TEUR,def,Euro,Euro,,TODO,1,,3.326314,False,verwaist_lemma_id,1.0,<NA>


In [682]:
# suche_debug
#mask = wiktionary_merkmal.data.isin(suche_debug)
mask = wiktionary_merkmal.lemma.isin(suche_debug)
#mask = maskA | maskB
grid(wiktionary_merkmal,mask)

67 rows out of 2416238


,lemma_id,merkmal_order,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,lemma
i,,,,,,,,,,,,
129289,Euro,1,def,Währung,Währung,,NN,1,,153.715988,False,Euro
129290,Euro,2,def,Europäischen_Währungsunion,Europäischen Währungsunion,,NN,1,,153.715988,False,Euro
129291,Euro,1,syn,Teuro,Teuro,,NN,1,scherzhaft,153.715988,False,Euro
129292,Euro,1,up,Währung,Währung,,NN,1,,153.715988,False,Euro
129293,Euro,1,familie,Eurobond,Eurobond,,NN,,,153.715988,False,Euro
...,...,...,...,...,...,...,...,...,...,...,...,...
3529622,Euro_1,1,ipa,,ˈɔɪʁo,,NN,,,4.110498,False,Euro
3893764,zu_viel,1,ipa,,ˈtsuː fiːl,,ADV,,,14.845442,False,zu viel
4346687,zu_viel,1,syllables,,zu⇀viel,,ADV,,,14.845442,False,zu viel


In [683]:
# Nochmal sicherstellen: Alle Datensätze in wiktionary_merkmal haben einen passenden DS in wiktionary_lemma
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [684]:
# Spalten sortieren
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal, ['lemma_id','merkmal','is_lex','data_id','data','data_tag','lemma','lemma_tag','num','meta',])
wiktionary_merkmal.head(1)

,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
i,,,,,,,,,,,,
0,Hallo,syn,False,Gejohle,Gejohle,,Hallo,NN,1,,1,34.813919


### Grundformen anfügen

In [685]:
# Grundformen (merkmal 'lemma')
spalten = ['lemma_id', 'lemma', 'tag_0', 'lemma_score' ]
grundformen = wiktionary_lemma[spalten].copy()
grundformen = pak.rename_col(grundformen, 'tag_0', 'lemma_tag')
grundformen = pak.rename_col(grundformen, 'lemma_score', 'data_score')
grundformen['merkmal']  = 'lemma'
grundformen['is_lex']   = True
grundformen['data']     = grundformen.lemma.copy()
grundformen['data_id']  = grundformen.lemma_id.copy()
grundformen['data_tag'] = grundformen.lemma_tag.copy()
grundformen['meta']     = ''
grundformen['num']      = ''


# Spaltenvergleich
bpy.minivenn( grundformen.columns, wiktionary_merkmal.columns, format='print2')

left_only:  {}
right_only: {'merkmal_order'}



In [686]:
# add_rows
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, grundformen)
pak.sample(grundformen)

168007 rows added, now a total of 2584245


,lemma_id,lemma,lemma_tag,data_score,merkmal,is_lex,data,data_id,data_tag,meta,num
0,24/7,24/7,ADJA,14.942436,lemma,True,24/7,24/7,ADJA,,
1358,Schlange_stehen,Schlange stehen,VVFIN,18.997892,lemma,True,Schlange stehen,Schlange_stehen,VVFIN,,
2696,!_M,!,$.,6.718384,lemma,True,!,!_M,$.,,
2722,"1,4-Butandiol","1,4-Butandiol",NN,90.973969,lemma,True,"1,4-Butandiol","1,4-Butandiol",NN,,
2723,"1,5-Grad-Ziel","1,5-Grad-Ziel",NN,2508.954590,lemma,True,"1,5-Grad-Ziel","1,5-Grad-Ziel",NN,,
2893,30er,30er,ADJA,514.831238,lemma,True,30er,30er,ADJA,,
3122,Aadolf_V,Aadolf,NE,6.708904,lemma,True,Aadolf,Aadolf_V,NE,,
4603,Acre,Acre,NN,116.233994,lemma,True,Acre,Acre,NN,,
41415,Haus,Haus,NN,108960.914062,lemma,True,Haus,Haus,NN,,
79453,Riß_2,Riß,NN,0.000300,lemma,True,Riß,Riß_2,NN,,


In [687]:
# Sonderfall Mag.a löschen
mask = wiktionary_merkmal.data_id == 'Mag.a'
check_mask(wiktionary_merkmal,mask,0,2)

msg = 'Sonderfall Mag.a' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  2 rows
Move 2 rows from 2584245
2 rows out of 26937


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
527914,Mag.,weib,Mag.a,Mag.a.,,NN,1,,56.269928,True,Sonderfall Mag.a,1.0,Mag.
2476230,Mag.a,lemma,Mag.a,Mag.a,NN,NN,,,18.292961,True,Sonderfall Mag.a,NaN,Mag.a


In [688]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [689]:
# Verwaiste Merkmale: Gibt es nicht, weil die abgetrennt oder gelöscht wurden
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask( wiktionary_merkmal, mask, 0)
#grid( wiktionary_merkmal, mask, )

# löschen
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_merkmal, wiktionary_merkmal_trash, mask, 'verwaist4')
grid(wiktionary_merkmal_trash, color='green')

check_mask:  0 rows
No rows moved
26937 rows


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
2151033,Außerm_Pfarrgarten,Genus,,,,NE,,,26.970110,False,Leerstring data Müll,NaN,<NA>
2177557,Heisei,Genus,,,,NN,,,44.313446,False,Leerstring data Müll,NaN,<NA>
2177564,Reiwa,Genus,,,,NN,,,45.144207,False,Leerstring data Müll,NaN,<NA>
2183522,Covid,Genus,,,,NE,,,44.336594,False,Leerstring data Müll,NaN,<NA>
2201278,Am_Bahnhof,Genus,,,,NE,,,26.080570,False,Leerstring data Müll,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831637,irgendwelche,lex,irgendwelche~irgendwelcher,irgendwelcher,,PIAT,,manuell,49.079304,True,Überflüssige lex,1.0,irgendwelche
569109,http,kombi,www,www,,TRUNC,1,http,4.404000,False,TRUNC,1.0,http
3794403,http,ipa,,haː teː teː ˈpeː,,TRUNC,,,4.404000,False,TRUNC,1.0,http
527914,Mag.,weib,Mag.a,Mag.a.,,NN,1,,56.269928,True,Sonderfall Mag.a,1.0,Mag.


In [690]:
# fillna
wiktionary_merkmal['meta']          = wiktionary_merkmal.meta.fillna('')
wiktionary_merkmal['data_tag']      = wiktionary_merkmal.data_tag.fillna('')
wiktionary_merkmal['lemma_tag']     = wiktionary_merkmal.lemma_tag.fillna('')
wiktionary_merkmal['num']           = wiktionary_merkmal.num.fillna('')
wiktionary_merkmal['merkmal_order'] = wiktionary_merkmal.merkmal_order.fillna(0)

In [691]:
#wiktionary_merkmal

In [692]:
# update_col aus wiktionary_lemma
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='lemma')
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, wiktionary_lemma, on='lemma_id', col='tag_0', col_rename='lemma_tag')
wiktionary_merkmal['lemma_tag'] =  wiktionary_merkmal.lemma_tag.fillna('')

update_col: 2584243 cells written into existing column
update_col: 2584243 cells written into existing column


In [693]:
# sicherstellen: lemma_tag ist gesetzt
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag == ''
mask  =  mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [694]:
#pak.analyse_cols(wiktionary_merkmal)

### Leer oder ultrakurz

In [695]:
# Prüfen: Keine Leerstrings
spalten = ['merkmal','data','lemma','lemma_tag']
for s in spalten:
    print(s)
    mask = wiktionary_merkmal[s] == ''
    check_mask( wiktionary_merkmal, mask, 0)

merkmal
check_mask:  0 rows
data
check_mask:  0 rows
lemma
check_mask:  0 rows
lemma_tag
check_mask:  0 rows


In [696]:
# noSteig ermitteln und löschen
# data ist nur '-'
mask1 = wiktionary_merkmal.data.str.len() <= 1
mask2 = wiktionary_merkmal.lemma.str.len() > 1
mask3 = wiktionary_merkmal.merkmal.isin(['Komparativ','Superlativ'])
mask4 = wiktionary_merkmal.is_lex == True
mask  =  mask1  &  mask2  &  mask3  &  mask4 
noSteig = wiktionary_merkmal[mask].lemma_id # merken
check_mask( wiktionary_merkmal, mask, 16369)

msg = 'noSteig' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  16369 rows
Move 16369 rows from 2584243
16369 rows out of 43306


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
2052168,ausnehmend,Komparativ,-_M,-,,ADJA,,,12.886402,True,noSteig,1.0,ausnehmend
2052169,virenverseucht,Komparativ,-_M,-,,ADJA,,,60.643898,True,noSteig,1.0,virenverseucht
2052171,unergründbar,Komparativ,-_M,-,,ADJA,,,14.526186,True,noSteig,1.0,unergründbar
2052173,finanziert,Komparativ,-_M,-,,ADJA,,,249.671036,True,noSteig,1.0,finanziert
2052174,organisiert,Komparativ,-_M,-,,ADJA,,,55.607700,True,noSteig,1.0,organisiert
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080190,davysch,Superlativ,-_M,-,,ADJA,,,21.166990,True,noSteig,1.0,davysch
2080191,myanmarisch,Superlativ,-_M,-,,ADJA,,,15.830819,True,noSteig,1.0,myanmarisch
2080192,sinnstiftend,Superlativ,-_M,-,,ADJA,,,26.401480,True,noSteig,1.0,sinnstiftend
2080193,entfernbar,Superlativ,-_M,-,,ADJA,,,21.731077,True,noSteig,1.0,entfernbar


In [697]:
# noSteig in wiktionary_lemma kennzeichnen
mask1 =  wiktionary_lemma.lemma_id.isin(noSteig)
mask2 = ~wiktionary_lemma.member.str.contains('noSteig')
mask3 =  wiktionary_lemma.tagZ == 'ADJ'
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_lemma, mask, 154)

wiktionary_lemma.loc[mask,'member'] += ' noSteig' 
grid( wiktionary_lemma, mask )

check_mask:  154 rows
154 rows out of 168007


,lemma_id,page_id,page_part,lemma,lemma_lower,isupper,lemma_len,tag,tag_0,tag_1,tagZ,tagZZ,member,data_id,lemma_score
1732,amtierend,105889,0,amtierend,amtierend,False,1,ADJA,ADJA,TitlLa2QtX,ADJ,A,Ptz1 manuell noSteig,amtieren,188.885895
1780,ausnehmend,426867,0,ausnehmend,ausnehmend,False,1,ADJA,ADJA,RSGs6X1bRN,ADJ,A,Ptz1 manuell noSteig,ausnehmen,45.377594
1888,dauernd,242274,0,dauernd,dauernd,False,1,ADJA,ADJA,RvKs7khV86,ADJ,A,Ptz1 manuell noSteig,dauern,140.483337
2059,fünfte,119583,0,fünfte,fünfte,False,1,ADJA CARD,ADJA,CARD,ADJ,A,manuell ergänzt noSteig,,154.422852
2341,schwimmend,242155,0,schwimmend,schwimmend,False,1,ADJA,ADJA,JAH8A0YnTp,ADJ,A,Ptz1 manuell noSteig,schwimmen,268.757294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166207,zweiköpfig,538260,0,zweiköpfig,zweiköpfig,False,1,ADJA,ADJA,RJGz2LDIe3,ADJ,A,noSteig,,55.199440
166274,zwiegespalten,417229,0,zwiegespalten,zwiegespalten,False,1,ADJA,ADJA,OaD5nsgT1h,ADJ,A,noSteig,,20.141333
167101,äußer_glex,681179,0,äußer,äußer,False,1,ADJA,ADJA,tg9vI5LJ7X,ADJ,A,noSteig,,304.442383
167131,ökonometrisch,393779,0,ökonometrisch,ökonometrisch,False,1,ADJA,ADJA,zvFyblp88e,ADJ,A,noSteig,,51.162460


In [698]:
# Genus korrigieren
genus_m = ['Stromboli','Micky_Maus','Jesus_Christus','Jehova','Daniel_Düsentrieb','Dagobert_Duck','Tacken','Donald_Duck']

mask1 = wiktionary_merkmal.merkmal == 'Genus'
mask2 = wiktionary_merkmal.data == '0'
mask3 = wiktionary_merkmal.lemma_id.isin(genus_m)
mask  =  mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,len(genus_m))

wiktionary_merkmal.loc[mask,'data'] = 'm'
wiktionary_merkmal[mask]

check_mask:  8 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
1073148,Jesus_Christus,Genus,False,,m,,Jesus Christus,NE,,,1.0,47.556717
1097973,Tacken,Genus,False,,m,,Tacken,NN,,,1.0,93.550880
1099500,Stromboli,Genus,False,,m,,Stromboli,NE,,,1.0,82.534531
1119086,Jehova,Genus,False,,m,,Jehova,NE,,,1.0,139.055496
1135837,Daniel_Düsentrieb,Genus,False,,m,,Daniel Düsentrieb,NE,,,1.0,19.459846
1135885,Dagobert_Duck,Genus,False,,m,,Dagobert Duck,NE,,,1.0,40.256443
1136031,Micky_Maus,Genus,False,,m,,Micky Maus,NE,,,1.0,44.799252
1137261,Donald_Duck,Genus,False,,m,,Donald Duck,NE,,,1.0,22.199026


In [699]:
# Genus korrigieren
genus_f = ['Arminia','Trauerwoche']

mask1 = wiktionary_merkmal.merkmal == 'Genus'
mask2 = wiktionary_merkmal.data == '0'
mask3 = wiktionary_merkmal.lemma_id.isin(genus_f)
mask  =  mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,len(genus_f))

wiktionary_merkmal.loc[mask,'data'] = 'f'
wiktionary_merkmal[mask]

check_mask:  2 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
1103031,Trauerwoche,Genus,False,,f,,Trauerwoche,NN,,,1.0,30.284775
1110182,Arminia,Genus,False,,f,,Arminia,NE,,,1.0,22.124098


In [700]:
# Genus n falsch
mask1 =  wiktionary_merkmal.merkmal == 'Genus'
mask2 =  wiktionary_merkmal.data == 'n'
mask3 = ~wiktionary_merkmal.lemma_tag.isin(['NN','NE'])
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0, 3)

msg = 'Genus n falsch' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [701]:
# Ungültige Genus löschen
mask1 =  wiktionary_merkmal.merkmal == 'Genus'
mask2 = ~wiktionary_merkmal.data.isin(['m','f','n'])
mask  =  mask1  &  mask2 
genus_ungültig = wiktionary_merkmal[mask].lemma_id # merken
check_mask( wiktionary_merkmal, mask, 725)

msg = 'Genus ungültig' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 

mask = wiktionary_merkmal_trash.msg == msg
pak.analyse_freqs( wiktionary_merkmal_trash[mask],['data','lemma'])

check_mask:  725 rows
Move 725 rows from 2567874


,data,data_count,data_percent,lemma,lemma_count
0,0,610,84.1,"[Pipox, Pipock, Hauptleute, Nullerjahre, Metadaten, Liparische Inseln, Acta Sanctorum, Acta Martyrum, Acta Apostolorum, Reisekosten, Landesfarben, Spompanadeln, Fasten, Freundschaftsbande, Wirren, Svear, Mittelsleute, Feuerwehrleute, Fusilli, Allgemeine Geschäftsbedingungen]","[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,x,98,13.5,"[Great Plains, Admiralitätsinseln, Aiko, Akira, Alexis, Alfke, Alison, Alkan, Aydan, Azar, Balter, Bente, Berna, Billy, Bobbi, Bälter, Carol, Cato, Connie, Curly]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,m+n,13,1.8,"[Port Erin, Libanon, Jemen, Oman, Kongo, Südsudan, Sudan, Senegal, Tschad, Niger, Irak, Iran, Kosovo]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,f+n,2,0.3,"[Hinterbrühl, Antarktika]","[1, 1]"
4,o,1,0.1,Mixed Media,1
5,pl,1,0.1,Seschellen,1


In [702]:
# genus_ungültig in wiktionary_lemma betrachten
# Sind da Kandidaten für manuelle Löschung oder Genuss setzen dabei?
mask1 =  wiktionary_lemma.lemma_id.isin(genus_ungültig)
mask2 = ~wiktionary_lemma.member.str.contains('noSg')
mask3 = ~wiktionary_lemma.member.str.contains('noPl')
mask4 = ~wiktionary_lemma.lemma_id.str.endswith('_V')
mask5 = ~wiktionary_lemma.member.str.contains('Person_vorname')
mask6 = ~wiktionary_lemma.member.str.contains('Geo')
mask  =  mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6
check_mask( wiktionary_lemma, mask, 17)

spalten = ['lemma_id','tag','lemma_score','member']
a = wiktionary_lemma[mask][spalten].sort_values(['tag'])
grid(a)

check_mask:  17 rows
17 rows


,lemma_id,tag,lemma_score,member
5987,Allah,NE,429.435516,
81425,Sanssouci,NE,291.748779,
80670,SARS-CoV-2,NE,242.369843,
74229,Post_Covid,NE,124.849808,
74220,Post-Covid-19,NE,283.675659,
74219,Post-Covid,NE,875.175293,
68636,Obamacare,NE,54.890327,
74211,Post-COVID,NE,210.853485,
38336,Google,NE,476.271973,
19907,Christi_Himmelfahrt,NE,95.063065,Wortverbindung


In [703]:
# data ist nur '-'
mask1 =  wiktionary_merkmal.data == '-'
mask2 =  wiktionary_merkmal.lemma_id != '-_M'
#mask3 = ~wiktionary_merkmal.lemma_tag.isin(['VVFIN'])
mask = mask1  &  mask2 # &  mask3
check_mask( wiktionary_merkmal, mask, 1053)

msg = 'data ist nur -' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 

mask = wiktionary_merkmal_trash.msg == msg
pak.analyse_freqs(wiktionary_merkmal_trash[mask], ['merkmal','lemma_id'] ).head(7)

check_mask:  1053 rows
Move 1053 rows from 2567149


,merkmal,merkmal_count,merkmal_percent,lemma_id,lemma_id_count
0,Präsens ich,361,34.3,"[garnen_1, aussüßen, dahinplätschern, düstern, plattern, hallen, hinströmen, angrenzen, mendeln, zusammenbacken, überaltern, irradiieren, zusammenläppern, abperlen, trippen_1, tiefgefrieren, verunkrauten, maßschneidern, kugelstoßen, interferieren]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,Präsens du,361,34.3,"[lumineszieren, verölen, generalüberholen, niederregnen, zubleiben, weihnachten, beruhen, verfließen, zubacken, anbetreffen, fluten, zurückstrahlen, knospen, bauchreden, akkreszieren, gegenargumentieren, myelinisieren, wintern, rückbringen, rückdenken]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,Konj2 ich,92,8.7,"[sippen, wärmebehandeln, chronifizieren, tiefgefrieren, segelfliegen, urformen, lauben_1, kugelstoßen, maßschneidern, drecken_1, drießen, bruchrechnen, dammen_1, größern, garnen_1, angedeihen, fluten, rückführen, bruchlanden, dienstverpflichten]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,Präteritum ich,92,8.7,"[rückrufen, rückreiten, rückschauen, rückschallen, rückschreiten, rückschreiben, rücksenden, rückschwingen, klugscheißen, rücklassen, rücklenken, rücklaufen, rechtschreiben, tiefkühlen, zueinanderhalten, angedeihen, segelfliegen, vorbelasten, voranmelden, rücksinken]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,Präsens ersiees,83,7.9,"[gegenargumentieren, hausschlachten, kontraktieren, gegenfahren, gegenstreben, gegenleisten, interdependieren, generalüberholen, urformen, preiskegeln, kunststopfen, bausparen, funkentstören, handstoppen, manndecken, umfragen, wahrhaben, kopfrechnen, abendessen, mittagessen]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
5,Ptz2,40,3.8,"[dammen_1, milben, größern, drießen, gegenstreben, gegenfahren, drecken_1, garnen_1, interdependieren, wettstreiten, wettrennen, wettlaufen, wettturnen, wettsegeln, wahrhaben, kontraktieren, kugelstoßen, rückschauen, rückschallen, rückreiten]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
6,Positiv,11,1.0,"[weltgrößte, allerbeste, zweitbeste, zweitgrößte, allerliebst, zweitkleinste, allerheiligste, allerunglücklichste, allerniedrigste, neuntgrößte, viertgrößte]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [704]:
# superkurz >> alles bleibt wie es ist
mask1 =  wiktionary_merkmal.data.str.len() <= 1
mask2 =  wiktionary_merkmal.lemma.str.len() > 1   
mask3 = ~wiktionary_merkmal.lemma_tag.isin(['$(','$.','$,'])
mask  =  mask1  &  mask2  &  mask3     

#check_mask( wiktionary_merkmal, mask, 1292)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='superkurz' ) 

pak.analyse_freqs(wiktionary_merkmal[mask],['merkmal','data','lemma_id','lemma_tag'])
#grid( wiktionary_merkmal, mask )

,merkmal,merkmal_count,merkmal_percent,data,data_count,lemma_id,lemma_id_count,lemma_tag,lemma_tag_count
0,Genus,100884,99.9,"[f, m, n]","[43299, 37373, 20212]","[Gams, Wirrwarr, Flipchart, Nudossi, Art_deco, Zigarillo, Cap_1, Sahnejoghurt, Frisbee, Ischias, Halfter, Nutella, Sahnejogurt, Jus, Katschi, Werd, Clickbait, Quandel, Preview, Gendergap]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[NN, NE, CARD]","[95280, 5387, 217]"
1,rhymes,33,0.0,"[a, ɛ, ɔ]","[23, 5, 5]","[Ucha, Abasa, Isba, Sobriquet, Yue, voila, Niamey, Duschanbe, a_bas, Chardonnay, ha, Fauxpas, baba, so_4, aha, Aa_2, tja, na, Cha-Cha-Cha, Omerta]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[NN, ITJ, NE, ADV]","[17, 9, 6, 1]"
2,def,20,0.0,"[8, 2, 5, 9, 3, 1, 7, 4, 6, 0]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[acht, Zwei, fünf, neun, drei, eins, Eins, Fünf, Acht, Drei, Sieben, Vier, Neun, Sechs, Null, sieben, sechs, zwei, null_1, vier]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[CARD, NN]","[10, 10]"
3,lex,10,0.0,"[4, 8, 7, 3, 5, 6, 2, 0, 9, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[vier, acht, sieben, drei, fünf, sechs, zwei, null_1, neun, eins]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",CARD,10
4,ipa,2,0.0,ɛ,2,"[Ain, Ain_1]","[1, 1]",NE,2


### Allgemein

In [705]:
# Explizit gesetzte Tags werden Merkmale
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.data_tag != ''
mask3 = wiktionary_merkmal.data_tag != wiktionary_merkmal.lemma_tag
mask4 = ~((wiktionary_merkmal.data_tag == 'ADJA')  &   (wiktionary_merkmal.lemma_tag == 'VVFIN')) # Partizipien
mask5 = wiktionary_merkmal.merkmal == 'lex'
mask  = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(wiktionary_merkmal,mask,1,50)

# Merkmal erstellen
wiktionary_merkmal.loc[mask,'merkmal'] = wiktionary_merkmal[mask].data_tag

grid(wiktionary_merkmal,mask)
#a = pak.analyse_freqs(wiktionary_merkmal[mask], ['lemma_tag','data_tag'])
#grid(a)

check_mask:  8 rows
8 rows out of 2566096


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
975761,einreden,VVIZU,True,einzureden,einzureden,VVIZU,einreden,VVFIN,,manuell,1.0,77.859055
975762,loswerden,VVIZU,True,loszuwerden,loszuwerden,VVIZU,loswerden,VVFIN,,manuell,1.0,520.682129
975763,herumpimpeln_M,VVIZU,True,herumzupimpeln,herumzupimpeln,VVIZU,herumpimpeln,VVFIN,,manuell,1.0,2.239461
975764,hereinlassen_M,VVIZU,True,hereinlassen_M~hereinzulassen,hereinzulassen,VVIZU,hereinlassen,VVFIN,,manuell,1.0,2.239461
975765,herauslassen_M,VVIZU,True,herauslassen_M~herauszulassen,herauszulassen,VVIZU,herauslassen,VVFIN,,manuell,1.0,2.239461
975766,aufrütteln_M,VVIZU,True,aufzurütteln,aufzurütteln,VVIZU,aufrütteln,VVFIN,,manuell,1.0,2.239461
975767,anstaunen,VVIZU,True,anzustaunen,anzustaunen,VVIZU,anstaunen,VVFIN,,manuell,1.0,26.530531
975768,angucken_M,VVIZU,True,anzugucken,anzugucken,VVIZU,angucken,VVFIN,,manuell,1.0,2.239461


In [706]:
# manuelle Korrekturen
mask = wiktionary_merkmal.merkmal == 'genitiv Sg'
wiktionary_merkmal.loc[mask,'merkmal'] = 'Gen Sg'
mask = wiktionary_merkmal.merkmal ==         'Sg'
wiktionary_merkmal.loc[mask,'merkmal'] = 'Nom Sg'

In [707]:
# Überblick wiktionary_merkmal
a = pak.analyse_freqs(wiktionary_merkmal, ['merkmal', 'data'] )
gridt(a)

73 rows


In [708]:
# alle Merkmale sind in plan_merkmal erfasst
mask = ~a.merkmal.isin(plan_merkmal.merkmal)
check_mask(a,mask,0)

check_mask:  0 rows


In [709]:
#grid(a,mask, color='green')

In [710]:
# suche_debug wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
search_str(wiktionary_merkmal[mask], suche_debug)

,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
349907,zu_viel,alt,True,zu_viel~zuviel,zuviel,,zu viel,ADV,,,1.0,14.845442
975760,zu_viel,lex,True,zu_viele,zu viele,ADV,zu viel,ADV,,manuell,1.0,14.845442
1208674,Euro,Nom Sg,True,Euro,Euro,,Euro,NN,,,1.0,153.715988
1208679,Euro_1,Nom Sg,True,Euro,Euro,,Euro,NN,,,1.0,4.110498
1282015,Euro,Gen Sg,True,Euro,Euro,,Euro,NN,,,1.0,153.715988
1282092,Euro_1,Gen Sg,True,Euro,Euro,,Euro,NN,,,1.0,4.110498
1344642,Euro,Gen Sg,True,Euros,Euros,,Euro,NN,,,2.0,153.715988
1404752,Euro_1,Dat Sg,True,Euro,Euro,,Euro,NN,,,1.0,4.110498
1404762,Euro,Dat Sg,True,Euro,Euro,,Euro,NN,,,1.0,153.715988
1474688,Euro_1,Akk Sg,True,Euro,Euro,,Euro,NN,,,1.0,4.110498


In [711]:
assert not pak.any_nan(wiktionary_merkmal)
#pak.nnan(wiktionary_merkmal)

In [712]:
#mask = wiktionary_merkmal.data_score.isnull()
#pak.sample(wiktionary_merkmal[mask])

### Wortarten -------

#### Partizipien 

In [713]:
# Alle Adjektive
mask = wiktionary_lemma.tagZZ == 'A'
adj = wiktionary_lemma[mask].lemma
#adj

In [714]:
# Alle Partizipien
mask = wiktionary_merkmal.merkmal == 'Ptz1'
ptz1 = wiktionary_merkmal[mask]
mask = wiktionary_merkmal.merkmal == 'Ptz2'
ptz2 = wiktionary_merkmal[mask].copy()
ptz_all = pak.add_rows(ptz1, ptz2)
#ptz_all

14213 rows added, now a total of 27848


In [715]:
# Alle Ptz1 sind auch als Adjektiv registriert
mask = ~ptz1.data.isin(adj)
check_mask(ptz1,mask,0)

check_mask:  0 rows


In [716]:
# Die meisten Ptz2 sind auch als Adjektiv registriert
mask = ~ptz2.data.isin(adj)
check_mask(ptz2,mask,0,120)
grid(ptz2,mask)

check_mask:  109 rows
109 rows out of 14213


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
949081,tun,Ptz2,True,getan,getan,,tun,VAFIN,,,1.0,1.053309
963952,nottun,Ptz2,True,notgetan,notgetan,,nottun,VVFIN,,,1.0,0.391496
965048,dartun,Ptz2,True,dargetan,dargetan,,dartun,VVFIN,,,1.0,0.711443
966125,abtun,Ptz2,True,abgetan,abgetan,,abtun,VVFIN,,,1.0,0.496868
966130,betun,Ptz2,True,betan,betan,,betun,VVFIN,,,1.0,0.536124
...,...,...,...,...,...,...,...,...,...,...,...,...
2023299,hängen_lassen,Ptz2,True,hängen_lassen,hängen lassen,,hängen lassen,VVFIN,,,2.0,27.317692
2023304,zurückwollen,Ptz2,True,zurückwollen,zurückwollen,,zurückwollen,VVFIN,,,2.0,195.421127
2023305,mitwollen,Ptz2,True,mitwollen,mitwollen,,mitwollen,VVFIN,,,2.0,30.395487
2023307,erschallen,Ptz2,True,erschollen,erschollen,,erschallen,VVFIN,,,2.0,144.076263


#### Verben

In [717]:
# Kontrolle: Es gibt keine Doppeltags in lemma_tag und data_tag
mask1 =  wiktionary_merkmal.is_lex    == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3A = wiktionary_merkmal.lemma_tag.str.contains(' ')
mask3B = wiktionary_merkmal.data_tag.str.contains(' ')
mask  =  mask1  &  mask2  &  (mask3A | mask3B)
check_mask( wiktionary_merkmal, mask, 0)

check_mask:  0 rows


In [718]:
# Übersicht: Alle V-Tags in wiktionary_lemma
mask = wiktionary_lemma.tag.str.startswith('V')
a = pak.analyse_freqs(wiktionary_lemma[mask],['tag','lemma'])
grid(a)

3 rows


,tag,tag_count,tag_percent,lemma,lemma_count
0,VVFIN,13853,99.9,"[strippen, scheren, überhängen, erschrecken, docken, durchbrechen, eingeben, gichten, keilen, kollern, losen, löschen, nicken, ramschen, reihen, schlacken, tüdern, umhängen, durchfahren, durchlaufen]","[6, 5, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2]"
1,VMFIN,7,0.1,"[brauchen, dürfen, können, mögen, müssen, sollen, wollen]","[1, 1, 1, 1, 1, 1, 1]"
2,VAFIN,5,0.0,"[tun, haben, sein, werden]","[2, 1, 1, 1]"


In [719]:
# Übersicht: Alle V-Tags in wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask = mask1  &  mask2
a = pak.analyse_freqs(wiktionary_merkmal[mask],['data_tag','lemma_tag','data','lemma'])
a = pak.drop_cols(a,['data_tag_percent','data_count','lemma_count'])
grid(a)

6 rows


,data_tag,data_tag_count,lemma_tag,lemma_tag_count,data,lemma
0,,136462,"[VVFIN, VMFIN, VAFIN]","[136339, 62, 61]","[strippe, strippt, strippte, schere, schert, lost, scherte, erschreckt, löscht, verschleißt, vermisst, beschert, vertiert, konzertiert, befestigt, verführt, detoniert, graviert, laviert, gestrippt]","[strippen, scheren, erschrecken, anbellen, überhängen, löschen, nicken, umhängen, durchkämmen, keilen, reihen, unterschieben, docken, ramschen, tüdern, durchbrechen, backen, losen, murmeln, eingeben]"
1,ADJA,16480,VVFIN,16480,"[aufzuwärmende, aufzuwärmendes, aufzuwärmender, aufzuwärmenden, aufzuwärmendem, abzugießende, abzugießendes, abzugießender, abzugießenden, abzugießendem, nasszuschwitzende, nasszuschwitzendes, nasszuschwitzender, nasszuschwitzenden, nasszuschwitzendem, entzweizureißende, entzweizureißendes, entzweizureißender, entzweizureißenden, entzweizureißendem]","[aufwärmen, abgießen, nassschwitzen, entzweireißen, festnageln, weiterführen, runtersetzen, herausoperieren, abfliegen, abgrenzen, abholzen, weiterforschen, abisolieren, ablassen, abnabeln, abnutzen, abnützen, abringen, abschieben, absegeln]"
2,VVFIN,13853,VVFIN,13853,"[strippen, scheren, überhängen, erschrecken, docken, durchbrechen, eingeben, gichten, keilen, kollern, losen, löschen, nicken, ramschen, reihen, schlacken, tüdern, umhängen, durchfahren, durchlaufen]","[strippen, scheren, überhängen, erschrecken, docken, durchbrechen, eingeben, gichten, keilen, kollern, losen, löschen, nicken, ramschen, reihen, schlacken, tüdern, umhängen, durchfahren, durchlaufen]"
3,VVIZU,8,VVFIN,8,"[einzureden, loszuwerden, herumzupimpeln, hereinzulassen, herauszulassen, aufzurütteln, anzustaunen, anzugucken]","[einreden, loswerden, herumpimpeln, hereinlassen, herauslassen, aufrütteln, anstaunen, angucken]"
4,VMFIN,7,VMFIN,7,"[brauchen, dürfen, können, mögen, müssen, sollen, wollen]","[brauchen, dürfen, können, mögen, müssen, sollen, wollen]"
5,VAFIN,5,VAFIN,5,"[tun, haben, sein, werden]","[tun, haben, sein, werden]"


In [720]:
# Infinitiv
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal == 'lemma'
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask, 13888)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal.lemma_tag.str[0:2] + 'INF'
grid(wiktionary_merkmal,mask)

check_mask:  13865 rows
13865 rows out of 2566096


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
2417596,Schlange_stehen,lemma,True,Schlange_stehen,Schlange stehen,VVINF,Schlange stehen,VVFIN,,,0.0,18.997892
2417926,abbeißen,lemma,True,abbeißen,abbeißen,VVINF,abbeißen,VVFIN,,,0.0,279.654022
2417927,abberufen,lemma,True,abberufen,abberufen,VVINF,abberufen,VVFIN,,,0.0,233.622742
2417928,abfahren,lemma,True,abfahren,abfahren,VVINF,abfahren,VVFIN,,,0.0,2072.352051
2417930,abküssen,lemma,True,abküssen,abküssen,VVINF,abküssen,VVFIN,,,0.0,85.658691
...,...,...,...,...,...,...,...,...,...,...,...,...
2584215,überzählen,lemma,True,überzählen,überzählen,VVINF,überzählen,VVFIN,,,0.0,237.076874
2584219,überzüchten,lemma,True,überzüchten,überzüchten,VVINF,überzüchten,VVFIN,,,0.0,57.731117
2584228,übrigbleiben,lemma,True,übrigbleiben,übrigbleiben,VVINF,übrigbleiben,VVFIN,,,0.0,168.506989
2584234,übrighaben,lemma,True,übrighaben,übrighaben,VVINF,übrighaben,VVFIN,,,0.0,78.089584


In [721]:
# Imperativ
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal.str.contains('Imp')
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask, 37471)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal.lemma_tag.str[0:2] + 'IMP'
grid(wiktionary_merkmal,mask)

check_mask:  37471 rows
37471 rows out of 2566096


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
1569896,vertausendfältigen,Imp Sg,True,vertausendfältig,vertausendfältig,VVIMP,vertausendfältigen,VVFIN,,,1.0,57.528530
1569897,umbrechen,Imp Sg,True,brich_um,brich um,VVIMP,umbrechen,VVFIN,,,1.0,44.905483
1569898,entwurzeln,Imp Sg,True,entwurzel,entwurzel,VVIMP,entwurzeln,VVFIN,,,1.0,25.882940
1569899,nachstreben,Imp Sg,True,streb_nach,streb nach,VVIMP,nachstreben,VVFIN,,,1.0,38.286331
1569900,kahlschlagen,Imp Sg,True,schlag_kahl,schlag kahl,VVIMP,kahlschlagen,VVFIN,,,1.0,91.149025
...,...,...,...,...,...,...,...,...,...,...,...,...
1951854,herziehen,Imp Pl,True,ziehet_her,ziehet her,VVIMP,herziehen,VVFIN,,,2.0,132.659744
1951855,wertschätzen,Imp Pl,True,wertschätzt,wertschätzt,VVIMP,wertschätzen,VVFIN,,,2.0,210.459717
1951856,vorauswissen,Imp Pl,True,wisset_voraus,wisset voraus,VVIMP,vorauswissen,VVFIN,,,2.0,44.799763
1951857,dazugeben,Imp Pl,True,gebet_dazu,gebet dazu,VVIMP,dazugeben,VVFIN,,,2.0,46.132950


In [722]:
# Imperativ: VMIMP gibt es gar nicht
mask = wiktionary_merkmal.data_tag == 'VMIMP'
check_mask(wiktionary_merkmal,mask, 3)
wiktionary_merkmal.loc[mask,'data_tag'] = 'VVIMP'
#grid(wiktionary_merkmal,mask)

check_mask:  3 rows


In [723]:
# Partizip2
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal == 'Ptz2' 
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,14214)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal.lemma_tag.str[0:2] + 'PP'
grid(wiktionary_merkmal,mask)

check_mask:  14213 rows
14213 rows out of 2566096


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
948801,dezidieren,Ptz2,True,dezidiert_2,dezidiert,VVPP,dezidieren,VVFIN,,,1.0,0.057074
948802,dedizieren,Ptz2,True,dediziert_1,dediziert,VVPP,dedizieren,VVFIN,,,1.0,0.056685
948803,verrücken,Ptz2,True,verrückt_2,verrückt,VVPP,verrücken,VVFIN,,,1.0,0.041750
948804,designieren,Ptz2,True,designiert_1,designiert,VVPP,designieren,VVFIN,,,1.0,0.058751
948805,verfluchen,Ptz2,True,verflucht_2,verflucht,VVPP,verfluchen,VVFIN,,,1.0,0.039684
...,...,...,...,...,...,...,...,...,...,...,...,...
2023593,wissen,Ptz2,True,gewußt_MADJ,gewußt,VVPP,wissen,VVFIN,,manuell,2.0,235.562454
2023594,misslingen,Ptz2,True,mißlungen_MADJ,mißlungen,VVPP,misslingen,VVFIN,,manuell,2.0,61.479610
2023595,missverstehen,Ptz2,True,mißverstanden_MADJ,mißverstanden,VVPP,missverstehen,VVFIN,,manuell,2.0,51.494343
2023596,missglücken,Ptz2,True,mißglückt_MADJ,mißglückt,VVPP,missglücken,VVFIN,,manuell,2.0,36.162766


In [724]:
# Partizip1 
mask1 =  wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 = wiktionary_merkmal.merkmal == 'Ptz1'
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 13635) 

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag
grid(wiktionary_merkmal,mask)

check_mask:  13635 rows
13635 rows out of 2566096


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
948806,bedeuten,Ptz1,True,bedeutend_1,bedeutend,VVFIN,bedeuten,VVFIN,,,1.0,0.059268
948807,belasten,Ptz1,True,belastend_1,belastend,VVFIN,belasten,VVFIN,,,1.0,0.070373
948811,nachtragen,Ptz1,True,nachtragend_1,nachtragend,VVFIN,nachtragen,VVFIN,,,1.0,0.062625
948813,rezidivieren,Ptz1,True,rezidivierend_1,rezidivierend,VVFIN,rezidivieren,VVFIN,,,1.0,0.064949
948816,retardieren,Ptz1,True,retardierend_1,retardierend,VVFIN,retardieren,VVFIN,,,1.0,0.075280
...,...,...,...,...,...,...,...,...,...,...,...,...
2023558,hinauswollen_M,Ptz1,True,hinauswollend_MADJ,hinauswollend,VVFIN,hinauswollen,VVFIN,,manuell,1.0,2.239461
2023559,haaren,Ptz1,True,haarend,haarend,VVFIN,haaren,VVFIN,,manuell,2.0,23.534218
2023560,abwerten,Ptz1,True,abwertend,abwertend,VVFIN,abwerten,VVFIN,,manuell,2.0,95.695267
2023561,anstecken,Ptz1,True,ansteckend,ansteckend,VVFIN,anstecken,VVFIN,,manuell,2.0,1585.667480


In [725]:
# (1) Gerundivum: Schon fertig
mask = wiktionary_merkmal.merkmal == 'Gerundivum'
grid(wiktionary_merkmal,mask)

16480 rows out of 2566096


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
2399758,aufwärmen,Gerundivum,True,aufzuwärmende,aufzuwärmende,ADJA,aufwärmen,VVFIN,,,1.0,1.813820
2399759,aufwärmen,Gerundivum,True,aufzuwärmendes,aufzuwärmendes,ADJA,aufwärmen,VVFIN,,,2.0,1.133051
2399760,aufwärmen,Gerundivum,True,aufzuwärmender,aufzuwärmender,ADJA,aufwärmen,VVFIN,,,3.0,1.254846
2399761,aufwärmen,Gerundivum,True,aufzuwärmenden,aufzuwärmenden,ADJA,aufwärmen,VVFIN,,,4.0,2.143308
2399762,aufwärmen,Gerundivum,True,aufzuwärmendem,aufzuwärmendem,ADJA,aufwärmen,VVFIN,,,5.0,0.804719
...,...,...,...,...,...,...,...,...,...,...,...,...
2416233,entgegenwerfen,Gerundivum,True,entgegenzuwerfende,entgegenzuwerfende,ADJA,entgegenwerfen,VVFIN,,,1.0,1.839462
2416234,entgegenwerfen,Gerundivum,True,entgegenzuwerfendem,entgegenzuwerfendem,ADJA,entgegenwerfen,VVFIN,,,2.0,0.846041
2416235,entgegenwerfen,Gerundivum,True,entgegenzuwerfenden,entgegenzuwerfenden,ADJA,entgegenwerfen,VVFIN,,,3.0,2.233051
2416236,entgegenwerfen,Gerundivum,True,entgegenzuwerfender,entgegenzuwerfender,ADJA,entgegenwerfen,VVFIN,,,4.0,1.299718


In [726]:
# Adjektivformen der Verben: ADJA ergänzen 
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.merkmal.isin(['Ptz1','Ptz2'])
mask3 = ~wiktionary_merkmal.data_tag.str.contains('ADJA')
mask  = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal, mask, 27849)

adjaformen = wiktionary_merkmal[mask].copy()
adjaformen['data_tag'] = 'ADJA'
adjaformen['merkmal_order'] += 10

wiktionary_merkmal = pak.add_rows(wiktionary_merkmal,adjaformen)
grid(adjaformen)

check_mask:  27848 rows
27848 rows added, now a total of 2593944
27848 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
948801,dezidieren,Ptz2,True,dezidiert_2,dezidiert,ADJA,dezidieren,VVFIN,,,11.0,0.057074
948802,dedizieren,Ptz2,True,dediziert_1,dediziert,ADJA,dedizieren,VVFIN,,,11.0,0.056685
948803,verrücken,Ptz2,True,verrückt_2,verrückt,ADJA,verrücken,VVFIN,,,11.0,0.041750
948804,designieren,Ptz2,True,designiert_1,designiert,ADJA,designieren,VVFIN,,,11.0,0.058751
948805,verfluchen,Ptz2,True,verflucht_2,verflucht,ADJA,verfluchen,VVFIN,,,11.0,0.039684
...,...,...,...,...,...,...,...,...,...,...,...,...
2023593,wissen,Ptz2,True,gewußt_MADJ,gewußt,ADJA,wissen,VVFIN,,manuell,12.0,235.562454
2023594,misslingen,Ptz2,True,mißlungen_MADJ,mißlungen,ADJA,misslingen,VVFIN,,manuell,12.0,61.479610
2023595,missverstehen,Ptz2,True,mißverstanden_MADJ,mißverstanden,ADJA,missverstehen,VVFIN,,manuell,12.0,51.494343
2023596,missglücken,Ptz2,True,mißglückt_MADJ,mißglückt,ADJA,missglücken,VVFIN,,manuell,12.0,36.162766


In [727]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [728]:
# Übersicht: Partizipien und Gerundivum
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.merkmal.isin(['Ptz1','Ptz2','Gerundivum'])
mask  = mask1  &  mask2
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag','data'] )
a

,merkmal,merkmal_count,merkmal_percent,data_tag,data_tag_count,data,data_count
0,Ptz2,28426,39.4,"[ADJA, VVPP, VMPP, VAPP]","[14213, 14199, 8, 6]","[gestrippt, geschert, erschrocken, gewahrt, gekeilt, gelost, genickt, eingegeben, gekollert, gereiht, geschnoben, gedockt, gegichtet, geschlackt, geramscht, getüdert, verwunden, gedacht, geraten, gehört]","[12, 8, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4]"
1,Ptz1,27270,37.8,"[ADJA, VVFIN, VMFIN, VAFIN]","[13635, 13624, 7, 4]","[bedeutend, belastend, nachtragend, rezidivierend, retardierend, reizend, alternierend, quälend, gravierend, veraltend, irisierend, abwertend, dringend, anstrengend, überbordend, wütend, vermögend, oszillierend, beißend, heranwachsend]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]"
2,Gerundivum,16480,22.8,ADJA,16480,"[aufzuwärmende, aufzuwärmendes, aufzuwärmender, aufzuwärmenden, aufzuwärmendem, abzugießende, abzugießendes, abzugießender, abzugießenden, abzugießendem, nasszuschwitzende, nasszuschwitzendes, nasszuschwitzender, nasszuschwitzenden, nasszuschwitzendem, entzweizureißende, entzweizureißendes, entzweizureißender, entzweizureißenden, entzweizureißendem]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [729]:
# Alle wiktionary_merkmal.data die aussehen wie Erweiterte Infinitive sind merkmal VVIZU 
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 =  wiktionary_merkmal.merkmal != 'lemma'
mask4 =  wiktionary_merkmal.data.str.replace('zu','') == wiktionary_merkmal.lemma.str.replace('zu','') # erst wenn alle ZU entfernt werden ist es wieder gleich
mask5 =  wiktionary_merkmal.data.str.len() == wiktionary_merkmal.lemma.str.len() + 2
mask6 = ~wiktionary_merkmal.data.str.startswith('zu')
mask7 =  wiktionary_merkmal.merkmal != 'VVIZU'
mask  =  mask1  &  mask2  &  mask3  &  mask4  &  mask5  &  mask6  &  mask7
check_mask(wiktionary_merkmal,mask, 0)
#grid(wiktionary_merkmal,mask)

check_mask:  0 rows


In [730]:
# Alle merkmal VVIZU sehen aus wie Erweiterte Infinitive
mask1  = wiktionary_merkmal.merkmal == 'VVIZU'
mask2N =  wiktionary_merkmal.data.str.replace('zu','') == wiktionary_merkmal.lemma.str.replace('zu','') # erst wenn alle ZU entfernt werden ist es wieder gleich
mask3N =  wiktionary_merkmal.data.str.len() == wiktionary_merkmal.lemma.str.len() + 2
mask  =  mask1  &  ~(mask2N  &  mask3N)  
check_mask(wiktionary_merkmal,mask, 0)
#grid(wiktionary_merkmal,mask)

check_mask:  0 rows


In [731]:
# translate_lex suche_debug
#grid(search_str(translate_lex, suche_debug))

In [732]:
# TODO_VVIZU
# VVIZU in translate_lex
mask1 = translate_lex.lemma_tag.str.startswith('V')
mask2 = translate_lex.data.str.replace('zu','') == translate_lex.lemma.str.replace('zu','') # erst wenn alle ZU entfernt werden ist es wieder gleich
mask3 = translate_lex.data.str.len() == translate_lex.lemma.str.len() + 2
mask  =  mask1  &  mask2  &  mask3  
check_mask(translate_lex,mask, 5715)

vvizuformen = translate_lex[mask].copy()

check_mask:  5715 rows


In [733]:
vvizuformen

,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
4659,aufzustocken,aufzustocken,,0.538190,wiki Erweiterter_Infinitiv Verb,aufstocken,aufstocken,VVFIN,V,536.487732,aufstocken
7460,auszusprechen,auszusprechen,,0.478273,wiki Erweiterter_Infinitiv Verb,aussprechen,aussprechen,VVFIN,V,418.778046,aussprechen
7461,aufzukleben,aufzukleben,,0.461744,wiki Erweiterter_Infinitiv Verb,aufkleben,aufkleben,VVFIN,V,101.949570,aufkleben
7462,aufzukommen,aufzukommen,,0.544388,wiki Erweiterter_Infinitiv Verb,aufkommen,aufkommen,VVFIN,V,422.478607,aufkommen
7463,aufzukreuzen,aufzukreuzen,,0.486537,wiki Erweiterter_Infinitiv Verb,aufkreuzen,aufkreuzen,VVFIN,V,160.325668,aufkreuzen
...,...,...,...,...,...,...,...,...,...,...,...
667021,vollzusaugen,vollzusaugen,,0.439017,wiki Erweiterter_Infinitiv Verb,vollsaugen,vollsaugen,VVFIN,V,22.681517,vollsaugen
667062,entgegenzuwerfen,entgegenzuwerfen,,0.478273,wiki Erweiterter_Infinitiv Verb,entgegenwerfen,entgegenwerfen,VVFIN,V,475.184418,entgegenwerfen
667186,beieinanderzuhocken,beieinanderzuhocken,,0.507198,wiki Erweiterter_Infinitiv Verb,beieinanderhocken,beieinanderhocken,VVFIN,V,315.757355,beieinanderhocken
667234,einzusargen,einzusargen,,0.432818,wiki Erweiterter_Infinitiv Verb,einsargen,einsargen,VVFIN,V,86.188210,einsargen


In [734]:
# vvizuformen anpassen 
vvizuformen = pak.drop_cols(vvizuformen,['lemma_tagZZ','lemma_lower','lemma_score','data_tagZZ'])
vvizuformen = pak.rename_col(vvizuformen,'member',    'meta'     )
#vvizuformen = pak.rename_col(vvizuformen,'data_score', 'score'     )
vvizuformen['merkmal']       = 'VVIZU'
vvizuformen['is_lex']        = True
vvizuformen['data_tag']      = 'VVIZU'
vvizuformen['num']           = ''
vvizuformen['merkmal_order'] = 0

# Spaltenvergleich
bpy.minivenn( vvizuformen.columns, wiktionary_merkmal.columns, format='print2')

left_only:  {}
right_only: {}



In [735]:
#mask = wiktionary_merkmal.merkmal == 'VVIZU'
#wiktionary_merkmal[mask]

In [736]:
# vvizuformen an wiktionary_merkmal anfügen
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, vvizuformen, only_new=['lemma_id','lemma','data_id','data'])

9 rows not attached
5706 rows added, now a total of 2599650


In [737]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


In [738]:
# Kontrolle: Keine anderen Infinitiv-like
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3 =  wiktionary_merkmal.data == wiktionary_merkmal.lemma
mask4 = ~wiktionary_merkmal.merkmal.isin( ['lemma','Ptz2'] )
mask  = mask1  &  mask2  &  mask3  &  mask4

check_mask( wiktionary_merkmal, mask, 0)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='Infinitiv-like' ) 
#grid(wiktionary_merkmal,mask)

check_mask:  0 rows


In [739]:
# Kontrolle: Es gibt keine Doppeltags in lemma_tag und data_tag
mask1 =  wiktionary_merkmal.is_lex    == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('V')
mask3A = wiktionary_merkmal.lemma_tag.str.contains(' ')
mask3B = wiktionary_merkmal.data_tag.str.contains(' ')
mask  =  mask1  &  mask2  &  (mask3A | mask3B)
check_mask( wiktionary_merkmal, mask, 0)

check_mask:  0 rows


In [740]:
# data_tag setzen (auf lemma_tag)
mask1 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask2 = wiktionary_merkmal.is_lex    == True
mask3 = wiktionary_merkmal.data_tag  == ''
mask  = mask1  &  mask2  &  mask3

#grid(wiktionary_merkmal,mask)
wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag

In [741]:
# Übersicht: Alle V-Tags in wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask = mask1  &  mask2
a = pak.analyse_freqs(wiktionary_merkmal[mask],['data_tag','lemma_tag','data','lemma'])
a = pak.drop_cols(a,['data_tag_percent','data_count','lemma_count'])
grid(a)

13 rows


,data_tag,data_tag_count,lemma_tag,lemma_tag_count,data,lemma
0,VVFIN,84686,VVFIN,84686,"[strippte, scherte, lost, strippst, strippt, strippe, schlackte, keilte, tüderte, ramschte, nickte, loste, kollerte, gichtete, dockte, scherst, schert, schere, reihte, misst]","[strippen, scheren, erschrecken, anbellen, löschen, überhängen, nicken, reihen, losen, durchbrechen, eingeben, kollern, keilen, unterschieben, durchkämmen, umhängen, docken, gichten, ramschen, tüdern]"
1,ADJA,44328,"[VVFIN, VMFIN, VAFIN]","[44303, 15, 10]","[gestrippt, geschert, erschrocken, gewahrt, gekeilt, gelost, genickt, eingegeben, gekollert, gereiht, geschnoben, gedockt, gegichtet, geschlackt, geramscht, getüdert, bedeutend, belastend, nachtragend, rezidivierend]","[überhängen, umfassen, umhängen, durchschlagen, durchkämmen, durchbrechen, anreihen, zusammenhängen, durchwühlen, durchpflügen, umspannen, umbauen, überstreichen, anhängen, untergraben, überwerfen, unterschieben, durchjagen, durchnagen, überholen]"
2,VVIMP,37457,"[VVFIN, VMFIN]","[37454, 3]","[strippe, strippt, schere, schert, reih, lose, gib ein, nick, gichte, tüder, ramsch, koller, dock, keil, docke, keile, ramsche, tüdere, nicke, lost]","[strippen, scheren, umhängen, nicken, tüdern, ramschen, docken, durchkämmen, unterschieben, keilen, anbellen, anwandeln, sporteln, umspannen, durchschnüffeln, murmeln, erschrecken, aufschrecken, einwiegen, löschen]"
3,VVPP,14199,VVFIN,14199,"[gestrippt, geschert, erschrocken, gewahrt, gekeilt, gelost, genickt, eingegeben, gekollert, gereiht, geschnoben, gedockt, gegichtet, geschlackt, geramscht, getüdert, verwunden, gedacht, geraten, gehört]","[überhängen, scheren, strippen, erschrecken, reihen, umhängen, löschen, bescheren, umfassen, backen, durchbrechen, heißen, bleichen, durchkämmen, anreihen, raten, untersetzen, schaffen, wiederholen, umwandeln]"
4,VVINF,13853,VVFIN,13853,"[strippen, scheren, überhängen, erschrecken, docken, durchbrechen, eingeben, gichten, keilen, kollern, losen, löschen, nicken, ramschen, reihen, schlacken, tüdern, umhängen, durchfahren, durchlaufen]","[strippen, scheren, überhängen, erschrecken, docken, durchbrechen, eingeben, gichten, keilen, kollern, losen, löschen, nicken, ramschen, reihen, schlacken, tüdern, umhängen, durchfahren, durchlaufen]"
5,VVIZU,5714,VVFIN,5714,"[auszupowern, einzureden, loszuwerden, herumzupimpeln, hereinzulassen, herauszulassen, aufzurütteln, anzustaunen, anzugucken, aufzustocken, auszusprechen, aufzukleben, aufzukommen, aufzukreuzen, aufzuladen, aufzulassen, aufzulegen, abzudanken, abzudecken, abzudichten]","[auspowern, einreden, loswerden, herumpimpeln, hereinlassen, herauslassen, aufrütteln, anstaunen, angucken, aufstocken, aussprechen, aufkleben, aufkommen, aufkreuzen, aufladen, auflassen, auflegen, abdanken, abdecken, abdichten]"
6,VMFIN,51,VMFIN,51,"[darf, will, kann, soll, mag, muss, wollte, brauchte, sollte, könnend, wollend, brauchend, müssend, mögend, dürfend, sollend, wills, brauchs, solls, darfs]","[brauchen, müssen, können, wollen, mögen, dürfen, sollen]"
7,VAFIN,41,VAFIN,41,"[tust, tut, tu, tue, tat, täte, seyn, syn, frühneuhochdeutsch, ham, thun, seiend, habend, werdend, tuend, werds, tus, bins, habs, wirst]","[tun, sein, haben, werden]"
8,VAIMP,14,VAFIN,14,"[tu, tue, tut, werde, sei, habe, werd, hab, werdet, seid, habt]","[tun, werden, haben, sein]"
9,VMPP,8,VMFIN,8,"[gewollt, gebraucht, gesollt, gemocht, gekonnt, gemusst, gedurft, wollen]","[wollen, brauchen, sollen, mögen, können, müssen, dürfen]"


#### Nomen

In [742]:
# Übersicht über alle explizit gesetzten tags (Tagquelle: lexeme_manuell)
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('N')
mask3 = wiktionary_merkmal.data_tag != ''
mask  = mask1  &  mask2  &  mask3
#grid(wiktionary_merkmal,mask)
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['lemma_tag','data_tag'])
(a)

,lemma_tag,lemma_tag_count,lemma_tag_percent,data_tag,data_tag_count
0,NN,96710,88.9,NN,96710
1,NE,12118,11.1,NE,12118


In [743]:
# Keine Doppeltags
mask1 =  wiktionary_merkmal.is_lex    == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('N')
mask3 =  wiktionary_merkmal.lemma_tag.str.contains(' ')
mask  =  mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0)

check_mask:  0 rows


In [744]:
# Irgendwelche unbekannten tags?
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.lemma_tag.str.startswith('N')
mask3 = ~wiktionary_merkmal.data_tag.isin( ['NN','NE',''] )
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 0)
#grid(wiktionary_merkmal,mask)

check_mask:  0 rows


In [745]:
# data_tag setzen (auf lemma_tag)
mask1 = wiktionary_merkmal.lemma_tag.str.startswith('N')
mask2 = wiktionary_merkmal.is_lex    == True
mask3 = wiktionary_merkmal.data_tag  == ''
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 761247)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag
#grid(wiktionary_merkmal,mask)

check_mask:  807764 rows


In [746]:
# Nomen analysieren
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.lemma_tag.str.startswith('N')
mask  = mask1  &  mask2  
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag'] )
grid(a)

27 rows


,merkmal,merkmal_count,merkmal_percent,data_tag,data_tag_count
0,Gen Sg,119275,13.0,"[NN, NE]","[112557, 6718]"
1,Dat Sg,112400,12.3,"[NN, NE]","[109538, 2862]"
2,lemma,108828,11.9,"[NN, NE]","[96710, 12118]"
3,Nom Sg,99640,10.9,"[NN, NE]","[92935, 6705]"
4,Akk Sg,95642,10.4,"[NN, NE]","[92962, 2680]"
5,Nom Pl,86255,9.4,"[NN, NE]","[84231, 2024]"
6,Akk Pl,86246,9.4,"[NN, NE]","[84226, 2020]"
7,Gen Pl,86246,9.4,"[NN, NE]","[84223, 2023]"
8,Dat Pl,85481,9.3,"[NN, NE]","[83502, 1979]"
9,alt_2,8518,0.9,"[NN, NE]","[7398, 1120]"


#### Rest

In [747]:
# data_tag setzen (auf lemma_tag)
mask1 = wiktionary_merkmal.is_lex    == True
mask2 = wiktionary_merkmal.data_tag  == ''
mask3 = ~wiktionary_merkmal.lemma_tag.str[0].isin(['V','N'])
mask  = mask1  &  mask2  &  mask3
check_mask( wiktionary_merkmal, mask, 31447)

wiktionary_merkmal.loc[mask,'data_tag'] = wiktionary_merkmal[mask].lemma_tag
#a = pak.analyse_freqs( wiktionary_merkmal[mask], ['lemma_tag','data_tag'] )
#grid(a)

check_mask:  31420 rows


#### zweittag

In [748]:
spalten = ['lemma_id','tag_1']
mask = wiktionary_lemma.tag.str.contains(' ')
check_mask( wiktionary_lemma, mask, 179)

zweitags = wiktionary_lemma[mask][spalten]

check_mask:  175 rows


In [749]:
# Merkmale anfügen mit zweittag
mask1 = wiktionary_merkmal.lemma_id.isin(zweitags.lemma_id)
mask2 = wiktionary_merkmal.lemma_tag == wiktionary_merkmal.data_tag
mask = mask1  &  mask2
check_mask( wiktionary_merkmal, mask, 606)

zweitlexeme = wiktionary_merkmal[mask].copy()
zweitlexeme = pak.update_col(zweitlexeme, zweitags, on='lemma_id', col='tag_1', col_rename='data_tag')
zweitlexeme.meta = (zweitlexeme.meta + ' zweittag').str.strip()
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal,zweitlexeme)
grid(zweitlexeme)

check_mask:  619 rows
update_col: 619 cells written into existing column
619 rows added, now a total of 2600269
619 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
88989,viel,alt_2,True,Viel,Viel,PIAT,viel,ADJA,,zweittag,1.0,430.210968
148687,dich,alt_2,True,Dich,Dich,PRF,dich,PPER,,zweittag,1.0,13.250154
168868,aberhundert,alt_1,True,aberhundert~aberhunderte,aberhunderte,CARD,aberhundert,ADJA,,zweittag,1.0,28.599972
168869,aberhundert,alt_2,True,aberhundert~Aberhundert,Aberhundert,CARD,aberhundert,ADJA,,zweittag,1.0,28.599972
169695,beide,alt,True,beide~beyde,beyde,CARD,beide,PIAT,,zweittag,1.0,100.831711
...,...,...,...,...,...,...,...,...,...,...,...,...
2564348,zweiundzwanzigste,lemma,True,zweiundzwanzigste,zweiundzwanzigste,CARD,zweiundzwanzigste,ADJA,,zweittag,0.0,175.004791
2564410,zwote,lemma,True,zwote,zwote,CARD,zwote,ADJA,,zweittag,0.0,102.335976
2564416,zwölffach,lemma,True,zwölffach,zwölffach,CARD,zwölffach,ADJA,,zweittag,0.0,66.035995
2564431,zwölfte,lemma,True,zwölfte,zwölfte,CARD,zwölfte,ADJA,,zweittag,0.0,401.481995


In [750]:
# Prüfen: Alle Lexeme haben eine eindeutige data_id
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


#### Adjektive

In [751]:
# Merkmal Positiv ist sinnlos
mask1 = wiktionary_merkmal.merkmal == 'Positiv'
mask2 = wiktionary_merkmal.data != wiktionary_merkmal.lemma
mask = mask1  &  mask2
check_mask( wiktionary_merkmal, mask, 0,10)
wiktionary_merkmal[mask]

check_mask:  1 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
2049199,Wiesbadner,Positiv,True,Wiesbadener,Wiesbadener,ADJA,Wiesbadner,ADJA,,,1.0,12.086063


In [752]:
# Merkmal Positiv löschen
mask = wiktionary_merkmal.merkmal == 'Positiv'
check_mask( wiktionary_merkmal, mask, 14036)

msg = 'Merkmal Positiv ist sinnlos' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  14036 rows
Move 14036 rows from 2600269
14036 rows out of 59120


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
2036466,meistgeklickt,Positiv,meistgeklickt,meistgeklickt,ADJA,ADJA,,,10.719136,True,Merkmal Positiv ist sinnlos,1.0,meistgeklickt
2036467,vitaminarm,Positiv,vitaminarm,vitaminarm,ADJA,ADJA,,,17.558193,True,Merkmal Positiv ist sinnlos,1.0,vitaminarm
2036468,thalassogen,Positiv,thalassogen,thalassogen,ADJA,ADJA,,,33.211369,True,Merkmal Positiv ist sinnlos,1.0,thalassogen
2036469,billiontel,Positiv,billiontel,billiontel,ADJA,ADJA,,,11.067892,True,Merkmal Positiv ist sinnlos,1.0,billiontel
2036470,entere,Positiv,entere,entere,ADJA,ADJA,,,34.663174,True,Merkmal Positiv ist sinnlos,1.0,entere
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600073,zweifach,Positiv,zweifach,zweifach,CARD,ADJA,,zweittag,33.335075,True,Merkmal Positiv ist sinnlos,1.0,zweifach
2600074,probeweise,Positiv,probeweise,probeweise,ADJA,ADV,,zweittag,21.527624,True,Merkmal Positiv ist sinnlos,1.0,probeweise
2600075,dritte,Positiv,dritte,dritte,CARD,ADJA,,zweittag,135.270599,True,Merkmal Positiv ist sinnlos,1.0,dritte
2600076,zweite,Positiv,zweite,zweite,CARD,ADJA,,zweittag,188.358322,True,Merkmal Positiv ist sinnlos,1.0,zweite


In [753]:
# Ist kein Adjektiv
mask1 =  wiktionary_merkmal.data_tag.isin(['ADJA','ADJD'])
mask2 = ~wiktionary_merkmal.merkmal.isin(['lemma','Ptz1','Ptz2','Gerundivum','Positiv','Komparativ','Superlativ','alt','alt_1','alt_2','lexAlt','lex','alt_abk','klein','weib','männ'])
mask3 =  wiktionary_merkmal.is_lex == True
mask  = mask1  &  mask2  &  mask3   
check_mask( wiktionary_merkmal, mask, 20)

msg = 'Ist kein Adjektiv' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  16 rows
Move 16 rows from 2586233
16 rows out of 59136


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
2599745,sämtlich,Nom Sg f,sämtliche,sämtliche,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2599759,sämtlich,Nom Sg m,sämtlicher,sämtlicher,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2599768,sämtlich,Nom Sg n,sämtliches,sämtliches,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2599785,sämtlich,Gen Sg f,sämtlicher,sämtlicher,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2599805,sämtlich,Gen Sg m,sämtliches,sämtliches,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2599808,sämtlich,Gen Sg n,sämtliches,sämtliches,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2599829,sämtlich,Dat Sg f,sämtlicher,sämtlicher,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2599843,sämtlich,Dat Sg m,sämtlichem,sämtlichem,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2599850,sämtlich,Dat Sg n,sämtlichem,sämtlichem,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich
2599874,sämtlich,Akk Sg f,sämtliche,sämtliche,ADJA,PIAT,,zweittag,85.837029,True,Ist kein Adjektiv,1.0,sämtlich


In [754]:
# Adjektiv-only Merkmale
mask1 = ~wiktionary_merkmal.data_tag.isin(['ADJA','ADJD','ADV','PIAT'])
mask2 =  wiktionary_merkmal.merkmal.isin(['Gerundivum','Komparativ','Superlativ'])
mask3 =  wiktionary_merkmal.is_lex == True
mask  = mask1  &  mask2  &  mask3   
check_mask( wiktionary_merkmal, mask, 0)

#a = wiktionary_merkmal[mask].sort_values(['data_tag','lemma_id'])
#grid(a)

msg = 'Adjektiv-only Merkmale' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  0 rows
No rows moved
No rows, mask filters them all away


In [755]:
# Anschauen: Steigerbare Adverbien (TODO_Steigerbare Adverbien)
mask1 =  wiktionary_merkmal.data_tag == 'ADV'
mask2 =  wiktionary_merkmal.merkmal.isin(['Komparativ','Superlativ'])
mask3 =  wiktionary_merkmal.is_lex == True
mask4 = ~wiktionary_merkmal.lemma.isin(['sehr','gern','gerne','oft', 'bald','wohl'])
    
mask  = mask1  &  mask2  &  mask3  &  mask4 
check_mask( wiktionary_merkmal, mask, )

a = wiktionary_merkmal[mask].sort_values(['merkmal','data_tag','lemma_id'])
grid(a)

# 'sehr', 'mehr','am meisten'
# 'gern', 'lieber','am liebsten'
# 'bald', 'eher','am ehesten'
## 'oft',  'öfter','am öftesten'  # drin
## 'wohl', 'besser','am besten'   # drin

check_mask:  16 rows
16 rows


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
2054948,gewiss,Komparativ,True,gewisser,gewisser,ADV,gewiss,ADV,,,1.0,109.957718
2055400,hoch_1,Komparativ,True,höher,höher,ADV,hoch,ADV,,,1.0,4.868667
2052974,inwendig,Komparativ,True,inwendiger,inwendiger,ADV,inwendig,ADV,,,1.0,24.664410
2053399,lange,Komparativ,True,länger,länger,ADV,lange,ADV,,,1.0,45.577663
2053067,schal,Komparativ,True,schaler,schaler,ADV,schal,ADV,,,1.0,41.949757
2055327,unumstößlich,Komparativ,True,unumstößlicher,unumstößlicher,ADV,unumstößlich,ADV,,,1.0,35.523693
2053545,wahrlich,Komparativ,True,wahrlicher,wahrlicher,ADV,wahrlich,ADV,,,1.0,103.879677
2053695,äbich,Komparativ,True,äbichor,äbichor,ADV,äbich,ADV,,,1.0,37.883972
2056829,gewiss,Superlativ,True,gewissesten,gewissesten,ADV,gewiss,ADV,,,1.0,109.957718
2060392,hoch_1,Superlativ,True,höchsten,höchsten,ADV,hoch,ADV,,,1.0,4.868667


In [756]:
# Die gleichen lemmas als Adjektive
mask1 = wiktionary_merkmal.lemma.isin(a.lemma)
mask2 = wiktionary_merkmal.lemma_tag == 'ADJA'
mask3 =  wiktionary_merkmal.merkmal.isin(['Komparativ','Superlativ'])
mask = mask1  &  mask2  &  mask3
wiktionary_merkmal[mask]

,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
2055332,hoch,Komparativ,True,höher,höher,ADJA,hoch,ADJA,,,1.0,1241.171753
2060368,hoch,Superlativ,True,höchsten,höchsten,ADJA,hoch,ADJA,,,1.0,1241.171753


#### analysieren

In [757]:
# Merkmale analysieren
mask1 = wiktionary_merkmal.is_lex == True
mask  = mask1   
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag'] )
grid(a)

53 rows


,merkmal,merkmal_count,merkmal_percent,data_tag,data_tag_count
0,lemma,168181,14.2,"[NN, ADJA, VVINF, NE, ADV, KOMBI, CARD, ITJ, APPR, FM, PIAT, PTKANT, KOUS, KON, DIV, PDAT, PPER, ART, $(, APPO]","[96710, 41958, 13853, 12121, 1599, 449, 419, 297, 182, 103, 71, 67, 63, 53, 38, 36, 32, 16, 15, 13]"
1,Gen Sg,119298,10.1,"[NN, NE, PIAT, PWAT, PRF, PRELAT, PTKANT, PRO]","[112557, 6723, 7, 3, 3, 2, 2, 1]"
2,Dat Sg,112417,9.5,"[NN, NE, PIAT, PTKANT, PRF, PRELAT, PWAT]","[109538, 2867, 7, 2, 1, 1, 1]"
3,Nom Sg,99655,8.4,"[NN, NE, PIAT, PWAT, PTKANT, PRELAT, PRO]","[92935, 6709, 5, 2, 2, 1, 1]"
4,Akk Sg,95660,8.1,"[NN, NE, PIAT, PWAT, PTKANT, PRELAT, PRF, PRO]","[92962, 2684, 7, 2, 2, 1, 1, 1]"
5,Nom Pl,86322,7.3,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PWAT, PPER, PRELS, PTKANT, PRO, PDS]","[84231, 2025, 19, 15, 10, 6, 4, 3, 2, 2, 2, 1, 1, 1]"
6,Gen Pl,86316,7.3,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, PRF, CARD, PWAT, PRELS, PPER, PTKANT, PRO, PDS]","[84223, 2024, 19, 15, 10, 6, 4, 3, 3, 2, 2, 2, 1, 1, 1]"
7,Akk Pl,86314,7.3,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PWAT, PRELS, PPER, PRF, PDS, PRO, PTKANT]","[84226, 2021, 19, 15, 10, 6, 4, 3, 2, 2, 2, 1, 1, 1, 1]"
8,Dat Pl,85552,7.2,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PWAT, PRELS, PPER, PRF, PTKANT, PDS, PRO]","[83502, 1980, 20, 16, 11, 6, 4, 3, 2, 2, 2, 1, 1, 1, 1]"
9,Ptz2,28426,2.4,"[ADJA, VVPP, VMPP, VAPP]","[14213, 14199, 8, 6]"


In [758]:
# Keine Verwaisten lemma_id
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask( wiktionary_merkmal, mask, 0)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='verwaist' ) 
#grid( wiktionary_merkmal, mask)

check_mask:  0 rows


### Felder -------

#### data

In [759]:
# Prüfen: Keine mehrfache Spaces
mask = wiktionary_merkmal.data.str.contains('  ')
check_mask(wiktionary_merkmal,mask,0)
wiktionary_merkmal['data'] = pak.replace_str( wiktionary_merkmal.data, {'  ':' '})

check_mask:  0 rows


In [760]:
# Prüfen: nicht leer
mask = wiktionary_merkmal.data.str.len() == 0
check_mask( wiktionary_merkmal, mask, 0)
#wiktionary_merkmal, wiktionary_merkmal_trash = pak.drop_rows( wiktionary_merkmal, mask, wiktionary_merkmal_trash, msg='leer3' ) 

check_mask:  0 rows


In [761]:
# '⇀' ist nur in syllables
mask1 =  wiktionary_merkmal.data.str.contains('⇀')
mask2 = ~wiktionary_merkmal.merkmal.isin(['syllables'])
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [762]:
# Apostroph anschauen
mask =  wiktionary_merkmal.data.str.contains("'")
check_mask(wiktionary_merkmal,mask,32)
grid(wiktionary_merkmal,mask)

check_mask:  32 rows
32 rows out of 2586217


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
48524,Maat_1,alt_1,True,Maat_1~Ma'at,Ma'at,NE,Maat,NE,,,1.0,2.849264
93974,ein,alt_abk,True,ein~'nen,'nen,ART,ein,ART,,,1.0,80.404617
204159,Scharia,alt_2,True,Scharia~Schari'a,Schari'a,NN,Scharia,NN,,,1.0,61.271812
495871,Micmac,alt_2,True,Micmac~Mi'kmaq,Mi'kmaq,NN,Micmac,NN,,,1.0,35.002872
553379,ausm,alt_2,True,ausm~aus'm,aus'm,DIV,ausm,DIV,,,1.0,9.454154
560630,Cabernet,down,False,Cabernet_d'Anjou,Cabernet d'Anjou,,Cabernet,NN,2,,6.0,47.284927
584537,Faradaykäfig,alt_2,True,Faradaykäfig~Faraday'sche Käfig,Faraday'sche Käfig,NN,Faradaykäfig,NN,,,2.0,76.690163
668560,Fraunhoferlinie,alt_1,True,Fraunhoferlinie~Fraunhofer'sche Linie,Fraunhofer'sche Linie,NN,Fraunhoferlinie,NN,,,1.0,42.633888
1208723,Cabernet_d'Anjou,Nom Sg,True,Cabernet_d'Anjou,Cabernet d'Anjou,NN,Cabernet d'Anjou,NN,,,1.0,12.235624
1249676,Titus,Gen Sg,True,Titus~Titus',Titus',NE,Titus,NE,,,1.0,70.384621


In [763]:
# Belöschen: Komma_in_data
mask1 =  wiktionary_merkmal.data.str.contains(",")
mask2 = ~wiktionary_merkmal.lemma.str.contains(",")
mask3 =  wiktionary_merkmal.data.str.endswith(('säure','ol','in','an'))
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal,mask,16)
#grid(wiktionary_merkmal,mask)

msg = 'Komma_in_data' 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg) 
grid( wiktionary_merkmal_trash, msg )

check_mask:  16 rows
Move 16 rows from 2586217
16 rows out of 59152


,lemma_id,merkmal,data_id,data,data_tag,lemma_tag,num,meta,data_score,is_lex,msg,merkmal_order,lemma
165730,Abietinsäure,syn,"Abieta-7,14-dien-19-carbonsäure","Abieta-7,14-dien-19-carbonsäure",,NN,1,,55.998070,False,Komma_in_data,1.0,Abietinsäure
217823,moin,alt_1,"moin~Moin, Moin","Moin, Moin",PTKANT,PTKANT,,,58.166126,True,Komma_in_data,2.0,moin
232398,Aconitsäure,syn,"1,2,3-Propentricarbonsäure","1,2,3-Propentricarbonsäure",,NN,1,,44.564732,False,Komma_in_data,1.0,Aconitsäure
299601,Tetramethylenglycol,def,"1,4-Butandiol","1,4-Butandiol",,NN,1,Chemie,12.677977,False,Komma_in_data,1.0,Tetramethylenglycol
315394,Weinsäure,syn,"2,3-Dihydroxybutandisäure","2,3-Dihydroxybutandisäure",,NN,1,,55.846737,False,Komma_in_data,1.0,Weinsäure
315396,Weinsäure,syn,"2,3-Dihydroxybernsteinsäure","2,3-Dihydroxybernsteinsäure",,NN,1,,55.846737,False,Komma_in_data,3.0,Weinsäure
315405,Terephthalsäure,syn,"1,4-Benzoldicarbonsäure","1,4-Benzoldicarbonsäure",,NN,1,,35.536873,False,Komma_in_data,1.0,Terephthalsäure
315449,Pikrinsäure,syn,"2,4,6-Trinitrophenol","2,4,6-Trinitrophenol",,NN,1,,48.590359,False,Komma_in_data,1.0,Pikrinsäure
315454,Phthalsäure,syn,"1,2-Benzoldicarbonsäure","1,2-Benzoldicarbonsäure",,NN,1,,45.471306,False,Komma_in_data,1.0,Phthalsäure
315514,Gluconsäure,syn,"2,3,4,5,6-Pentahydroxyhexansäure","2,3,4,5,6-Pentahydroxyhexansäure",,NN,1,,31.847654,False,Komma_in_data,1.0,Gluconsäure


In [764]:
# Restliche Kommas anschauen
mask1 =  wiktionary_merkmal.data.str.contains(",")
mask2 = ~wiktionary_merkmal.lemma.str.contains(",")
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,19)
grid(wiktionary_merkmal,mask)

check_mask:  19 rows
19 rows out of 2586201


,lemma_id,merkmal,is_lex,data_id,data,data_tag,lemma,lemma_tag,num,meta,merkmal_order,data_score
34627,Flasche,down,False,"0,2-Liter-Flasche","0,2-Liter-Flasche",,Flasche,NN,1,Fassungsvermögen,46.0,1516.371216
34628,Flasche,down,False,"0,33-Liter-Flasche","0,33-Liter-Flasche",,Flasche,NN,1,Fassungsvermögen,47.0,1516.371216
34629,Flasche,down,False,"0,5-Liter-Flasche","0,5-Liter-Flasche",,Flasche,NN,1,Fassungsvermögen,48.0,1516.371216
34630,Flasche,down,False,"0,7-Liter-Flasche","0,7-Liter-Flasche",,Flasche,NN,1,Fassungsvermögen,49.0,1516.371216
34631,Flasche,down,False,"0,75-Liter-Flasche","0,75-Liter-Flasche",,Flasche,NN,1,Fassungsvermögen,50.0,1516.371216
51328,paffen,familie,False,"piff,_paff","piff, paff",,paffen,VVFIN,,,1.0,71.216179
149329,Ziel,down,False,"1,5-Grad-Ziel","1,5-Grad-Ziel",,Ziel,NN,1,,1.0,609.851990
303933,ruckzuck,alt_2,True,"ruck,_zuck","ruck, zuck",ADV,ruckzuck,ADV,,,1.0,58.453896
402355,predigen,kombi,False,"landauf,_landab","landauf, landab",,predigen,VVFIN,2,figurativ,29.0,228.334396
415663,bundesweit,lateral,False,"landauf,_landab","landauf, landab",,bundesweit,ADJA,1,,2.0,115.302048


#### num

In [765]:
# num ist bei den meisten Lexemen sinnlos
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.num.str.len() > 0
#mask3 = wiktionary_merkmal.merkmal.str.startswith('alt') 
mask  =  mask1  &  mask2  #&  mask3

check_mask( wiktionary_merkmal, mask, 19930)
wiktionary_merkmal.loc[mask,'num'] = ''   
#grid(wiktionary_merkmal,mask, color='green')

check_mask:  19939 rows


In [766]:
#pak.analyse_freqs(wiktionary_merkmal[mask], ['num','data',])

#### meta

In [767]:
# remove_words
d = [ '(', ')', 'auch','nicht','bis','Adjektive','Adjektiv','Kombination','Substantive','Substantiv',
      'allgemein','Adverbien','Verb','Adverbien','Adverb','andere','über','Erweiterter_Infinitiv','wiki',
    ]       
wiktionary_merkmal['meta'] = pak.remove_words( wiktionary_merkmal.meta, d )

In [768]:
# replace_str
r = {       
    'seltener':'selten',
    'Schweiz':'schweizerisch',    
    'Österreich':'österreichisch', 
    'Süddeutschland':'süddeutsch',    
    'bairisch':'bayrisch',        
    'bairisch':'bayrisch',       
     
    }
wiktionary_merkmal['meta'] = pak.replace_str( wiktionary_merkmal.meta, r )

In [769]:
# leeren: meta == lemma 
mask = wiktionary_merkmal.lemma == wiktionary_merkmal.meta
check_mask(wiktionary_merkmal,mask,61000)

wiktionary_merkmal.loc[mask,'meta'] = ''
#wiktionary_merkmal[mask]
#pak.analyse_freqs(wiktionary_merkmal[mask],['merkmal'])

check_mask:  61033 rows


In [770]:
# meta ist typisch für ein einzelnes lemma
mask = wiktionary_merkmal.meta.str.len() > 1

a = pak.analyse_freqs(wiktionary_merkmal[mask],['meta','lemma'])
mask = a.lemma.str.isalpha().notnull() # meta ist typisch für ein einzelnes lemma
löschmich = list(a[mask].meta)

# meta belöschen
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal.loc[mask,'meta'] = pak.remove_words( wiktionary_merkmal[mask].meta, löschmich )

In [771]:
# meta Überblick Lexeme
mask = wiktionary_merkmal.is_lex == True
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['meta','merkmal','lemma_id'], splits=[' ', None])
a = pak.drop_cols(a,['lemma_id_count'])
gridt(a)

119 rows


In [772]:
# Seltene meta löschen
mask = a.meta_count < 20
löschmich = list(a[mask].meta)
#löschmich

# meta belöschen
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal.loc[mask,'meta'] = pak.remove_words( wiktionary_merkmal[mask].meta, löschmich )

In [773]:
# meta Überblick Nichtlexeme
mask = wiktionary_merkmal.is_lex == False
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['meta','merkmal','lemma_id'], splits=[' ', None])
a = pak.drop_cols(a,['lemma_id_count'])
gridt(a)

8031 rows


In [774]:
# Seltene meta löschen
mask = a.meta_count < 20
löschmich = list(a[mask].meta)
#löschmich

# meta belöschen
mask = wiktionary_merkmal.is_lex == False
wiktionary_merkmal.loc[mask,'meta'] = pak.remove_words( wiktionary_merkmal[mask].meta, löschmich )

#### tag-Felder

In [775]:
# data_tagZ
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='data_tag',  right_on='tag', col='tagZ', col_rename='data_tagZ')
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZ', col_rename='data_tagZ', cond='null')
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'data_tagZ','data_tag')

update_col: 1180261 cells written into new column
update_col: 1405940 cells written into existing column


In [776]:
# data_tagZZ
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='data_tag',  right_on='tag', col='tagZZ', col_rename='data_tagZZ')
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZZ', col_rename='data_tagZZ', cond='null')
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'data_tagZZ','data_tagZ')

update_col: 1180261 cells written into new column
update_col: 1405940 cells written into existing column


In [777]:
# Wieder löschen wenn kein Lexem
mask = wiktionary_merkmal.is_lex != True
wiktionary_merkmal.loc[mask,'data_tagZ']  = ''
wiktionary_merkmal.loc[mask,'data_tagZZ'] = ''

In [778]:
# data_tagZ und data_tagZZ überall gesetzt
mask1  = wiktionary_merkmal.is_lex == True
mask2A = wiktionary_merkmal.data_tagZ  == ''
mask2B = wiktionary_merkmal.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(wiktionary_merkmal,mask, 0)

check_mask:  0 rows


In [779]:
# alle verwendeten tags
mask = wiktionary_merkmal.is_lex == True
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['data_tag','merkmal'], splits=[' ', None] )
grid(a)

46 rows


,data_tag,data_tag_count,data_tag_percent,merkmal,merkmal_count
0,NN,868286,73.6,"[Gen Sg, Dat Sg, lemma, Akk Sg, Nom Sg, Nom Pl, Akk Pl, Gen Pl, Dat Pl, alt_2, weib, männ, alt_1, alt, klein, alt_abk, lexAlt, alt_3, alt_weib, alt_männ]","[112557, 109538, 96710, 92962, 92935, 84231, 84226, 84223, 83502, 7398, 6829, 5592, 3739, 2017, 1042, 352, 297, 56, 26, 25]"
1,ADJA,101601,8.6,"[lemma, Gerundivum, Ptz2, Ptz1, Superlativ, Komparativ, alt_2, alt_1, alt, lexAlt, lex, alt_abk, männ, klein, weib]","[41958, 16480, 14213, 13635, 5904, 5875, 1957, 1016, 499, 49, 6, 5, 2, 1, 1]"
2,VVFIN,84686,7.2,"[Präsens ich, Konj2 ich, Präteritum ich, Präsens ersiees, Präsens du, Ptz1, alt_2, alt_1, alt, lexAlt, alt_abk, lex, Konj1 ich]","[14423, 13983, 13909, 13792, 13627, 13624, 562, 448, 176, 84, 33, 24, 1]"
3,NE,48331,4.1,"[lemma, Gen Sg, Nom Sg, alt_3, Dat Sg, Akk Sg, Nom Pl, Gen Pl, Akk Pl, Dat Pl, alt_2, alt_klein, alt_abk, alt_männ, alt_weib, alt_1, klein, alt, Pl, Gen]","[12121, 6723, 6709, 5768, 2867, 2684, 2025, 2024, 2021, 1980, 1120, 661, 390, 349, 334, 259, 163, 75, 38, 6]"
4,VVIMP,37457,3.2,"[Imp Sg, Imp Pl]","[23924, 13533]"
5,VVPP,14199,1.2,Ptz2,14199
6,VVINF,13853,1.2,lemma,13853
7,VVIZU,5714,0.5,VVIZU,5714
8,ADV,2083,0.2,"[lemma, alt_1, alt_2, alt, Komparativ, Superlativ, lex, alt_abk, lexAlt]","[1599, 210, 208, 26, 14, 14, 6, 3, 3]"
9,CARD,823,0.1,"[lemma, lex, alt_2, alt_1, alt, Nom Sg f, Nom Sg m, Nom Sg n, Gen Sg f, Gen Sg m, Gen Sg n, Dat Sg f, Dat Sg m, Dat Sg n, Akk Sg f, Akk Sg m, Akk Sg n, Nom Pl, Gen Pl, Dat Pl]","[419, 238, 58, 35, 13, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3]"


In [780]:
# Gibt es unregistrierte tags?
alle_realen_tags = set(alle_tags) - {'AFIX','ZFIX','TODO','LEX_A','LEX_N', 'APPRART','MFIX','NNE','XY','_SP',}
seltsame_tags = set(a.data_tag) - alle_realen_tags
assert len(seltsame_tags) == 0

#mask = wiktionary_merkmal.tag.isin(seltsame_tags)
#wiktionary_merkmal[mask]

In [781]:
# studieren
mask1 = wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data_tag == 'TRUNC'
mask  =  mask1  &  mask2

grid(wiktionary_merkmal,mask, color='green')

3 rows out of 2586201


,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,num,meta,merkmal_order,data_score
2400220,http,lemma,True,http,http,TRUNC,z,Z,http,TRUNC,,,0.0,19.930384
2400669,www,lemma,True,www,www,TRUNC,z,Z,www,TRUNC,,,0.0,7.598153
2535483,https_M,lemma,True,https_M,https,TRUNC,z,Z,https,TRUNC,,,0.0,6.718384


In [782]:
# Alle merkmale und ihre tags
mask = wiktionary_merkmal.is_lex == True
a = pak.analyse_freqs( wiktionary_merkmal[mask], ['merkmal','data_tag'] )
grid(a)

53 rows


,merkmal,merkmal_count,merkmal_percent,data_tag,data_tag_count
0,lemma,168181,14.2,"[NN, ADJA, VVINF, NE, ADV, KOMBI, CARD, ITJ, APPR, FM, PIAT, PTKANT, KOUS, KON, DIV, PDAT, PPER, ART, $(, APPO]","[96710, 41958, 13853, 12121, 1599, 449, 419, 297, 182, 103, 71, 67, 63, 53, 38, 36, 32, 16, 15, 13]"
1,Gen Sg,119298,10.1,"[NN, NE, PIAT, PWAT, PRF, PRELAT, PTKANT, PRO]","[112557, 6723, 7, 3, 3, 2, 2, 1]"
2,Dat Sg,112417,9.5,"[NN, NE, PIAT, PTKANT, PRF, PRELAT, PWAT]","[109538, 2867, 7, 2, 1, 1, 1]"
3,Nom Sg,99655,8.4,"[NN, NE, PIAT, PWAT, PTKANT, PRELAT, PRO]","[92935, 6709, 5, 2, 2, 1, 1]"
4,Akk Sg,95660,8.1,"[NN, NE, PIAT, PWAT, PTKANT, PRELAT, PRF, PRO]","[92962, 2684, 7, 2, 2, 1, 1, 1]"
5,Nom Pl,86322,7.3,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PWAT, PPER, PRELS, PTKANT, PRO, PDS]","[84231, 2025, 19, 15, 10, 6, 4, 3, 2, 2, 2, 1, 1, 1]"
6,Gen Pl,86316,7.3,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, PRF, CARD, PWAT, PRELS, PPER, PTKANT, PRO, PDS]","[84223, 2024, 19, 15, 10, 6, 4, 3, 3, 2, 2, 2, 1, 1, 1]"
7,Akk Pl,86314,7.3,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PWAT, PRELS, PPER, PRF, PDS, PRO, PTKANT]","[84226, 2021, 19, 15, 10, 6, 4, 3, 2, 2, 2, 1, 1, 1, 1]"
8,Dat Pl,85552,7.2,"[NN, NE, PDAT, PIAT, ART, PPOSS, PRELAT, CARD, PWAT, PRELS, PPER, PRF, PTKANT, PDS, PRO]","[83502, 1980, 20, 16, 11, 6, 4, 3, 2, 2, 2, 1, 1, 1, 1]"
9,Ptz2,28426,2.4,"[ADJA, VVPP, VMPP, VAPP]","[14213, 14199, 8, 6]"


In [783]:
# studieren
mask1 = wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data_tag == 'ADV'
mask3 =  wiktionary_merkmal.merkmal == 'Superlativ'
mask  =  mask1  &  mask2  &  mask3

grid(wiktionary_merkmal,mask, color='green')

14 rows out of 2586201


,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,num,meta,merkmal_order,data_score
2056418,wohl,Superlativ,True,wohlsten,wohlsten,ADV,ADV,A,wohl,ADV,,,1.0,141.056320
2056615,unumstößlich,Superlativ,True,unumstößlichsten,unumstößlichsten,ADV,ADV,A,unumstößlich,ADV,,,1.0,35.523693
2056829,gewiss,Superlativ,True,gewissesten,gewissesten,ADV,ADV,A,gewiss,ADV,,,1.0,109.957718
2057254,gerne,Superlativ,True,liebsten,liebsten,ADV,ADV,A,gerne,ADV,,,1.0,108.105972
2058599,lange,Superlativ,True,längsten,längsten,ADV,ADV,A,lange,ADV,,,1.0,45.577663
2058811,oft,Superlativ,True,öftesten,öftesten,ADV,ADV,A,oft,ADV,,,1.0,200.598923
2059035,schal,Superlativ,True,schalsten,schalsten,ADV,ADV,A,schal,ADV,,,1.0,41.949757
2059952,inwendig,Superlativ,True,inwendigsten,inwendigsten,ADV,ADV,A,inwendig,ADV,,,1.0,24.664410
2060392,hoch_1,Superlativ,True,höchsten,höchsten,ADV,ADV,A,hoch,ADV,,,1.0,4.868667
2061576,äbich,Superlativ,True,äbichsten,äbichsten,ADV,ADV,A,äbich,ADV,,,1.0,37.883972


In [784]:
# lemma_tagZZ schreiben
wiktionary_merkmal = pak.update_col( wiktionary_merkmal, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZZ', col_rename='lemma_tagZZ', verbose=False )
wiktionary_merkmal = pak.move_cols(wiktionary_merkmal,'lemma_tagZZ','lemma_tag')
wiktionary_merkmal['lemma_tagZZ'] = wiktionary_merkmal.lemma_tagZZ.fillna('')

In [785]:
# check
mask1 = wiktionary_merkmal.lemma_tag != ''
mask2 = wiktionary_merkmal.lemma_tagZZ == ''
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [786]:
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal[mask].head(1)

,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,data_score
46,Subspezies,alt_2,True,Subspecies,Subspecies,NN,NOM,N,Subspezies,NN,N,,,1.0,46.296391


#### hunspell

In [787]:
# Hunspell instance
from hunspell import Hunspell
hun = Hunspell('de_DE', hunspell_data_dir='/usr/share/hunspell') 

In [788]:
# verify_hunspell
# liefert 1=verifiziert, 0=nicht verifiziert, -1=Besonderes Unicode-Zeichen
wiktionary_merkmal['hunspell'] = -2
mask = wiktionary_merkmal.is_lex == True
wiktionary_merkmal[mask] = wiktionary_merkmal[mask].swifter.apply(verify_hunspell, col_check='data', col_result='hunspell', hunspellinstance=hun, axis=1)
#wiktionary_merkmal = pak.change_datatype(wiktionary_merkmal, verbose=False)  

Pandas Apply:   0%|          | 0/1180261 [00:00<?, ?it/s]

In [789]:
# prüfen: alle erfasst?
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.hunspell < 0
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

check_mask:  0 rows


In [790]:
# abwerten
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.hunspell == 0
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,521518)
wiktionary_merkmal.loc[mask,'data_score'] *= 0.1
grid(wiktionary_merkmal,mask)

check_mask:  517812 rows
517812 rows out of 2586201


,lemma_id,merkmal,is_lex,data_id,data,data_tag,data_tagZ,data_tagZZ,lemma,lemma_tag,lemma_tagZZ,num,meta,merkmal_order,data_score,hunspell
46,Subspezies,alt_2,True,Subspecies,Subspecies,NN,NOM,N,Subspezies,NN,N,,,1.0,4.629639,0
93,Dezember,alt,True,Dezember~December,December,NN,NOM,N,Dezember,NN,N,,,1.0,21.269392,0
248,Mai,alt_1,True,Maien,Maien,NN,NOM,N,Mai,NN,N,,veraltet,1.0,26.315842,0
334,August_1,alt_abk,True,August_1~Gust,Gust,NE,NAM,N,August,NE,N,,,1.0,0.925968,0
335,August_1,alt_klein,True,August_1~Gustel,Gustel,NE,NAM,N,August,NE,N,,,1.0,0.925968,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600251,zehntausendste,lemma,True,zehntausendste,zehntausendste,CARD,X_zahl,X,zehntausendste,ADJA,A,,zweittag,0.0,11.541921,0
2600254,zig,lemma,True,zig,zig,ADJA,ADJ,A,zig,NN,N,,zweittag,0.0,10.829743,0
2600262,zweihundertste,lemma,True,zweihundertste,zweihundertste,CARD,X_zahl,X,zweihundertste,ADJA,A,,zweittag,0.0,14.701342,0
2600265,zwote,lemma,True,zwote,zwote,CARD,X_zahl,X,zwote,ADJA,A,,zweittag,0.0,10.233598,0


## translate_lex aufbauen ==========
* translate_lex enthält derzeit alle Wiktionary-Seitentitel, die keine Lemmas sind.
* Diverse Listen werden hier zusätzlich angefügt.
* Nach jeder Anfügung werden Dups entfernt.

In [791]:
pak.specials(translate_lex)

,data_id,data,data_tagZZ,data_score,member,lemma_id,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
0,lieben_1,lieben,,0.311824,wiki Deklinierte_Form,lieb,lieb,ADJA,A,82.432930,lieb
1,Polen_1,Polen,,0.113761,wiki Deklinierte_Form,Pole,Pole,NN,N,473.835083,pole
1003,Achille,Achille,,0.788190,wiki Deklinierte_Form,Achill,Achill,NE,N,281.867065,achill
1491,Häuser,Häuser,,0.812983,wiki Deklinierte_Form,Haus,Haus,NN,N,108960.914062,haus
38725,mildernde,mildernde,,1.790744,wiki Deklinierte_Form,mildernd,mildernd,ADJA,A,1.366636,mildernd
188180,überführte,überführte,,1.812539,wiki Deklinierte_Form,überführt_4,überführt,ADJA,A,0.032591,überführt
188314,α-Zerfällen,α-Zerfällen,,0.441083,wiki Deklinierte_Form,α-Zerfall,α-Zerfall,NN,N,345.928619,α-zerfall
233183,aushöbt,aushöbt,,1.431752,wiki Konjugierte_Form Verb,ausheben,ausheben,VVFIN,V,128.367111,ausheben
334320,durchzuboxen,durchzuboxen,,0.478273,wiki Erweiterter_Infinitiv Verb,durchboxen,durchboxen,VVFIN,V,174.290771,durchboxen
574396,-ible,-ible,,1.689462,wiki Deklinierte_Form,-ibel,-ibel,ZFIX,Z,62.837814,-ibel


In [792]:
from x32_Funktionen import *
translate_lex = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(translate_lex)

rank: 0 rows less, now 667534 rows
update_col: 667534 cells written into existing column
update_col: 667445 cells written into existing column
update_col: 0 cells written into existing column
update_col: 0 cells written into existing column
update_col: 667445 cells written into new column
update_col: 667445 cells written into existing column


In [793]:
# translate_lex suche_debug
#grid(search_str(translate_lex, suche_debug))

In [794]:
# data_id vorhanden und eindeutig?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

assert translate_lex.data_id.is_unique

#a = pak.same_but_different(translate_lex,same='data_id',different='data')
#assert a.shape[0] == 0
#grid(a)

check_mask:  0 rows


### translate_abk

In [795]:
# Abkürzungen, die nicht als Lemma registriert sind
# >> abtrennen. Der Rest wird translate_lemma hinzugefügt.
mask = ~translate_abk.kurz_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_abk,mask,3031)

translate_abk, df = pak.move_rows(translate_abk, mask, msg=None)

check_mask:  3500 rows
Move 3500 rows from 5714


In [796]:
# Spalten anpassen
df = pak.rename_col(df, 'kurz_id',     'data_id')
df = pak.rename_col(df, 'kurz',        'data')
df = pak.rename_col(df, 'kurz_tag',    'data_tag')
df = pak.rename_col(df, 'kurz_score',  'data_score')
df = pak.rename_col(df, 'lang_id',     'lemma_id')
df = pak.rename_col(df, 'lang',        'lemma')
df = pak.rename_col(df, 'lang_tag',    'lemma_tag')
df = pak.rename_col(df, 'lang_score',  'lemma_score')

df['member'] = (df.kurz_member + ' ' + df.lang_member).str.strip() 
df = pak.move_cols(df,'member','data_tag')

df = pak.drop_cols(df, ['todo','rank','konkurrenz','kurz_member','lang_member'] )

In [797]:
df = pflege_translate_tabelle(df, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(df)

rank: 0 rows less, now 667534 rows
update_col: 0 cells written into existing column
update_col: 3497 cells written into existing column
update_col: 0 cells written into existing column
update_col: 0 cells written into new column
update_col: 3497 cells written into new column
update_col: 3497 cells written into existing column


In [804]:
mask = translate_lex.data == 'H2O'
translate_lex[mask]

,data_id,data_home,data,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower


In [798]:
# Spaltenvergleich
bpy.minivenn( df.columns, translate_lex.columns, format='print2')

left_only:  {'data_tag'}
right_only: {}



In [802]:
pak.specials(df)

,data_id,data_home,data,data_tag,data_tagZZ,data_score,member,lemma_id,lemma_home,lemma,lemma_tag,lemma_tagZZ,lemma_score,lemma_lower
0,Band~Bd,,Bd,NN,,0.0,manuell Abkürzung manuell,Band,L,Band,NN,N,41571.363281,band
2,Wasser~H2O,,H2O,NN,,0.0,Abkürzung,Wasser,L,Wasser,NN,N,56061.152344,wasser
85,erkundigen~erk,,erk,VVFIN,,0.0,Abkürzung,erkundigen,L,erkundigen,VVFIN,V,870.192627,erkundigen
260,Prozent~%,,%,NN,,0.0,manuell Abkürzung Einheit manuell,Prozent,L,Prozent,NN,N,1309.592773,prozent
407,täglich~tgl,,tgl,ADJA,,0.0,manuell Abkürzung noSteig manuell,täglich,L,täglich,ADJA,A,245.535614,täglich
1615,üblich~übl,,übl,ADJA,,0.0,Abkürzung,üblich,L,üblich,ADJA,A,225.302490,üblich
2799,Quadratmikrometer~μm²,,μm²,NN,,0.0,Abkürzung,Quadratmikrometer,L,Quadratmikrometer,NN,N,374.549469,quadratmikrometer
3017,Diplom-Volkswirt~Dipl Volksw,,Dipl Volksw,NN,,0.0,Abkürzung,Diplom-Volkswirt,L,Diplom-Volkswirt,NN,N,235.341034,diplom-volkswirt
3116,üblicherweise~üblw,,üblw,ADV,,0.0,Abkürzung,üblicherweise,L,üblicherweise,ADV,A,198.932861,üblicherweise
3223,-prozentig~-%ig,,-%ig,ZFIX,,0.0,Abkürzung Gebundenes_Lexem,-prozentig,,-prozentig,<NA>,,14.858729,-prozentig


In [799]:
# check score
maskA = translate_lex.data_score == 0
maskB = translate_lex.lemma_score == 0
mask = maskA | maskB
check_mask(translate_lex,mask,0)

check_mask:  0 rows


In [800]:
# check score
maskA = df.data_score == 0
maskB = df.lemma_score == 0
mask = maskA | maskB
check_mask(df,mask,0)

Exception: ERROR: 3500 rows, but it should be a maximum of 0

In [ ]:
# anfügen
translate_lex = pak.add_rows( translate_lex, df, only_new=['data', 'lemma'])

In [ ]:
# data_tag
translate_lex = pak.move_cols(translate_lex,'data_tag','data')
translate_lex['data_tag'] = translate_lex.data_tag.fillna('')
translate_lex

### translate_gm
* gm-Merkmale sind die Merkmale auf den Wiktionary-Seiten, die keine Lemmas sind. Also diejenigen, die unter wiktionary_nolemma bzw. translate_lex gespeichert sind.
* Daher weisen ihre lemma_id nicht auf ein Lemma, sondern auf ein Lexem. lemma_id und data_id müssen also hier vertauscht werden.

In [ ]:
# translate_lex 
# Dies sind die Seiten, deren Merkmale translate_gm enthält.
# data_id ist die ID der Nicht-Lemma-Wiktionary-Seite.
(search_str(translate_lex, ['besonne']))

In [ ]:
# lemma und data vertauschen
translate_gm = pak.rename_col(translate_gm, 'lemma_id',    'data_id_neu')
translate_gm = pak.rename_col(translate_gm, 'data_id',     'lemma_id')
translate_gm = pak.rename_col(translate_gm, 'data_id_neu', 'data_id')
translate_gm = pak.rename_col(translate_gm, 'data',        'lemma')
#translate_gm = pak.rename_col(translate_gm, 'score',       'data_score')

In [ ]:
# lemma_id nachschlagen (und dabei korrigieren!)
translate_gm = pak.update_col(translate_gm, translate_lex, on='data_id', col='lemma_id' ) 

In [ ]:
# translate_gm umformen
translate_gm = pak.rename_col(translate_gm, 'merkmal', 'member')
translate_gm['member'] = (translate_gm.member + ' ' + translate_gm.meta).str.strip()

translate_gm = pak.drop_cols(translate_gm, ['is_lex','data_tag','num','meta',])

In [ ]:
# translate_gm
# data_id ist die ID der Nicht-Lemma-Wiktionary-Seite.
mask = translate_gm.lemma == 'besonnen'
a = translate_gm[mask].sort_values(['lemma_id','lemma'])
grid(a)

In [ ]:
from x32_Funktionen import *
translate_gm = pflege_translate_tabelle(translate_gm, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(translate_gm)

In [ ]:
# translate_gm
# data_id ist die ID der Nicht-Lemma-Wiktionary-Seite.
mask = translate_gm.lemma == 'besonnen'
a = translate_gm[mask].sort_values(['lemma_id','lemma'])
grid(a)

In [ ]:
assert translate_gm.lemma_home.nunique() == 2 # stand mal auf 1
assert translate_gm.data_home.nunique() == 2

In [ ]:
pak.analyse_freqs(translate_gm, ['data_home','lemma_home'])

In [ ]:
# translate_lex_trash pflegen
translate_lex_trash = pak.drop_cols(translate_lex_trash,['tag','tag_0','tag_1','tagZ','tagZZ'])
translate_lex_trash = pak.move_cols( translate_lex_trash, translate_lex.columns)
#translate_lex_trash

In [ ]:
pak.nnan(translate_gm)

In [ ]:
mask = translate_gm.lemma_tag.isnull()
check_mask(translate_gm,mask,90)

msg = 'verwaist_gm'
translate_gm, translate_lex_trash = pak.move_rows(translate_gm, translate_lex_trash, mask, msg)
grid(translate_lex_trash,msg)

In [ ]:
# Seltene Kombination (gibt's nicht mehr)
mask1 = translate_gm.lemma_home == 'T'
mask2 = translate_gm.data_home == 'T'
mask = mask1  &  mask2
check_mask(translate_gm,mask,0,51)

#grid(translate_gm,mask)
msg = 'Kombi TT'
translate_gm, translate_lex_trash = pak.move_rows(translate_gm, translate_lex_trash, mask, msg)
grid(translate_lex_trash,msg)

In [ ]:
# Bedeutungsloses löschen
mask1 = translate_gm.data == translate_gm.lemma
mask2A = translate_gm.data_tagZZ == translate_gm.lemma_tagZZ
mask2B = translate_gm.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(translate_gm,mask,235)

translate_gm, df = pak.move_rows(translate_gm,mask)
grid(df)

In [ ]:
# Schrott rauslöschen
löschmich = ['Nominativ','Genitiv','Dativ','Akkusativ','Maskulinum','Femininum','Neutrum','Singular','Plural',
            'Hauptsatzkonjugation','Nebensatzkonjugation','Artikel','unbestimmt','Fugenelement','Allomorph',
             'Deklinationsendung','Komposita','Astronomischer Kalender','Konjugationsendung','Wortbildung',
             'am Lateinischen angelehnte Deklination','Hilfsverb','backen (regelmäßig)','Hauptsatz','Endung',
             'lateinisch','unbestimmter','Maskulinums','vernetzt','Derivatem','Ableitungsmorphem','griechisch',
             'Hapaxlegomenon','Interfix','Vollverb','unbestimmt','Artikel',
            ]
mask1A = translate_gm.lemma.isin(löschmich)
mask1B = translate_gm.data.isin(löschmich)
mask1C = translate_gm.lemma_id.isin(löschmich)
mask1D = translate_gm.data_id.isin(löschmich)
mask2 = translate_gm.lemma_id.str[0] != translate_gm.data_id.str[0]
mask = (mask1A | mask1B | mask1C | mask1D)  &  mask2
check_mask(translate_gm,mask,9)

msg = 'Schrott1'
translate_gm, translate_lex_trash = pak.move_rows(translate_gm, translate_lex_trash, mask, msg)
grid(translate_lex_trash,msg)

In [ ]:
# Weiteren Schrott finden
mask1 = translate_gm.lemma_id.str[0] != translate_gm.data_id.str[0]
mask2 = translate_gm.lemma_tagZZ != 'V'    # keine Verben
mask3 = ~translate_gm.lemma.str.endswith(('en','ern','eln','sein','tun')) # und sieht auch nicht aus wie ein Verb
mask = mask1  &  mask2  &  mask3
a = pak.analyse_freqs(translate_gm[mask],['lemma','data',])
grid(a)

In [ ]:
# suche_debug translate_gm
#search_str(translate_gm, suche_debug)

In [ ]:
# not any_nan
assert not pak.any_nan(translate_gm)

In [ ]:
# Spalten anpassen
translate_gm = pak.drop_cols(translate_gm,['merkmal_order'])

In [ ]:
# Spaltenvergleich
bpy.minivenn( translate_gm.columns, translate_lex.columns, format='print2')

In [ ]:
# check score
maskA = translate_lex.data_score == 0
maskB = translate_lex.lemma_score == 0
mask = maskA | maskB
check_mask(translate_lex,mask,0)

In [ ]:
# check score
maskA = translate_gm.data_score == 0
maskB = translate_gm.lemma_score == 0
mask = maskA | maskB
check_mask(translate_gm,mask,0)

In [ ]:
# add_rows
translate_lex = pak.add_rows(translate_lex,translate_gm)

In [ ]:
# check score
maskA = translate_lex.data_score == 0
maskB = translate_lex.lemma_score == 0
mask = maskA | maskB
check_mask(translate_lex,mask,0)

In [ ]:
pak.specials(translate_lex)

In [ ]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

In [ ]:
#pak.sample(translate_lex)

In [ ]:
# nan entfernen
translate_lex['lemma_tag'] = translate_lex.lemma_tag.fillna('')
translate_lex['data_tag']  = translate_lex.data_tag.fillna('')

In [ ]:
#pak.nnan(translate_lex)

In [ ]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id','data','data_tag','data_tagZZ','data_score','member', 'lemma_id','lemma','lemma_tag','lemma_tagZZ','lemma_lower','lemma_score','data_home','lemma_home', ]
col_funcs   =  [ '',       '',    '',        '',          'sum',       aggwords, '',        '',     '',         '',           '',           'sum',        '',         ''          , ]   
col_names   =  [ '',       '',    '',        '',          'data_score','member', '',        '',     '',         '',           '',           'lemma_score','',         ''          , ] 

In [ ]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

In [ ]:
# data_id vorhanden und nicht widerspüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

#assert translate_lex.data_id.is_unique

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [ ]:
gridt(a)

In [ ]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

### translate_m
* Alle Lexeme aus wiktionary_merkmal

In [ ]:
# Bei lemma: lemma_id und data_id müssen gleich sein
mask1 = wiktionary_merkmal.merkmal == 'lemma'
mask2 = wiktionary_merkmal.lemma_id != wiktionary_merkmal.data_id
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

In [ ]:
# Alle Lexeme aus wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
translate_m = wiktionary_merkmal[mask].copy()
#translate_m

In [ ]:
# translate_m umformen
translate_m = pak.rename_col(translate_m, 'merkmal', 'member')
translate_m['member'] = ('merkmal ' + translate_m.member + ' ' + translate_m.meta).str.strip()

translate_m = pak.drop_cols(translate_m, ['is_lex','num','meta'])

In [ ]:
# lemma_tagZZ schreiben
translate_m = pak.update_col( translate_m, translate_tagZ, left_on='lemma_tag', right_on='tag', col='tagZZ', col_rename='lemma_tagZZ', verbose=False )
translate_m = pak.move_cols(  translate_m,'lemma_tagZZ','lemma_tag')
translate_m['lemma_tagZZ'] =  translate_m.lemma_tagZZ.fillna('')

In [ ]:
# temp
mask1 = translate_m.data  == 'ausgerutscht'
mask2 = translate_m.lemma == 'ausrutschen'
mask = mask1  &  mask2
translate_m[mask1]

In [ ]:
# data_id löschen, wenn nötig
mask1 = translate_m.data_id != translate_m.data
mask2 = translate_m.data_id == translate_m.lemma_id
mask = mask1  &  mask2
check_mask(translate_m,mask,22339)

translate_m.loc[mask,'data_id'] = ''
grid(translate_m,mask)
#a = pak.analyse_freqs(translate_m[mask],['member','data'])
#grid(a)

In [ ]:
# wiktionary_lemma suche_debug
grid(search_str(translate_lex, suche_debug))

In [ ]:
raise Stop

In [ ]:
# data_id: Wenn leer, neu schreiben
mask =  translate_m.data_id == ''
check_mask(translate_m,mask,22339)
translate_m[mask] = pak.update_col( translate_m[mask], wiktionary_lemma, cond='null', left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'], col='lemma_id', col_rename='data_id', col_score='lemma_score') 
translate_m[mask] = pak.update_col( translate_m[mask], translate_lex,    cond='null',      on=['data','lemma_tagZZ'],                             col='data_id',                        col_score='data_score') 

In [ ]:
# data_id neu erzeugen
mask =  translate_m.data_id == ''
check_mask(translate_m,mask,0)

translate_m.loc[mask,'data_id'] = translate_m[mask].lemma + '~' + translate_m[mask].data + '~N'

In [ ]:
search_str(translate_m, suche_debug)

In [ ]:
#translate_m_bak = translate_m.copy()
#translate_m = translate_m_bak.copy()

In [ ]:
from x32_Funktionen import *
translate_m = pflege_translate_tabelle(translate_m, translate_tagZ, wiktionary_lemma, translate_lex )
#pak.sample(translate_m)

In [ ]:
# Bedeutungsloses löschen
mask1 = ~translate_m.member.str.contains('lemma')
mask2 =  translate_m.data == translate_m.lemma
mask3A = translate_m.data_tagZZ == translate_m.lemma_tagZZ
mask3B = translate_m.data_tagZZ == ''
mask = mask1  &  mask2  &  (mask3A | mask3B)
#grid(translate_m,mask)
check_mask(translate_m,mask,374075)

translate_m, df = pak.move_rows(translate_m,mask)
grid(df)

In [ ]:
pak.analyse_freqs(translate_m, ['lemma_home','data_home'])

In [ ]:
# Neuzugänge
mask = translate_m.data_home == ''
grid(translate_m,mask)

In [ ]:
assert not pak.any_nan(translate_m)

In [ ]:
# Spalten anpassen
translate_m = pak.drop_cols(translate_m,['data_tagZ', 'hunspell', 'merkmal_order'])
#translate_m = pak.rename_col(translate_m, 'score', 'data_score')

In [ ]:
# Spaltenvergleich
bpy.minivenn( translate_m.columns, translate_lex.columns, format='print2')

In [ ]:
# check score
maskA = translate_lex.data_score == 0
maskB = translate_lex.lemma_score == 0
mask = maskA | maskB
check_mask(translate_lex,mask,0)

In [ ]:
# check score
maskA = translate_m.data_score == 0
maskB = translate_m.lemma_score == 0
mask = maskA | maskB
check_mask(translate_m,mask,0)

In [ ]:
raise Stop

In [ ]:
search_str(translate_m, suche_debug)

In [ ]:
# add_rows
translate_lex = pak.add_rows(translate_lex,translate_m)

In [ ]:
raise Stop

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

In [ ]:
# nan entfernen
translate_lex['data_tag']   = translate_lex.data_tag.fillna('')
translate_lex['data_tagZZ'] = translate_lex.data_tagZZ.fillna('')

In [ ]:
pak.nnan(translate_lex)

In [ ]:
# Bleibt: Konfiguration Dups entfernen
# aggwords    =  pak.agg_words_nodup
# col_origins =  [ 'data_id','data','data_tag','data_tagZZ','data_score','member', 'lemma_id','lemma','lemma_tag','lemma_tagZZ','lemma_lower','lemma_score','data_home','lemma_home', ]
# col_funcs   =  [ '',       '',    '',        '',          'sum',       aggwords, '',        '',     '',         '',           '',           'sum',        '',         ''          , ]   
# col_names   =  [ '',       '',    '',        '',          'data_score','member', '',        '',     '',         '',           '',           'lemma_score','',         ''          , ] 

In [ ]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

In [ ]:
# data_id vorhanden und nicht widersprüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0

In [ ]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

In [ ]:
#pak.sample(translate_lex)

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

In [ ]:
mask = translate_lex.data_tag != ''
check_mask(translate_lex,mask,562350)

### translate_lma und translate_lmb
* aus wiktionary_lexmerkmal

In [ ]:
# zunächst wiktionary_lexmerkmal: Nichtlexeme löschen. Das sind vor allem syllables, ipa und rhymes.
mask = wiktionary_lexmerkmal.is_lex == False
check_mask( wiktionary_lexmerkmal, mask, 1575569) # 1523133

msg = 'Nichtlexeme'
wiktionary_lexmerkmal, wiktionary_merkmal_trash = pak.move_rows(wiktionary_lexmerkmal, wiktionary_merkmal_trash, mask, msg )
#grid(wiktionary_merkmal_trash, msg)

In [ ]:
#pak.analyse_freqs(wiktionary_lexmerkmal,['merkmal','data'])

In [ ]:
# wiktionary_lexmerkmal
wiktionary_lexmerkmal.head(1)

In [ ]:
# lemma_id für data_id finden
wiktionary_lexmerkmal['lemma_id'] = ''
wiktionary_lexmerkmal = pak.update_col(wiktionary_lexmerkmal, translate_lex, on='data_id',  col='lemma_id')

In [ ]:
# translate_lma erstellen
mask = wiktionary_lexmerkmal.lemma_id != ''
translate_lma = wiktionary_lexmerkmal[mask].copy()

translate_lma = pak.rename_col(translate_lma, 'merkmal', 'member')
translate_lma['member'] = ('lma ' + translate_lma.member + ' ' + translate_lma.meta ).str.strip()

translate_lma = pak.drop_cols(translate_lma, ['lex_id','is_lex','data_tag','num','meta','data'])

In [ ]:
# lemma_id für lex_id finden
wiktionary_lexmerkmal['lemma_id'] = ''
wiktionary_lexmerkmal = pak.update_col(wiktionary_lexmerkmal, translate_lex, left_on='lex_id', right_on='data_id',  col='lemma_id', col_score='data_score')

In [ ]:
# translate_lmb erstellen
mask = wiktionary_lexmerkmal.lemma_id != ''
translate_lmb = wiktionary_lexmerkmal[mask].copy()

translate_lmb = pak.rename_col(translate_lmb, 'merkmal', 'member')
translate_lmb['member'] = ('lmb ' + translate_lmb.member + ' ' + translate_lmb.meta ).str.strip()

translate_lmb = pak.drop_cols(translate_lmb, ['data_id','data','data_tag','is_lex','num','meta','data'])

translate_lmb = pak.rename_col(translate_lmb, 'lex_id', 'data_id')

In [ ]:
# pflege_translate_tabelle
from x32_Funktionen import *
translate_lma = pflege_translate_tabelle(translate_lma, translate_tagZ, wiktionary_lemma, translate_lex )
translate_lmb = pflege_translate_tabelle(translate_lmb, translate_tagZ, wiktionary_lemma, translate_lex )

In [ ]:
# Bedeutungsloses löschen
mask1  = translate_lma.data       == translate_lma.lemma
mask2A = translate_lma.data_tagZZ == translate_lma.lemma_tagZZ
mask2B = translate_lma.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(translate_lma,mask,16606)

translate_lma, df = pak.move_rows(translate_lma,mask)
grid(df)

In [ ]:
# Bedeutungsloses löschen
mask1  = translate_lmb.data       == translate_lmb.lemma
mask2A = translate_lmb.data_tagZZ == translate_lmb.lemma_tagZZ
mask2B = translate_lmb.data_tagZZ == ''
mask = mask1  &  (mask2A | mask2B)
check_mask(translate_lmb,mask,9)

translate_lmb, df = pak.move_rows(translate_lmb,mask)
grid(df)

In [ ]:
# Spalten anpassen
translate_lma = pak.drop_cols(translate_lma,['merkmal_order'])
translate_lma['data_score'] = 0

In [ ]:
# Spaltenvergleich
bpy.minivenn( translate_lma.columns, translate_lex.columns, format='print2')

In [ ]:
# check score
maskA = translate_lex.data_score == 0
maskB = translate_lex.lemma_score == 0
mask = maskA | maskB
check_mask(translate_lex,mask,0)

In [ ]:
# check score
maskA = translate_lma.data_score == 0
maskB = translate_lma.lemma_score == 0
mask = maskA | maskB
check_mask(translate_lma,mask,0)

In [ ]:
raise Stop

In [ ]:
# Spalten anpassen
translate_lmb = pak.drop_cols(translate_lmb,['merkmal_order'])

In [ ]:
# Spaltenvergleich
bpy.minivenn( translate_lmb.columns, translate_lex.columns, format='print2')

In [ ]:
# check score
maskA = translate_lmb.data_score == 0
maskB = translate_lmb.lemma_score == 0
mask = maskA | maskB
check_mask(translate_lex,mask,0)

In [ ]:
raise Stop

In [ ]:
# Scores reduzieren
translate_lma['data_score'] /= 3
translate_lma['lemma_score'] /= 3
translate_lmb['data_score'] /= 3
translate_lmb['lemma_score'] /= 3

In [ ]:
#pak.sample(translate_lma)

In [ ]:
# add_rows
translate_lex = pak.add_rows(translate_lex,translate_lma)
translate_lex = pak.add_rows(translate_lex,translate_lmb)

### pflegen

In [ ]:
# data_id vorhanden und nicht widersprüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [ ]:
# verwaiste lemma
mask = translate_lex.lemma.isnull()
check_mask(translate_lex,mask,0,10)

msg = 'verwaist_lemma_10'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

In [ ]:
# verwaiste lemma_id
mask = ~translate_lex.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lex,mask,89)

msg = 'verwaist_lemma_id_100'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

In [ ]:
#data_tag fillna
translate_lex['data_tag'] = translate_lex.data_tag.fillna('')

In [ ]:
assert not pak.any_nan(translate_lex)
#pak.nnan(translate_lex)

In [ ]:
# pflege_translate_tabelle
#from x32_Funktionen import *
translate_lex = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma, translate_lex )

### analyse_stringdiff

In [ ]:
from strsimpy.qgram import QGram 

qgram = QGram(2)
a = 'herbeispringen'
b = 'spring herbei'
c = 'herbeispringendes'
d = 'xfalsche Anfänger'
print(qgram.distance(a,b))
print(qgram.distance(a,c))
print(qgram.distance(a,d))

In [ ]:
from strsimpy.metric_lcs import MetricLCS
metric_lcs = MetricLCS()

print(metric_lcs.distance(a,b))
print(metric_lcs.distance(a,c))
print(metric_lcs.distance(a,d))

In [ ]:
# data_swaped: dächtest um >> umdächtest
translate_lex['data_swaped'] = translate_lex.data.str.split()
mask = translate_lex.data_swaped.str.len() == 2
translate_lex.loc[ mask, 'data_swaped'] = translate_lex[mask].data_swaped.str[-1] + translate_lex[mask].data_swaped.str[0]
translate_lex.loc[~mask, 'data_swaped'] = ''

In [ ]:
# Ermittelt die Differenz zwischen lex und lemma

def analyse_stringdiff(zeile):
    if len(zeile.data_swaped) > 0:
        result_a = metric_lcs.distance( zeile.data.lower(), zeile.lemma.lower() ) 
        result_b = metric_lcs.distance( zeile.data_swaped.lower(), zeile.lemma.lower() )     
        zeile['same_lexlemma'] = 1.0 - min(result_a, result_b)
    else:
        result   = metric_lcs.distance( zeile.data.lower(), zeile.lemma.lower() ) 
        zeile['same_lexlemma'] = 1.0 - result      
    return zeile

In [ ]:
# apply analyse_stringdiff 
translate_lex['same_lexlemma']   = 0.0
translate_lex = translate_lex.swifter.apply( analyse_stringdiff, axis=1 )
#translate_lex = pak.change_datatype(translate_lex, verbose=False)  

In [ ]:
# data_swaped wieder löschen
translate_lex = pak.drop_cols(translate_lex,'data_swaped')

In [ ]:
# same_lexlemma auf scores übertragen
translate_lex['data_score']  = translate_lex.data_score  * ((translate_lex.same_lexlemma + 0.1)**0.5)
translate_lex['lemma_score'] = translate_lex.lemma_score * ((translate_lex.same_lexlemma + 0.1)**0.5)

### data_id eindeutig

In [ ]:
raise Stop

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

In [ ]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  ['data_id','data','data_tag','data_tagZZ','data_score','member', 'lemma_id','lemma','lemma_tag','lemma_tagZZ','lemma_lower','lemma_score','data_home','lemma_home','same_lexlemma',]
col_funcs   =  ['',       '',    '',        '',          'sum',       aggwords, '',        '',     '',         '',           '',           'sum',        '',         ''          ,'first',        ]   
col_names   =  ['',       '',    '',        '',          'data_score','member', '',        '',     '',         '',           '',           'lemma_score','',         ''          ,'same_lexlemma',] 

In [ ]:
# Gruppieren!
lookup_lex_tag = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

### weniger data_ids 

In [ ]:
translate_lex['data_id_bak'] = '' # Sicherheitskopie von data_id

In [ ]:
# Eine halbe Million data_ids, die nirgendwohin verweisen. 
# Sichern und Leeren!
mask1 =  translate_lex.data != ''
mask2 = ~translate_lex.data_id.isin(wiktionary_lemma.lemma_id)
mask3 = ~translate_lex.data_id.isin(wiktionary_merkmal.data_id)
mask = mask1  &  mask2  &  mask3  
check_mask(translate_lex,mask,470316)

# kennzeichnen
translate_lex.loc[mask,'data_id_bak'] = translate_lex[mask].data_id  # sichern
translate_lex.loc[mask,'data_id']     = ''                           # leeren

#grid(translate_lex[mask],pak.sample )

In [ ]:
# Sicherheitskopie
#translate_lex_bak = translate_lex.copy()
#translate_lex = translate_lex_bak.copy()

In [ ]:
# Zweites data_id-Feld
translate_lex['data_id2'] = translate_lex.data_id.copy()
translate_lex = pak.move_cols(translate_lex,'data_id2','data_id')
#translate_lex

In [ ]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  ['data_id','data_id2','data_id_bak','data','data_tag','data_tagZZ','data_score','member','lemma_id','lemma','lemma_tag','lemma_tagZZ','lemma_lower','lemma_score','data_home','lemma_home','same_lexlemma',]
col_funcs   =  ['max',    'min',     'max',        '',    '',        '',          'sum',       aggwords,'',        '',     '',         '',           '',           'sum',        'first',    'first',     'first',        ]   
col_names   =  ['data_id','data_id2','data_id_bak','',    '',        '',          'data_score','member','',        '',     '',         '',           '',           'lemma_score','data_home','lemma_home','same_lexlemma',] 

In [ ]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)

In [ ]:
# Beide data_id leer? Dann data_id_bak verwenden
mask1 = translate_lex.data_id == ''
mask2 = translate_lex.data_id2 == ''
mask = mask1  &  mask2
check_mask(translate_lex,mask,354478)
translate_lex.loc[mask,'data_id'] = translate_lex[mask].data_id_bak
#grid(translate_lex,mask)

In [ ]:
# Prüfen: data_id immer belegt
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

In [ ]:
# data_ids sind verschieden, aber nahezu gleich
# z.B. abstoßend_1 und abstoßend
mask1 = translate_lex.data_id != translate_lex.data_id2
mask2 = translate_lex.data_id2 != ''
mask3 = pak.remove_str( translate_lex.data_id, ['_1','_2','_3','_4','_MADJ',], safemode=False )  ==  pak.remove_str( translate_lex.data_id2, ['_1','_2','_3','_4','_MADJ',], safemode=False )
mask = mask1  &  mask2  &  mask3
check_mask(translate_lex,mask,393)

translate_lex.loc[mask,'data_id'] = translate_lex[mask].data_id2 # das Minimum, also die kürzere id
translate_lex.loc[mask,'data_id2'] = ''
#grid(translate_lex,mask)

In [ ]:
# Jetzt gibt es keine widersprüchliche data_id mehr
mask1 = translate_lex.data_id != translate_lex.data_id2
mask2 = translate_lex.data_id2 != ''
mask = mask1  &  mask2
check_mask(translate_lex,mask,0)
#grid(translate_lex,mask,)

In [ ]:
# Hilfsspalten löschen
translate_lex = pak.drop_cols(translate_lex,['data_id2','data_id_bak'])

In [ ]:
# translate_lex suche_debug
grid(search_str(translate_lex, suche_debug))

In [ ]:
# pflege_translate_tabelle
translate_lex_ranked  = pak.rank(translate_lex, col_score='data_score', cols_group='data_id', on_conflict='first')
translate_lex         = pflege_translate_tabelle(translate_lex, translate_tagZ, wiktionary_lemma, translate_lex_ranked )

In [ ]:
# translate_lex: data_id vorhanden und nicht widersprüchlich?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

#assert translate_lex.data_id.is_unique

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [ ]:
assert not pak.any_nan(translate_lex)

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Alben'
translate_lex[mask]

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'ausführt'
translate_lex[mask]

In [ ]:
# Beispiel anzeigen
#finde = ['eine','einer','eines','einen','ein','einem']
#mask = translate_lex.data.isin(finde)
#translate_lex[mask]

In [ ]:
#pak.analyse_freqs(translate_lex,['data_home','lemma_home'])

## wiktionary_merkmal

In [ ]:
# check data_id
mask1 =  wiktionary_merkmal.data_id == ''
mask2 =  wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

In [ ]:
# Sicherheitskopie
#wiktionary_merkmal_bak = wiktionary_merkmal.copy()
#wiktionary_merkmal = wiktionary_merkmal_bak.copy()

In [ ]:
wiktionary_merkmal['data_id_bak'] = '' # Sicherheitskopie von data_id

In [ ]:
#translate_lex

In [ ]:
# verwaiste data_id:
# leeren und neu belegen
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data != ''
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4 
check_mask(wiktionary_merkmal,mask, 0, 34)

# kennzeichnen
wiktionary_merkmal.loc[mask,'data_id_bak'] = wiktionary_merkmal[mask].data_id  # sichern
wiktionary_merkmal.loc[mask,'data_id']     = ''                                # leeren

# data_id schreiben 
on = ['data','lemma_tagZZ']
wiktionary_merkmal[mask] = pak.update_col( wiktionary_merkmal[mask], wiktionary_lemma, cond='null', left_on=on, right_on=['lemma','tagZZ'], col='lemma_id', col_rename='data_id', col_score='lemma_score')
wiktionary_merkmal[mask] = pak.update_col( wiktionary_merkmal[mask], translate_lex,    cond='null',      on=on,                             col='data_id',  col_score='data_score', )

# Kontrolle
mask1 =  wiktionary_merkmal.is_lex == True
mask2 =  wiktionary_merkmal.data != ''
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4  
check_mask(wiktionary_merkmal,mask, 0)

# anschauen
mask = wiktionary_merkmal.data_id_bak != ''
grid(wiktionary_merkmal,mask)

In [ ]:
mask = wiktionary_merkmal.data_id == 'Petersen~Peterson'
wiktionary_merkmal[mask]

In [ ]:
# check data_id
mask1 =  wiktionary_merkmal.data_id == ''
mask2 =  wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

In [ ]:
# Verlinkung auf data prüfen 
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, translate_lex, on='data_id', col='data', col_rename='data_check', col_score='data_score')
mask_kaputt = wiktionary_merkmal.data != wiktionary_merkmal.data_check
check_mask(wiktionary_merkmal,mask_kaputt,0)
#grid(wiktionary_merkmal,mask_kaputt)

In [ ]:
# meta == lex 
wiktionary_merkmal = pak.update_col(wiktionary_merkmal, translate_lex, left_on='meta', right_on='data', col='lemma', col_rename='löschmich_lemma', col_score='data_score')

mask = wiktionary_merkmal.lemma == wiktionary_merkmal.löschmich_lemma
check_mask(wiktionary_merkmal,mask,2742)
wiktionary_merkmal.loc[mask,'meta'] = ''

wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,'löschmich_lemma')

In [ ]:
# Hilfsdaten löschen
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,['data_id_bak','notiz','data_check'])

In [ ]:
grid(wiktionary_merkmal,pak.sample)

In [ ]:
#mask = translate_lex.data.isin(['besonnte','überlegte'])
#grid(translate_lex,mask)

In [ ]:
#translate_lex_bak = translate_lex.copy()

# rückgängig
# translate_lex = translate_lex_bak.copy()

### Kontrolle wiktionary_merkmal (aus d01)
* Diverse Kontrollen

In [ ]:
# plan_merkmal Steuertabelle neu erstellen
try:
    del plan_merkmal
except:
    pass

from Steuertabellen.s01_Steuertabellen import *
plan_merkmal = plan_merkmal_erstellen()
#pak.sample(plan_merkmal)
grid(plan_merkmal)

In [ ]:
# Merkmale lt. Steuertabelle und lt. Realität
mask = (plan_merkmal.plan != 2)
merkmale_plan  = set(plan_merkmal[mask].merkmal) - set(member_replaces.keys())
merkmale_data  = set(wiktionary_merkmal.merkmal.unique())
merkmale_trash = set(wiktionary_merkmal_trash.merkmal.unique())

In [ ]:
# left_only:  Merkmale, die in der Realität nicht auftreten
# right_only: Merkmale, die nicht geplant sind
a = bpy.minivenn(merkmale_plan,merkmale_data)
bpy.minivenn(merkmale_plan,merkmale_data,format='print2')

In [ ]:
# Welche vermissten Merkmale treten in wiktionary_merkmal_trash auf?
a = bpy.minivenn( a['left_only'] , merkmale_trash)
mask = wiktionary_merkmal_trash.merkmal.isin(a['both'])
pak.analyse_freqs(wiktionary_merkmal_trash[mask],['merkmal','lemma'])

In [ ]:
# Veränderungen lemma_tag >> data_tag
# nach Merkmal
mask1 = wiktionary_merkmal.data_tag != wiktionary_merkmal.lemma_tag
mask2 = wiktionary_merkmal.data_tag != ''
mask = mask1  &  mask2
pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','lemma_tag','data_tag',])

In [ ]:
raise Stop

In [ ]:
# TODO_TAGS_IN_MERKMALE
bgc('orange')
mask = wiktionary_merkmal.merkmal.isin({'VAFIN', 'VMFIN', 'VVIMP', 'VVIZU',})
wiktionary_merkmal[mask]

In [ ]:
# Keine nan in data
assert pak.nnan(wiktionary_merkmal.data) == 0

In [ ]:
# Eindeutige Datentypen
for col in wiktionary_merkmal.columns:
    print(col)
    if col == 'num':
        continue
    assert pak.ntypes(wiktionary_merkmal[col]) == 1

In [ ]:
wiktionary_merkmal.shape

## translate_lemma ==========
* Datensätze aus translate_lex, deren IDs von Lemma zu Lemma zeigen
* Aber keine Lemmas (also keine Verweise auf sich selbst)
* Datensätze aus translate_abk

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Ackermann'
translate_lex[mask]

In [ ]:
# translate_lemma erzeugen
keine_lemmas = ['einen','beigen','Montage',]

mask1 =  translate_lex.data_home == 'B'             # beides sind lemmas
mask2 =  translate_lex.data != translate_lex.lemma  # sie sind verschieden
mask3 = ~translate_lex.data.isin(keine_lemmas)      # gehören nicht zu den Ausnahmen
mask4 = ~translate_lex.member.str.contains('Ptz1')  # kein Partizip1
mask5 = ~translate_lex.member.str.contains('Ptz2')  # kein Partizip2
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_lex,mask,60004)
#grid(translate_lex,mask)
translate_lex, translate_lemma = pak.move_rows(translate_lex,mask, msg=None)

In [ ]:
pak.sample(translate_lemma)

### translate_abk

In [ ]:
# Alle Kurzformen sind in wiktionary_lemma
mask_abk = wiktionary_lemma.member.str.contains('Abkürzung')

mask = ~translate_abk.kurz_id.isin(wiktionary_lemma[mask_abk].lemma_id)
check_mask(translate_abk,mask,0)

msg = 'kurz_id inzwischen verwaist'
translate_abk, translate_abk_trash = pak.move_rows( translate_abk, translate_abk_trash, mask, msg )
grid(translate_abk_trash,msg)

In [ ]:
assert not pak.any_nan(translate_abk)

In [ ]:
# Umbenennungen. data und lemma sind nicht passende Bezeichnungen, aber pflege_translate_tabelle wünscht diese. 
# Wird später durch passendere ersetzt.
spalten = ['kurz_id','lang_id','todo']
df = translate_abk[spalten].copy()
df = pak.rename_col(df, 'kurz_id', 'data_id')
df = pak.rename_col(df, 'lang_id', 'lemma_id') 
df = pak.rename_col(df, 'todo', 'member') 
df['member'] = ('Abkürzung ' + df.member).str.strip()
df.head(1)

In [ ]:
# lang_id war kein Lemma >> translate_lex nachschlagen
df['lemma_id_new'] = ''
mask = ~df.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(df,mask,31)

df.loc[mask] = pak.update_col( df[mask],          
                               translate_lex,
                               col_score='data_score', 
                               left_on='lemma_id',
                               right_on='data_id',
                               col='lemma_id',
                               col_rename='lemma_id_new',
                               
                       )   

mask = df.lemma_id_new != ''
df.loc[mask,'lemma_id'] = df[mask].lemma_id_new

df = pak.drop_cols(df, 'lemma_id_new')
#df[mask]

In [ ]:
# Müll löschen
löschmich = ['PLURV','PIMS','MRSA','EBRG','BodSchätzDB','ZÄPrO','SJZ','Mmes.','RÄ','GDPdU','E_570','E_163']
mask = df.data_id.isin(löschmich)
df = pak.drop_rows(df,mask)

In [ ]:
# dups löschen
mask = df.duplicated(subset=['data_id','lemma_id'])
df = pak.drop_rows(df,mask)

In [ ]:
assert not pak.any_nan(df)

In [ ]:
#pak.nnan(df)

In [ ]:
# score wird später zu data_score
df = pak.update_col(df,          
                    wiktionary_lemma,
                    left_on='data_id',
                    right_on='lemma_id',
                    col='lemma_score',
                       )   

In [ ]:
raise Stop

In [ ]:
# pflege_translate_tabelle
df = pflege_translate_tabelle(df, translate_tagZ, wiktionary_lemma, translate_lex )

In [ ]:
# data_swaped: dächtest um >> umdächtest
df['data_swaped'] = df.data.str.split()
mask = df.data_swaped.str.len() == 2
check_mask(df,mask)
df.loc[ mask, 'data_swaped'] = df[mask].data_swaped.str[-1] + df[mask].data_swaped.str[0]
df.loc[~mask, 'data_swaped'] = ''

In [ ]:
#pak.analyse_cols(df)

In [ ]:
# apply analyse_stringdiff 
df['same_lexlemma']   = 0.0
df = df.apply( analyse_stringdiff, axis=1 )

In [ ]:
# data_swaped wieder löschen
df = pak.drop_cols(df,'data_swaped')

In [ ]:
raise Stop

In [ ]:
# data_tag
df['data_tag'] = df.lemma_tag.copy()

In [ ]:
# Spaltenvergleich
bpy.minivenn( df.columns, translate_lemma.columns, format='print2')

In [ ]:
raise Stop

In [ ]:
# add_rows
translate_lemma = pak.add_rows(translate_lemma, df)

In [ ]:
# rename_col
translate_lemma = pak.rename_col(translate_lemma, 'data_id',     'lem_id')
translate_lemma = pak.rename_col(translate_lemma, 'data_home',   'lem_home')
translate_lemma = pak.rename_col(translate_lemma, 'data',        'lem')
translate_lemma = pak.rename_col(translate_lemma, 'data_score',  'lem_score')

In [ ]:
translate_lemma = pak.rename_col(translate_lemma, 'data_tag',  'lem_tag')
translate_lemma = pak.rename_col(translate_lemma, 'data_tagZZ','lem_tagZZ')

In [ ]:
grid(translate_lemma, pak.sample)

In [ ]:
# lem_tag
translate_lemma = pak.update_col( translate_lemma,          
                                  wiktionary_lemma,
                                  left_on='lem_id',
                                  right_on='lemma_id',
                                  col='tag',
                                  col_rename = 'lem_tag',
                       )   

translate_lemma = pak.update_col( translate_lemma,          
                                  wiktionary_lemma,
                                  left_on='lem_id',
                                  right_on='lemma_id',
                                  col='tagZZ',
                                  col_rename = 'lem_tagZZ',
                       )   

translate_lemma['lem_tag']   = translate_lemma.lem_tag.fillna('')
translate_lemma['lem_tagZZ'] = translate_lemma.lem_tagZZ.fillna('')
#translate_lemma = pak.move_cols( translate_lemma, ['lem_tag','lem_tagZZ'],'lem')      

### belöschen

In [ ]:
# translate_lemma_trash
translate_lemma_trash = translate_lemma.head(0)
translate_lemma_trash['msg'] = ''

In [ ]:
a = pak.analyse_freqs(translate_lemma,['lemma','lem'])
grid(a)

In [ ]:
#translate_lemma

In [ ]:
# aussortiert1
erlaubt = ['H2O','CAD','CH','GAU','DIN','UN','UNO','LCD','IGS','2D','3D','4D','ISP','IR','DRK','MPU','OOP','ÖPNV','° C','°C','HDTV','° F','°F']
mask1 =  translate_lemma['same_lexlemma'] < 0.3
mask2 =  translate_lemma.lem.str[0] != translate_lemma.lemma.str[0]
mask3 =  translate_lemma.lem.str.len() <= 4
mask4 =  translate_lemma.lem.str.isupper()
mask5 = ~translate_lemma.lem.isin(erlaubt)
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
check_mask(translate_lemma,mask,0,212)

msg = 'aussortiert1'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg)
grid(translate_lemma_trash,msg)

In [ ]:
# aussortiert2
# 

löschmich = ['-','Hag','DrIng','Deik','Fritsch','Frerk', 'Fricke','nomen nominandum',
             'wullacken','Diastema mediale','BM', 'JO', 'NB',
             'Rosch ha-Schana','Wiktionary','Fuzo', 'Fuzo.',
             'Intern', 'Intern.', 'iNet', 'iNet.', 'ZUM', 'ut supra','Art', 'GSSD', 
             'Met','DE', 'DFV', 'DHB', 'DSV','AN', 'AS', 'AUR','GVBl', 'GWB', 'GenG', 'Ges',
             'BKG', 'BVT', 'BfN', 'BfS','ritenuto','Folio','Aserbaidschan-Manat',
             'sensu lato','nes.','ceterum censeo',
             'pro rata','GGVS','IRPA','Tau','ABC',
             'ISA','DOS','ABF','ÖFB','VOC','KPÖ','IN','ZIF','PVP','BDO','g. e.','AUT','EUV',
            ]
behalten  = ['nen','ne','nes', 'geeint','einend', ]
mask1A =  translate_lemma.lem.isin(löschmich)
mask1B =  translate_lemma.lemma.isin(löschmich)
mask2  = ~translate_lemma.lem.isin(behalten)
mask = (mask1A | mask1B)  &  mask2
check_mask(translate_lemma,mask,88)

msg = 'aussortiert2'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg)
grid(translate_lemma_trash,msg)

In [ ]:
# unabhängige_lexeme

unabhängige_lexeme  = ['Alu','ALU','Arbeitslosenunterstützung','CO2', 'CO₂','chemisch','USK','vergl', 'verglasen','vergl.','24/7','Stunde','Schriftwerk','Buch','und','E 962','Verbindung',
                       'km/h','Kilometer','weiblich','männlich','Elefant','frühneuhochdeutsch','sein','Disko','modern','tun','Handlung',
                      ]
mask1 = translate_lemma.lem.isin(unabhängige_lexeme)
mask2 = translate_lemma.lemma.isin(unabhängige_lexeme)
mask = mask1  &  mask2
check_mask(translate_lemma,mask,16)

msg = 'unabhängige_lexeme'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg)
grid(translate_lemma_trash,msg)

In [ ]:
# Mögliche Probleme
# 
spalten = ['lem','lemma','member','lemma_score']
mask1 =  translate_lemma['same_lexlemma'] < 0.3
mask2 = translate_lemma.lem.str[0] != translate_lemma.lemma.str[0]
mask = mask1  &  mask2
a = translate_lemma[mask][spalten].sort_values(['lemma_score'], ascending=[False])
grid(a)

In [ ]:
# lem_tag übertragen, falls nötig
mask = translate_lemma.lem_tag.isnull()
check_mask(translate_lemma,mask,0,10)

translate_lemma.loc[mask,'lem_tag']   = translate_lemma[mask].lemma_tag
translate_lemma.loc[mask,'lem_tagZZ'] = translate_lemma[mask].lemma_tagZZ

In [ ]:
# alle getaggt
maskA = translate_lemma.lem_tag == ''
maskB = translate_lemma.lem_tagZZ == ''
mask = maskA | maskB
check_mask(translate_lemma,mask,0)

In [ ]:
mask = translate_lemma.isna().any(axis=1)
translate_lemma[mask]

In [ ]:
#assert not pak.any_nan(translate_lemma)
pak.nnan(translate_lemma)

In [ ]:
pak.sample(translate_lemma)

## wiktionary_merkmal_text ==========

In [ ]:
# lemma übertragen
assert not pak.any_nan(wiktionary_lemma, without=['page_id','page_part','data_id',])
wiktionary_merkmal_text = pak.update_col( wiktionary_merkmal_text, wiktionary_lemma, on='lemma_id', col='lemma'  )

In [ ]:
# verwaise_lemma_id finden
mask = wiktionary_merkmal_text.lemma.isnull()
check_mask(wiktionary_merkmal_text,mask,0)
#verwaise_lemma_id = wiktionary_merkmal_text[mask].lemma_id.drop_duplicates()
#verwaise_lemma_id

In [ ]:
# Striche entfernen, auch in wiktionary_merkmal_text
mask1 = wiktionary_merkmal_text.data.str.count("\'") > 1
mask2 = wiktionary_merkmal_text.merkmal.isin(['def','abk']) 
mask  =  mask1  &  mask2 
check_mask(wiktionary_merkmal_text,mask,2290)
wiktionary_merkmal_text.loc[mask,'data'] = wiktionary_merkmal_text[mask].data.str.replace("'", '')
#grid( wiktionary_merkmal_text, mask )

In [ ]:
# not any_nan
#wiktionary_merkmal_text['merkmal_order2'] = wiktionary_merkmal_text.merkmal_order2.fillna(0) 
assert not pak.any_nan(wiktionary_merkmal_text)
#pak.analyse_cols(wiktionary_merkmal_text)

In [ ]:
# Verwaiste löschen
mask = ~wiktionary_merkmal_text.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal_text,mask,0)
#grid(wiktionary_merkmal_text,mask)
#wiktionary_merkmal_text = pak.drop_rows(wiktionary_merkmal_text,mask)

In [ ]:
# meta == lemma
mask = wiktionary_merkmal_text.meta == wiktionary_merkmal_text.lemma
check_mask(wiktionary_merkmal_text,mask,0)
#wiktionary_merkmal_text.loc[mask,'meta'] = ''

In [ ]:
# data leer
mask = wiktionary_merkmal_text.data == ''
check_mask(wiktionary_merkmal_text,mask,0)
#wiktionary_merkmal_text = pak.drop_rows(wiktionary_merkmal_text,mask)

In [ ]:
# suche_debug
mask = wiktionary_merkmal_text.lemma.isin(suche_debug)
grid(wiktionary_merkmal_text,mask)
#wiktionary_lemma[mask].lemma

In [ ]:
# Was ist drin?
a = pak.analyse_freqs(wiktionary_merkmal_text,['merkmal','data'] )
grid(a)

## wiktionary_lemma =====

In [ ]:
# Anzahl der Lexeme untersuchen

mask = wiktionary_merkmal.is_lex == True
count_lex = pak.group_and_agg(wiktionary_merkmal[mask], 
              ['lemma_id', 'data' ],
              ['group',    'count'],
              ['',         'lex_anz'   ],                              
             )

# eintragen
wiktionary_lemma['lex_anz'] = 0
wiktionary_lemma = pak.update_col(wiktionary_lemma, count_lex, on='lemma_id', col='lex_anz')

In [ ]:
# Anzahl der Nichtlexeme untersuchen

mask = wiktionary_merkmal.is_lex == False
count_lex = pak.group_and_agg(wiktionary_merkmal[mask], 
              ['lemma_id', 'data' ],
              ['group',    'count'],
              ['',         'nolex_anz'   ],                              
             )

# eintragen
wiktionary_lemma['nolex_anz'] = 0
wiktionary_lemma = pak.update_col(wiktionary_lemma, count_lex, on='lemma_id', col='nolex_anz')

In [ ]:
# Dups
mask = wiktionary_lemma.duplicated(subset='lemma_id', keep=False)
grid(wiktionary_lemma,mask, color='green')

In [ ]:
raise Stop

In [ ]:
member überprüfen wie in d01

## wiktionary_merkmal ==========

In [ ]:
# verwaiste wiktionary_merkmal data_id?
mask1 =  wiktionary_merkmal.data_id != ''
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal,mask, 0)

In [ ]:
#a = pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','data_id'])
#grid(a)

In [ ]:
# Fehlende data_id?
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2 
check_mask(wiktionary_merkmal,mask,0, 10)

### Überschneidungen
* Lexeme, die sich in translate_lex finden aber auch direkt in wiktionary_lemma >> entweder oder!

In [ ]:
# mask_LL erstellen
mask1 = pak.isin(translate_lex, wiktionary_lemma, left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'])
mask2 = translate_lex.data != translate_lex.lemma
mask_LL = mask1  &  mask2
check_mask(translate_lex,mask_LL,0,1700)
grid(translate_lex[mask_LL].sort_values('data_score',ascending=False))

In [ ]:
# wiktionary_lemma belöschen: Abkürzungen wieder löschen
mask1 =  wiktionary_lemma.lemma.isin(translate_lex[mask_LL].data)
mask2 =  wiktionary_lemma.member.str.contains('Abkürzung')
mask3 = ~wiktionary_lemma.member.str.contains('manuell')
mask4 =  wiktionary_lemma.page_part == -1
#mask5 = ~wiktionary_lemma.lemma.isin(['pos',])
mask = mask1  &  mask2  &  mask3  &  mask4  
check_mask(wiktionary_lemma, mask, 0, 150)

msg = 'Überschneidungen_1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(wiktionary_lemma, wiktionary_lemma_trash, mask, msg)
grid(wiktionary_lemma_trash,msg)

In [ ]:
# wiktionary_merkmal: verwaiste_Lemmas löschen
mask1 =  wiktionary_merkmal.merkmal == 'lemma'
mask2 =  wiktionary_merkmal.data_id != ''
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3 &  mask4
check_mask(wiktionary_merkmal,mask,0,150)

msg = 'verwaiste_Lemmas'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

In [ ]:
# translate_lemma lem_id
mask = ~translate_lemma.lem_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0,150)
# translate_lemma[mask]

msg = 'verwaist_lem_id'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
grid(translate_lemma_trash,msg)

In [ ]:
# translate_lemma lemma_id
mask = ~translate_lemma.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0)
# translate_lemma[mask]

#msg = 'verwaist_lemma_id'
#translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
#grid(translate_lemma_trash,msg)

In [ ]:
# Löschen: Einige sonstige Verwaisungen
mask1 =  wiktionary_merkmal.data_id != ''
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal,mask,0,30)
#grid(wiktionary_merkmal,mask)

msg = 'verwaist_data_id_24'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

In [ ]:
# TODO Ungelöste Probleme 
mask1 = pak.isin(translate_lex, wiktionary_lemma, left_on=['data','lemma_tagZZ'], right_on=['lemma','tagZZ'])
mask2 = translate_lex.data != translate_lex.lemma
mask_LL = mask1  &  mask2
check_mask(translate_lex,mask_LL,0,1500)
grid(translate_lex[mask_LL].sort_values('data_score',ascending=False))

## translate_lex.data_tag 

In [ ]:
# temp
mask1 = translate_lex.data  == 'ausgerutscht'
mask2 = translate_lex.lemma == 'ausrutschen'
mask = mask1  &  mask2
translate_lex[mask1]

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Ackermann'
translate_lex[mask]

In [ ]:
# Beispiel anzeigen
mask = translate_lemma.lem == 'Ackermann'
translate_lemma[mask]

In [ ]:
# pflege_translate_tabelle
#from x32_Funktionen import *
translate_lex   = pflege_translate_tabelle(translate_lex,   translate_tagZ, wiktionary_lemma)
translate_lemma = pflege_translate_tabelle(translate_lemma, translate_tagZ, wiktionary_lemma, translate_lex )

In [ ]:
# Nachschlagetabelle vorbereiten
spalten = ['data_id','data','data_tag','lemma_id','lemma','lemma_tag','lemma_tagZZ']
mask = wiktionary_merkmal.is_lex == True
df = wiktionary_merkmal[mask][spalten].drop_duplicates().sort_values(spalten)
#df

In [ ]:
# Prüfe Anzahl data_tag
mask = translate_lex.data_tag != ''
check_mask(translate_lex,mask,562350)

In [ ]:
# Vorhandene data_tag übertragen
# sonst leeren
translate_lex['data_tag_0'] = translate_lex.data_tag
translate_lex['data_tag_1'] = translate_lex.data_tag
translate_lex = pak.move_cols(translate_lex, ['data_tag','data_tag_0','data_tag_1'],'data')

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'Ackermann'
translate_lex[mask]

In [ ]:
# data_tag übertragen 
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma_id'],            col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma_id'],            col='data_tag', col_rename='data_tag_1', func='last',  cond='null')
           
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_id'],               col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_id'],               col='data_tag', col_rename='data_tag_1', func='last',  cond='null')
  
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tag'],   col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tag'],   col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tagZZ'], col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data_id','lemma','lemma_tagZZ'], col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

print()
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tag'],      col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tag'],      col='data_tag', col_rename='data_tag_1', func='last',  cond='null')
   
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tagZZ'],    col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma','lemma_tagZZ'],    col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tag'],              col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tag'],              col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tagZZ'],            col='data_tag', col_rename='data_tag_0', func='first', cond='null')
translate_lex = pak.update_col(translate_lex, df, on=['data','lemma_tagZZ'],            col='data_tag', col_rename='data_tag_1', func='last',  cond='null')

In [ ]:
# data_tag fehlt
maskA = translate_lex.data_tag_0 == ''
maskB = translate_lex.data_tag_1 == ''
mask = maskA | maskB
check_mask(translate_lex,mask,356699)
translate_lex.loc[mask,'data_tag']   = translate_lex[mask].lemma_tag
translate_lex.loc[mask,'data_tag_0'] = translate_lex[mask].lemma_tag
translate_lex.loc[mask,'data_tag_1'] = ''
#grid(translate_lex,mask)

In [ ]:
# data_tag eindeutig
mask = translate_lex.data_tag_0 == translate_lex.data_tag_1
check_mask(translate_lex,mask,804272)
translate_lex.loc[mask,'data_tag']   = translate_lex[mask].data_tag_0
translate_lex.loc[mask,'data_tag_0'] = translate_lex[mask].lemma_tag
translate_lex.loc[mask,'data_tag_1'] = ''
#grid(translate_lex,mask)

In [ ]:
# data_tag uneindeutig
mask1 = translate_lex.data_tag_0 != translate_lex.data_tag_1
mask2 = translate_lex.data_tag_0 != ''
mask3 = translate_lex.data_tag_1 != ''
mask = mask1  &  mask2  &  mask3
check_mask(translate_lex,mask, 26421)
translate_lex.loc[mask,'data_tag']   = translate_lex[mask].data_tag_0 + ' ' + translate_lex[mask].data_tag_1
#grid(translate_lex,mask)

In [ ]:
# Sicherstellen: data_tag fehlt nirgens komplett
mask = translate_lex.data_tag == ''
check_mask(translate_lex,mask,0)
#grid(translate_lex,mask)

In [ ]:
# nach Priorität sortieren (laut Steuertabellen)
translate_lex['data_tag'] = tag_sortieren(translate_lex.data_tag)

In [ ]:
# translate_lex vorbereiten
translate_lex = pak.drop_cols(translate_lex,['data_tag_0','data_tag_1','data_tagZZ'])    
translate_lex = pak.rename_col(translate_lex,'data_tag','tag')    

In [ ]:
# pflege_tag_felder
translate_lex = pflege_tag_felder(translate_lex, translate_tagZ)

In [ ]:
# # translate_lex nachbereiten
translate_lex = pak.rename_col(translate_lex,'tag',   'data_tag')    
translate_lex = pak.rename_col(translate_lex,'tag_0', 'data_tag_0')   
translate_lex = pak.rename_col(translate_lex,'tag_1', 'data_tag_1')   
translate_lex = pak.rename_col(translate_lex,'tagZZ', 'data_tagZZ')   
translate_lex = pak.drop_cols(translate_lex,['tagZ'])    
grid(translate_lex,pak.sample)

In [ ]:
mask = translate_lex.data == 'vervollständigt'
translate_lex[mask]

In [ ]:
#mask = wiktionary_lemma.tag.str.contains(' ')
#wiktionary_lemma[mask]

In [ ]:
# Beispiel anzeigen
mask = translate_lex.data == 'lachen'
translate_lex[mask]

In [ ]:
# temp
mask1 = translate_lex.data  == 'ausgerutscht'
mask2 = translate_lex.lemma == 'ausrutschen'
mask = mask1  &  mask2
translate_lex[mask]

## Prüfen ==========

### Unique

In [ ]:
# wiktionary_lemma: lemma_id vorhanden und eindeutig?
mask = wiktionary_lemma.lemma_id == ''
check_mask(wiktionary_lemma,mask,0)

assert wiktionary_lemma.lemma_id.is_unique

In [ ]:
# wiktionary_merkmal: data_id vorhanden und nicht widersprüchlich?
mask1 = wiktionary_merkmal.data_id == ''
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

# Vergebene data_id nicht widersprüchlich?
mask_notempty = wiktionary_merkmal.data_id != ''
a = pak.same_but_different(wiktionary_merkmal[mask_notempty],same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

In [ ]:
# translate_lex: data_id vorhanden und eindeutig?
mask = translate_lex.data_id == ''
check_mask(translate_lex,mask,0)

#assert translate_lex.data_id.is_unique

a = pak.same_but_different(translate_lex,same='data_id',different='data')
assert a.shape[0] == 0
#grid(a)

### Mögliche Probleme

In [ ]:
# Mögliche Probleme
spalten = ['data','lemma','lemma_score']
mask =  translate_lex['same_lexlemma'] < 0.3
a = translate_lex[mask][spalten].sort_values(['lemma_score'], ascending=[False])
gridt(a)

In [ ]:
mask = translate_lex.same_lexlemma < 0.5
a = pak.analyse_freqs(translate_lex[mask],['lemma','data'])
gridt(a)

### Verwaisungen

In [ ]:
# translate_lex lemma_id
mask = ~translate_lex.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lex,mask,0,160)

msg = 'verwaist_lemma_id_endkontrolle'
translate_lex, translate_lex_trash = pak.move_rows( translate_lex, translate_lex_trash, mask, msg )
grid(translate_lex_trash,msg)

In [ ]:
# translate_lemma lem_id
mask = ~translate_lemma.lem_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0,32)
# translate_lemma[mask]

msg = 'verwaist_lem_id_endkontrolle'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
grid(translate_lemma_trash,msg)

In [ ]:
# translate_lemma lemma_id
mask = ~translate_lemma.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(translate_lemma,mask,0)
# translate_lemma[mask]

msg = 'verwaist_lemma_id_endkontrolle'
translate_lemma, translate_lemma_trash = pak.move_rows( translate_lemma, translate_lemma_trash, mask, msg )
grid(translate_lemma_trash,msg)

In [ ]:
# wiktionary_lemma.data_id (nur leeren)
mask1 =  wiktionary_lemma.data_id != ''
mask2 = ~wiktionary_lemma.data_id.isin(wiktionary_lemma.lemma_id)
mask3 = ~wiktionary_lemma.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_lemma,mask,1)

wiktionary_lemma.loc[mask,'data_id'] = ''
grid(wiktionary_lemma,mask)

In [ ]:
# wiktionary_merkmal lemma_id
mask = ~wiktionary_merkmal.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal,mask,0,160)
# wiktionary_merkmal[mask]

msg = 'verwaist_lemma_id_endkontrolle'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

In [ ]:
# verwaist wiktionary_merkmal data_id?
mask1 =  wiktionary_merkmal.data_id != ''
mask2 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask3 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask4 = wiktionary_merkmal.data_id.isin(translate_lex_trash.data_id)
mask = mask1  &  mask2  &  mask3 # &  mask4
check_mask(wiktionary_merkmal,mask)

#grid(wiktionary_merkmal,mask)

In [ ]:
# wiktionary_merkmal data_id
mask1 =  wiktionary_merkmal.data_id != ''
mask2 =  wiktionary_merkmal.is_lex == True
mask3 = ~wiktionary_merkmal.data_id.isin(wiktionary_lemma.lemma_id)
mask4 = ~wiktionary_merkmal.data_id.isin(translate_lex.data_id)
mask = mask1  &  mask2  &  mask3  &  mask4
check_mask(wiktionary_merkmal,mask,0,40)

msg = 'verwaist_data_id_endkontrolle'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg )
grid(wiktionary_merkmal_trash,msg)

In [ ]:
#grid(wiktionary_merkmal,mask)

In [ ]:
#pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','data_id'])

In [ ]:
# wiktionary_merkmal_text lemma_id
mask = ~wiktionary_merkmal_text.lemma_id.isin(wiktionary_lemma.lemma_id)
check_mask(wiktionary_merkmal_text,mask,0)
# wiktionary_merkmal_text[mask]

#msg = 'verwaist_lemma_id_endkontrolle'
#wiktionary_merkmal_text, df = pak.move_rows( wiktionary_merkmal_text, mask, msg )
#grid(df)

### NaN

In [ ]:
assert not pak.any_nan(wiktionary_lemma)
#pak.nnan(wiktionary_lemma)

In [ ]:
assert not pak.any_nan(wiktionary_merkmal)
#pak.nnan(wiktionary_merkmal)

In [ ]:
assert not pak.any_nan(wiktionary_merkmal_text)
#pak.nnan(wiktionary_merkmal_text)

In [ ]:
assert not pak.any_nan(translate_lemma)
#pak.nnan(translate_lemma)

In [ ]:
assert not pak.any_nan(translate_lex)
#pak.nnan(translate_lex)

### Div

In [ ]:
# lemma_id und data_id müssen gleich sein
mask1 = wiktionary_merkmal.merkmal == 'lemma'
mask2 = wiktionary_merkmal.lemma_id != wiktionary_merkmal.data_id
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

In [ ]:
# lemma und data müssen gleich sein
mask1 = wiktionary_merkmal.merkmal == 'lemma'
mask2 = wiktionary_merkmal.lemma != wiktionary_merkmal.data
mask = mask1  &  mask2
check_mask(wiktionary_merkmal,mask,0)

### Partizipien prüfen

In [ ]:
#suche_L = 'missraten' 
#suche_P = 'missraten'  

suche_L = 'gefriertrocknen' 
suche_P = 'gefriergetrocknet'  

In [ ]:
# wiktionary_merkmal
mask1 = wiktionary_merkmal.is_lex == True
mask2 = wiktionary_merkmal.data  == suche_P
mask = mask1  &  mask2
wiktionary_merkmal[mask]

In [ ]:
# das Verb in wiktionary_lemma
mask1 = wiktionary_lemma.lemma == suche_L
mask2 = wiktionary_lemma.tagZZ == 'V'
mask = mask1  &  mask2

suche_L_id = wiktionary_lemma[mask].iloc[0].lemma_id
wiktionary_lemma[mask]

In [ ]:
# das Adjektiv in wiktionary_lemma
mask1 = wiktionary_lemma.lemma == suche_P
mask2 = wiktionary_lemma.tagZZ == 'A'
mask3 = wiktionary_lemma.member.str.contains('Ptz')
mask4 = wiktionary_lemma.data_id == suche_L_id
mask = mask1  &  mask2  &  mask3  &  mask4

suche_P_id = wiktionary_lemma[mask].iloc[0].lemma_id
wiktionary_lemma[mask]

In [ ]:
# Das Ptz-Merkmal in wiktionary_lemma
mask1 = wiktionary_merkmal.data == suche_P
mask2 = wiktionary_merkmal.merkmal.str.startswith('Ptz')
mask3 = wiktionary_merkmal.lemma_tag.str.startswith('V')
mask4 = wiktionary_merkmal.lemma_id == suche_L_id   # zeigt aufs Verb
mask5 = wiktionary_merkmal.data_id  == suche_P_id   # zeigt aufs Adjektiv
mask = mask1  &  mask2  &  mask3  &  mask4  &  mask5
wiktionary_merkmal[mask]

### Testlexeme

In [ ]:
# lex_test, lemma_test
from x32_Funktionen import *
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=3 )
#grid(lemma_test)

In [ ]:
# lemma_test
from x32_Funktionen import *
lemma_test = check_lemma_test(lemma_test, wiktionary_lemma)
#lemma_test

In [ ]:
# Fehler anzeigen
maskA = lemma_test.lemma_id == ''
maskB = lemma_test.check_tag == ''
maskC = lemma_test.check_member == ''
mask = maskA | maskB | maskC
check_mask(lemma_test,mask)

probleme = list(lemma_test[mask].lemma)
lemma_test[mask]

In [ ]:
# Sicherstellen: Keine Fehler
check_mask(lemma_test,mask,0)

### lex_test

In [ ]:
# lex_test
from x32_Funktionen import *
lex_test = check_lex_test(lex_test, translate_lex)
#lex_test

In [ ]:
# Fehler
maskA = lex_test.lex_id == ''
maskB = lex_test.check_tag == ''
mask = maskA | maskB 
check_mask(lex_test,mask)

probleme = list(lex_test[mask].lemma)
lex_test[mask]

In [ ]:
# Sicherstellen: Keine Fehler
check_mask(lex_test,mask,0)

In [ ]:














bgc('green')

### Fehler analysieren

In [ ]:
sucheL = ['Bussi']
suche  = ['Bussi']

In [ ]:
mask = lex_test.lex.isin(suche)
lex_test[mask]

In [ ]:
mask = translate_lex.data.isin(suche)
translate_lex[mask]

In [ ]:
mask = translate_lex_trash.data.isin(sucheL)
translate_lex_trash[mask]

In [ ]:
mask = translate_lemma.lem.isin(suche)
translate_lemma[mask]

In [ ]:
mask = wiktionary_lemma.lemma.isin( sucheL + suche )
wiktionary_lemma[mask]

In [ ]:
mask1 = wiktionary_merkmal.data.isin(suche)
mask2 = wiktionary_merkmal.is_lex == True
mask = mask1  &  mask2
wiktionary_merkmal[mask].sort_values(['lemma_id','merkmal','data'])

In [ ]:
grid(lex_test)

In [ ]:
raise Stop

## Speichern ==========

In [ ]:
assert gründlich

In [ ]:
# Speichern  
pak.dump_pickle(wiktionary_lemma,         wiktionary_lemma_filename_save)   
pak.dump_pickle(wiktionary_lemma_fix,     wiktionary_lemma_fix_filename_save)   
pak.dump_pickle(wiktionary_merkmal,       wiktionary_merkmal_filename_save) 
pak.dump_pickle(wiktionary_merkmal_text,  wiktionary_merkmal_text_filename_save)  
pak.dump_pickle(translate_lemma,          translate_lemma_filename_save) 
pak.dump_pickle(translate_lex,            translate_lex_filename_save)
pak.dump_pickle(translate_lex_trash,      translate_lex_trash_filename_save)
os.utime(verzeichnis_save)

In [ ]:
# drop_cols
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal, ['data_id_old','data_id_bak'])

In [ ]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} /content/drive/MyDrive

In [ ]:
raise Stop

## suche_error ==========

In [ ]:
# Veränderungen lemma_tag >> data_tag
# nach Merkmal
mask1 = wiktionary_merkmal.data_tag != wiktionary_merkmal.lemma_tag
mask2 = wiktionary_merkmal.data_tag != ''
mask = mask1  &  mask2
pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','lemma_tag','data_tag',])

In [ ]:
suche_debug = [suche_error[1]]
suche_debug

In [ ]:
# suche_debug wiktionary_lemma
mask = wiktionary_lemma.lemma.isin(suche_debug)
(wiktionary_lemma[mask])

In [ ]:
# suche_debug wiktionary_lemma_trash
mask = wiktionary_lemma_trash.lemma.isin(suche_debug)
(wiktionary_lemma_trash[mask])

In [ ]:
# translate_lex
search_str(translate_lex, suche_debug)

In [ ]:
# suche_debug translate_lemma
search_str(translate_lemma, suche_debug)

In [ ]:
# wiktionary_merkmal
mask = wiktionary_merkmal.lemma.isin(suche_debug)
grid(wiktionary_merkmal,mask)

In [ ]:
# wiktionary_merkmal
mask = wiktionary_merkmal.data.isin(['besonne'])
grid(wiktionary_merkmal,mask)

In [ ]:
# suche_debug wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
a = search_str(wiktionary_merkmal[mask], suche_debug)
grid(a.sort_values(['lemma_id','merkmal']))

In [ ]:
# suche_debug wiktionary_merkmal
mask = wiktionary_merkmal.is_lex == True
a = search_str(wiktionary_merkmal[mask], suche_debug)
grid(a.sort_values(['lemma_id','merkmal']))

In [ ]:
# suche_debug wiktionary_merkmal_text
mask = wiktionary_merkmal.lemma_id.isin(suche_debug)
ids = set(wiktionary_merkmal[mask].lemma_id)
mask = wiktionary_merkmal_text.lemma_id.isin(ids)
grid(wiktionary_merkmal_text,mask)

In [ ]:
raise Stop

## Fragmente

In [ ]:
raise Stop

In [ ]:
# rename_col 
translate_lex = pak.rename_col(translate_lex, 'data_id',    'lex_id')
translate_lex = pak.rename_col(translate_lex, 'data_home',  'lex_home')
translate_lex = pak.rename_col(translate_lex, 'data',       'lex')
translate_lex = pak.rename_col(translate_lex, 'data_score', 'lex_score')

translate_lex_full = pak.rename_col(translate_lex_full, 'data_id',    'lex_id')
translate_lex_full = pak.rename_col(translate_lex_full, 'data_home',  'lex_home')
translate_lex_full = pak.rename_col(translate_lex_full, 'data',       'lex')
translate_lex_full = pak.rename_col(translate_lex_full, 'data_score', 'lex_score')

translate_lemma = pak.rename_col(translate_lemma, 'same_lexlemma', 'same_lemlemma')

In [ ]:
# rank!
#translate_lex = pak.rank(translate_lex, col_score='data_score', cols_group='data_id', on_conflict='first')

In [ ]:
# rank!
# Ein data_id pro data und lemma_tagZZ.
vorher = translate_lex.shape[0]
translate_lex = pak.rank(translate_lex, col_score='data_score', cols_group=['data','lemma_tagZZ'], on_conflict='first')
print(vorher - translate_lex.shape[0], 'Datensätze gelöscht')

In [ ]:
# Dups in data + lemma: Sind stets verschiedene lemma_tagZZ
mask = translate_lex.duplicated(subset=['data','lemma'], keep=False)
a = translate_lex[mask].sort_values('data_id')
#grid(a)

In [ ]:
# Konfiguration Dups entfernen
aggwords    =  pak.agg_words_nodup
col_origins =  [ 'data_id',  'data_id2', 'data_id_bak', 'data', 'data_tag', 'data_tagZZ', 'data_score', 'member',  'lemma_id', 'lemma', 'lemma_tag', 'lemma_tagZZ', 'lemma_lower', 'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ]
col_funcs   =  [ 'max',      'min',      'max',         '',     '',         '',           'sum',        aggwords,  '',         '',      '',          '',            '',            'sum',         'first',     'first',       'first',          'sum',   ]   
col_names   =  [ 'data_id',  'data_id2', 'data_id_bak', '',     '',         '',           'data_score', 'member',  '',         '',      '',          '',            '',            'lemma_score', 'data_home', 'lemma_home',  'same_lexlemma',  'score', ] 

In [ ]:
# Gruppieren!
translate_lex = pak.group_and_agg( translate_lex, col_origins, col_funcs, col_names, optimize=True)